In [ ]:
# Install dependencies that are not included in the environment
# Please make sur to have selected the right Python environment/kernel (mapping_RI_env) before running these commands
%pip install factor_analyzer
%pip install seaborn
%pip install PyWavelets
%pip install cmasher
%pip install fsspec

In [ ]:
import os
import re
import h5py
from h5py import string_dtype
import numpy as np
import scipy.io
from scipy.io import loadmat
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt, windows
from scipy.signal import filtfilt, windows, detrend
from factor_analyzer import FactorAnalyzer
import random
import os
import pandas as pd
from scipy.signal import csd, detrend
import pickle
from scipy.signal import welch
from scipy.fft import fft, fftfreq
import sys
import seaborn as sns
import re
import gc  # Import Python's garbage collector
import itertools
from scipy.optimize import least_squares
import csv
import itertools
from collections import defaultdict
import scipy.stats as stats
import matplotlib as gridspec
from matplotlib.gridspec import GridSpec
from IPython.display import display
from dataclasses import dataclass
import numpy.ma as ma
import pywt
from scipy.signal import butter, sosfiltfilt, windows
from sklearn.decomposition import NMF
import cmasher as cmr

In [ ]:
import warnings
warnings.filterwarnings("ignore", message=".*LOKY_MAX_CPU_COUNT.*")
warnings.filterwarnings("ignore", message=".*force_all_finite.*")
warnings.filterwarnings("ignore", message="No rotation will be performed when the number of factors equals 1.")

In [ ]:
path_of_files = "C://Users//franc//Documents//Mapping_Recurrent_Inhibition_minimal_data_to_run_scripts//one_example_experimental_MUedit_file//"
# ^ Replace this path with the path to the folder containing the .mat files to analyze.
# Running it on all files in the folder is quite long, so do not hesitate to create a subfolder with only one or two files to test the script first.
export_processed_spike_trains_as_hdf5_file_for_new_analysis = True

files_to_analyze_experimental_data = [f for f in os.listdir(path_of_files) if f.endswith('.mat')]
input_fsamp = 2048 # for experimental data
# Assuming grids of 13x5
# For nb of grids, check parameters a bit below
ISI_thresh_for_discontinuity = 1.0 # 0.4 # in s
single_or_double_diff_EMG = 'single' # 'single' or 'double' ; returns monopolar EMG if anything else
display_figures_inline = True # If false, no figure will be displayed when running the script.
# Seeting display_figures_inline = True is recommended if "select_plateaus_manually = True" because when this variable is set to True, all figures appear in new windows (which is quickly overwhelming)

# Force/torque processing and plateau selection parameters
force_is_3D = False # set to "True" to recover all force components (when recorded with a 3D force sensor)
select_plateaus_manually = False # Select plateaus manually (or not)
force_lowpass_cutoff = 5 # 100 # Hz # 0.33
force_differential_epsilon_up_or_down = 0.0001 # 0.0001 (not used if select_plateaus_manually = True)
if select_plateaus_manually:
    %matplotlib qt
    # Use an interactive backend
else:
    %matplotlib inline
    # Use an inline backend

# Initialize a dictionary with lists as default values
conditions_per_subject = defaultdict(list)

# Process filenames
for filename in files_to_analyze_experimental_data:
    subject, condition = filename.replace('.mat', '').split('_', 1)  # Split subject and condition
    conditions_per_subject[subject].append(condition)

# Convert defaultdict to a regular dictionary
conditions_per_subject = dict(conditions_per_subject)

# # # # CHOOSE NUMBER OF MUs TO RANDOMLY SELECT FOR ANALYSES IN THE CASE OF SIMULATED DATA
select_all_MNs_from_simulation = True # if False, a random sample of MUs will be selected
sim_nb_MUs_to_select_per_pool = 50 # Both continuous and/or discontinuous MUs may get selected
sim_ignore_discontinuous_MUs = True # if True, only continuous MUs will be selected for the analyses

# # # # # CHOOSE WHICH ANALYSES TO PERFORM # # # # # 
# # # # # All the analyses below correspond to analyses on experimental data that are NOT included in the paper - the main purpose of this script is to process experimental spike trains data and to create a usable .hd5 file for the "analyzer.py" script. They remain available here.
perform_analysis_dimensionality = False # True
perform_analysis_frequency_components_dimensionality = False
perform_analysis_coherence = False # True
perform_analysis_CST_PSD = False
perform_analysis_CST_torque_correlation = False # False
perform_analysis_recurrent_inhibition = False # True

# # # # # PARAMETERS FOR FORCE PREPROCESSING # # # # #
polynomial_degree_for_detrending = 1
force_smoothing_window_length = 1 # in s

# # # # # PARAMETERS FOR DIMENSIONALITY AND COH ANALYSIS # # # # # 
nb_plateaus_to_select = {}
nb_plateaus_to_select['10percent'] = 10
nb_plateaus_to_select['40percent'] = 20
nb_plateaus_to_select['flexion'] = 5 # only used for the 'fingers" file
nb_plateaus_to_select['abduction'] = 5 # only used for the 'fingers" file
# Minimum proportion of MUs to keep in the plateau nb selection - if no optimal selection can select enough MUs, then one less plateau is selected and the process repeats
min_proportion_of_MUs_to_keep = 0.5 # 0.33

# --------------------------------------
#       Factor analysis
# --------------------------------------
max_nb_of_factors = 6  # e.g., 6, but can be anything you need
surrogate_iter_nb = 10 # 1
# Setting the smoothing kernel for the smoothed discharge rates
Wind_s = 0.4  # hanning window duration. 0.4 for 2.5hz low-pass, 0.2 for 5hz low-pass
HanningW = 2 / round(input_fsamp * Wind_s) * windows.hann(round(input_fsamp * Wind_s))  # unitary area
fa_on_first_pool_only_for_sim = True
display_Rsquared_for_muscle_pair = False

# --------------------------------------
#       Coherence
# --------------------------------------
coh_window_length = 1 # in s
coh_windows_overlap = 0.5 # % relative to coh_window_length
# frequency resolution will be = 1/coh_window_length. If coh_window_length = 10s for instance, resolution will be 0.1hz
upsampling_frequency_resolution = 1 # 5 # for the 'nfft' parameter of scipy.csd(). Interpolates the coherence values over the frequencies, to smooth the figure a bit.
# For example, if the resolution defined by coh_window_length is 1hz, and if upsampling_frequency_resolution = 2, the resolution will be upsampled to 0.5hz
max_freq = 150 # maximum frequency that is kept
COH_calc_max_iteration_nb_per_group_size = 100 # More iteration for smaller group sizes, because the value obtained is very dependent upon the exact neurons selected, especially when only a few MNs are used to create the CST

# --------------------------------------
#       CST-torque correlation
# --------------------------------------
low_pass_freqs_to_try = [2.5] # [1, 2.5, 5, 10, 50]  # in Hz
arbitrary_lowpass_freq = 2.5   # must be in low_pass_freqs_to_try
edges_to_remove = (1/np.min(low_pass_freqs_to_try)) + 0.5  # seconds
edges_to_remove_samples = int(edges_to_remove * input_fsamp)
cross_correl_max_lag = int(0.6*input_fsamp)  # in samples - 0.6s is 300ms each side
# Force low-pass filter cutoff (Hz)
force_lowpass_cutoff = 2.5 # 2.5  
# Display choice: 'all' or list of muscles you want to show
CST_torque_corr_display_only_selected_muscles = 'all' # 'all' # ['VL']

#
# Experimental data parameters
nb_grids_origin = {}
# Make sure the names match with the filename conditions
nb_grids_origin['Quadriceps'] = 6
nb_grids_origin['TA'] = 4
nb_grids_origin['TricepsSurae'] = 6
nb_grids_origin['Fingers'] = 2
nb_grids_origin['FDI'] = 1
TA_grids = [0,1,2,3]
TA_color = 'green'
SOL_grids = [0,1]
SOL_color = 'blue'
GM_color = 'purple'
GM_grids = [2,3,4,5]
VM_grids = [0,1]
VL_grids = [2,3,4,5]
VM_color = 'orange'
VL_color = 'red'
FDI_grids = [0]
APB_grids = [1]
FDI_color = 'turquoise'
APB_color = 'pink'
NA_color = 'gray'

# Define the colors for the grids
grid_colormap = plt.colormaps.get_cmap('plasma')
# Generate 6 evenly spaced values between 0 and 1
n_colors = max(nb_grids_origin.values())
indices = np.linspace(0, 1, n_colors)
# Generate colors
grid_colors = [grid_colormap(index) for index in indices]


In [ ]:
import matplotlib.colors as mcolors

def mix_colors(color1, color2):
    """
    Mix two colors and return their average as a hex string.
    
    :param color1: First color (name or hex string).
    :param color2: Second color (name or hex string).
    :return: Mixed color as a hex string.
    """
    # Convert colors to RGB tuples in [0,1]
    rgb1 = mcolors.to_rgb(color1)
    rgb2 = mcolors.to_rgb(color2)
    
    # Compute the average of the two RGB tuples.
    mixed_rgb = [(c1 + c2) / 2 for c1, c2 in zip(rgb1, rgb2)]
    
    # Convert the averaged RGB back to a hex string.
    return mcolors.to_hex(mixed_rgb)

def modify_color(color, rgb_factor=0.5, rgb_constant=0.0):
    if type(rgb_factor) == float:
        rgb_factor = [rgb_factor] * 3
    if type(rgb_factor) == float:
        rgb_constant = [rgb_constant] * 3

    # Convert the color to an RGB triplet (values between 0 and 1)
    rgb = np.array(mcolors.to_rgb(color))  

    # Apply the modification (e.g., divide by 2 to darken)
    modified_rgb = np.clip(rgb * rgb_factor + rgb_constant, 0, 1)  # Ensure values stay between 0 and 1

    # Convert back to a hex string
    return mcolors.to_hex(modified_rgb)


In [ ]:
# Function to recursively navigate and load data from the .mat file
def load_file_h5py(file_path):
    mat_data = {}

    def resolve_reference(reference):
        """Helper function to resolve HDF5 references."""
        if isinstance(reference, h5py.Reference):
            return f[reference][()]  # Dereference and load the actual data
        return reference  # Otherwise, return the original data

    def flatten_and_dereference(nested_array):
        """
        Flatten any nested object arrays and resolve references.
        Recursively dereference any references stored in the array.
        Returns a list of lists instead of a NumPy array to avoid broadcasting errors.
        """
        if isinstance(nested_array, np.ndarray) and nested_array.dtype == 'O':  # Check for object (cell array)
            dereferenced = []
            for item in nested_array:
                if isinstance(item, h5py.Reference):
                    dereferenced.append(resolve_reference(item))
                elif isinstance(item, np.ndarray) and item.dtype == 'O':
                    dereferenced.append(flatten_and_dereference(item))  # Recursively flatten/dereference
                elif isinstance(item, np.ndarray):
                    dereferenced.append(item)  # Keep normal NumPy arrays
                else:
                    dereferenced.append(item)
            return dereferenced  # Return as a Python list, NOT a NumPy array
        return nested_array

    def process_spike_times(spike_times):
        """
        Process spike_times and return a dictionary with grid numbers as keys and lists of arrays of spike times as values.
        Also, ensure that empty arrays are represented as array([], dtype=float64).
        """

        # Ensure spike_times is always a 2D structure (list of lists)
        if isinstance(spike_times, np.ndarray) and spike_times.ndim == 1:  # If only 1 row (1xM), reshape to 2D
            spike_times = spike_times.reshape(1, -1)

        spike_dict = {}

        for i in range(len(spike_times)):  # Iterate over grids
            spike_dict[i] = []  # Initialize list for each grid
            for j in range(len(spike_times[i])):  # Iterate over motor units
                spike_array = spike_times[i][j]
                if isinstance(spike_array, np.ndarray):
                    # Replace array([0., 0.]) with empty array if it contains [0., 0.]
                    if np.array_equal(spike_array, np.array([0., 0.])):
                        spike_dict[i].append(np.array([], dtype=np.float64))
                    else:
                        # Convert to 1D array (flatten) and ensure dtype is float64
                        spike_dict[i].append(np.ravel(spike_array).astype(np.float64))
                else:
                    # Ensure it's an empty array for invalid or empty data
                    spike_dict[i].append(np.array([], dtype=np.float64))

        return spike_dict
    
    def process_channels_mask(channels_mask):
        """
        Process channels_mask to ensure it is an array of uint8 arrays (one per grid).
        """
        mask_list = []

        for i in range(len(channels_mask)):  # Iterate over grids
            mask_array = channels_mask[i]

            # Ensure the mask is treated as a 1D array and cast to uint8
            if isinstance(mask_array, np.ndarray):
                mask_array = mask_array.flatten().astype(np.uint8)
            mask_list.append(mask_array)

        return np.array(mask_list, dtype=object)  # Return as object array of uint8 arrays


    # Open the .mat file and access the necessary fields
    with h5py.File(file_path, 'r') as f:
        # Accessing 'signal' struct fields
        signal = f['signal']
        emg_data = signal['data'][()]  # Extract the full EMG data without slicing
        force_data = signal['path'][()]      # Load force data
        target_data = signal['target'][()]   # Load target data
        channels_mask = signal['EMGmask'][()]  # Load EMG mask as ndarray

        # Accessing 'edition' struct field for spike times
        edition = f['edition']
        spike_times = edition['Dischargetimes'][()]

        # Flatten and dereference spike_times and channels_mask
        spike_times = flatten_and_dereference(spike_times)

        # Convert list of lists into a NumPy object array before transposing
        spike_times = np.array(spike_times, dtype=object)
        # Ensure spike_times is always 2D (fixes issue when only 1 grid is present)
        if spike_times.ndim == 1:
            spike_times = spike_times.reshape(1, -1)
        # Now it's safe to transpose
        spike_times = process_spike_times(spike_times.T)
        
        # Process channels_mask to ensure each grid has its own uint8 array
        channels_mask = flatten_and_dereference(channels_mask)
        channels_mask = process_channels_mask(channels_mask)

        # Store the results in the dictionary for easy access
        mat_data['EMG'] = emg_data.T

        if mat_data['EMG'].shape[0] > (64 * nb_grids):
            mat_data['force'] = mat_data['EMG'][(64 * nb_grids), :]
            if force_is_3D:
                # plt.figure()
                # plt.title(f"{current_subject} - {current_condition} - 3D force")
                mat_data['force_3D'] = np.array([
                    mat_data['EMG'][(64 * nb_grids), :],
                    mat_data['EMG'][(64 * nb_grids)+1, :],
                    mat_data['EMG'][(64 * nb_grids)+2, :]])
                # plt.plot(mat_data['force_3D'][0,:],label="Force Z")
                # plt.plot(mat_data['force_3D'][1,:],label="Force X")
                # plt.plot(mat_data['force_3D'][2,:],label="Force Y")
                # mat_data['force_3D_magnitude'] = np.sqrt(np.sum(np.array(mat_data['force_3D'])**2,axis=0))
                # plt.plot(mat_data['force_3D_magnitude'],label="3D force magnitude",color="k")
                # plt.xlabel("Time (samples)")
                # plt.ylabel("Force (raw values)")
                # plt.legend()
                # if display_figures_inline:
                #     plt.show()
                # else:
                #     plt.close()
        else:
            mat_data['force'] = np.squeeze(target_data)

        # Check if force is reversed
        mean_center_force = np.mean(mat_data['force'])
        if np.abs(np.min(mean_center_force)) > np.abs(np.max(mean_center_force)):
            mat_data['force'] = -mat_data['force']

        mat_data['target'] = np.squeeze(target_data)
        mat_data['EMG'] = mat_data['EMG'][np.arange((64 * nb_grids)), :]  # Extracting EMG data (first (64*nb_grids) rows)
        mat_data['channels_mask'] = channels_mask  # Store channels_mask as object array of uint8 arrays
        mat_data['spike_times'] = spike_times  # Store spike_times as a dict

    # Clean up large variables to free memory
    gc.collect()

    return mat_data


In [ ]:
# Function to determine whether to use h5py or scipy based on the file format
def load_file(file_path):
    print(f"Loading file: {file_path}")
    try:
        print("Trying to load as an HDF5 format")
        # Try to open as an HDF5-based v7.3 file
        return load_file_h5py(file_path)
    except OSError as e:
        print("Error loading file as HDF5 format")
        print(f"OSError: {e}")
        return None

In [ ]:
def reorganize_emg(emg_data, channels_mask):
    """
    Reorganizes the EMG numpy array to reflect the spatial organization of electrodes, sets certain channels to NaN,
    and computes the differential EMG.

    Parameters:
    emg_data (np.ndarray): The EMG data array with shape ((64*nb_grids), N) where (64*nb_grids) is the number of channels.
    channels_mask (np.ndarray): A mask indicating which channels should be set to NaN (1 = NaN).

    Returns:
    np.ndarray: A 4D array of shape (N, 13-M, 5, N-1) representing the processed EMG for N grids, with 13-1 rows for differential EMG, or 13-2 rows for double differential EMG.
    """
    # Get the number of samples from the EMG data
    num_samples = emg_data.shape[1]  # N samples
    
    # Initialize a 4D array to store the reorganized data: (6 grids, 13 rows, 5 columns, N samples)
    num_grids = nb_grids
    grid_rows = 13
    grid_cols = 5
    
    # Output array to store the reorganized data
    reorganized_emg = np.zeros((num_grids, grid_rows, grid_cols, num_samples))
    
    # Define the channel mapping for one grid based on the raster scan pattern
    channel_map = []
    for col in range(grid_cols):
        if col % 2 == 0:
            # Even columns: top to bottom (excluding the first row, which is missing)
            channel_map.extend([(row, col) for row in range(grid_rows-1, 0, -1)])  # Rows 12 -> 1
        else:
            # Odd columns: bottom to top (excluding the first row, which is missing)
            channel_map.extend([(row, col) for row in range(1, grid_rows)])  # Rows 1 -> 12

    # Ensure channels_mask is a proper NumPy array
    channels_mask = np.array(channels_mask, dtype=object)
    
    # Reorganize data for each grid and apply the channel_mask
    for grid in range(num_grids):
        for channel, (row, col) in enumerate(channel_map):
            # Place the EMG data into the correct location in the 4D array
            channel_data = emg_data[grid * 64 + channel, :]
            
            mask_array = np.array(channels_mask[grid]).flatten()  # Ensure it's a 1D array

            if channel < len(mask_array):  # Prevent index out of bounds
                mask_value = mask_array[channel]  
            else:
                mask_value = 0  # Default to 0 (not masked) if out of bounds

            if np.isscalar(mask_value):  # Ensure it's a scalar
                mask_value = int(mask_value)  # Convert to integer if necessary
            else:
                mask_value = int(mask_value[0])  # Extract scalar value if it's an array

            if mask_value == 1:
                reorganized_emg[grid, row, col, :] = np.nan
            else:
                reorganized_emg[grid, row, col, :] = channel_data

        # Set the missing first channel in each grid to NaN
        reorganized_emg[grid, 0, 0, :] = np.nan

    # Compute differential EMG (difference along the columns, reducing the number of rows by 1)
    if single_or_double_diff_EMG == 'single':
        differential_emg = np.diff(reorganized_emg, axis=1)
    elif single_or_double_diff_EMG == 'double':
        differential_emg = np.diff(reorganized_emg, axis=1)
        differential_emg = np.diff(differential_emg, axis=1)
        # use double differential EMG ==== commented out means we are using single differential only
        # Apply the Laplacian kernel along the rows (axis=1), equivalent to using diff() twice
        # double_diff_kernel = np.array([1, -2, 1])
        # differential_emg = convolve1d(reorganized_emg, double_diff_kernel, axis=1, mode='constant', cval=np.nan)
    else:
        print("'single_or_double_diff_EMG' is not set to 'single', nor 'double': returning monopolar EMG")
        differential_emg = reorganized_emg
        
    return differential_emg


In [ ]:
def realign_spike_times(emg_data, spike_times, sampling_rate=2048, display_muap=False, filename = 'unknown file'):
    """
    Estimate the MUAP onset time using spike-triggered averaging and realign spike times.
    
    Parameters:
    emg_data (np.ndarray): The reorganized EMG array of shape (6, 11, 5, N).
    spike_times (dict): A dictionary with keys corresponding to grid numbers and motor unit numbers,
                        containing arrays of spike times.
    sampling_rate (int): The sampling rate of the EMG data (default: 2048 Hz).
    display_muap (bool): Whether to display the MUAP window, spike times, threshold, and selected channel.
    
    Returns:
    dict: A dictionary of realigned spike times.
    """
    # Spike-triggered averaging window
    pre_spike_window = int(40 / 1000 * sampling_rate)  # 40 ms in samples
    post_spike_window = int(40 / 1000 * sampling_rate)  # 40 ms in samples (after spike)

    realigned_spike_times = {}
    
    # Colors for plotting based on grid index
    colors = ['b', 'g', 'r', 'c', 'm', 'y']
    if single_or_double_diff_EMG == 'double':
        nb_of_rows_temp = 11
    elif single_or_double_diff_EMG == 'single':
        nb_of_rows_temp = 12
    else:
        nb_of_rows_temp = 13

    for grid in data[current_subject][current_condition]['spike_times'].keys():
        realigned_spike_times[grid] = []
        
        for mu_idx, spikes in enumerate(spike_times[grid]):
            # Initialize spike-triggered average
            spike_triggered_avg = np.zeros((nb_of_rows_temp, 5, pre_spike_window + post_spike_window))

            # Count valid spikes
            valid_spike_count = 0
            
            # Perform spike-triggered averaging
            for spike in spikes:
                spike_idx = int(spike)  # Convert spike time to integer index
                if spike_idx - pre_spike_window >= 0 and spike_idx + post_spike_window < emg_data.shape[-1]:
                    spike_window = emg_data[grid, :, :, spike_idx - pre_spike_window:spike_idx + post_spike_window]
                    spike_triggered_avg += spike_window
                    valid_spike_count += 1

            if valid_spike_count > 0:
                spike_triggered_avg /= valid_spike_count  # Average the signal

                # Find the best channel based on peak-to-peak amplitude ratio
                best_channel_row, best_channel_col = None, None
                max_ratio = -np.inf

                for row in range(nb_of_rows_temp):
                    for col in range(5):
                        # Compute peak-to-peak amplitude for this channel
                        signal = spike_triggered_avg[row, col, :]
                        peak_to_peak = np.ptp(signal)

                        # Compute noise level in the [40, 20] ms window before the peak and [20, 40] ms after the peak
                        noise_window = signal[0:int(pre_spike_window * 0.5)]  # 40 to 20 ms before spike
                        noise_window = np.append(noise_window, signal[int(pre_spike_window + post_spike_window * 0.5):])  # 20 to 40 ms after spike
                        noise_std = np.std(noise_window)
                        noise_mean = np.mean(noise_window)

                        # Ratio of signal amplitude to noise
                        if noise_std > 0:
                            ratio = peak_to_peak / noise_std
                        else:
                            ratio = 0  # Avoid division by zero

                        # Track the best channel (largest signal-to-noise ratio)
                        if ratio > max_ratio:
                            max_ratio = ratio
                            best_channel_row, best_channel_col = row, col

                # Detect the onset of the MUAP for the best channel
                best_channel_signal = spike_triggered_avg[best_channel_row, best_channel_col, :]
                noise_window = best_channel_signal[0:int(pre_spike_window * 0.5)]  # 40 to 20 ms before spike and [20, 40] ms after the spike
                noise_window = np.append(noise_window, best_channel_signal[int(pre_spike_window + post_spike_window * 0.5):])  # 20 to 40 ms after spike
                basal_mean = np.mean(noise_window)
                basal_std = np.std(noise_window)

                # Threshold for MUAP onset (6 times the standard deviation above the mean basal activity)
                threshold_high = basal_mean + 6 * basal_std
                threshold_low = basal_mean - 6 * basal_std

                # Find the first time point where the MUAP crosses the threshold
                onset_idx_high = np.argmax(best_channel_signal > threshold_high)
                onset_idx_low = np.argmax(best_channel_signal < threshold_low)
                onset_idx = np.min([onset_idx_high, onset_idx_low])  # Take the first occurrence of either condition

                # Adjust spike times by the onset time
                realigned_spikes = spikes - (pre_spike_window - onset_idx)  # Shift spikes to the onset

                realigned_spike_times[grid].append(realigned_spikes)

                # Display the MUAP if requested
                if display_muap:
                    plt.figure(figsize=(10, 6))
                    time_axis = np.arange(-pre_spike_window, post_spike_window) / sampling_rate * 1000  # in ms
                    plt.plot(time_axis, best_channel_signal, label="MUAP", color=colors[grid])
                    plt.axvline(x=0, color='k', linestyle='--', label="Original Spike Time")
                    plt.axvline(x=(onset_idx - pre_spike_window) / sampling_rate * 1000, color='r', linestyle='--', label="Realigned Spike Time")
                    plt.axhline(y=threshold_high, color='g', linestyle='-', label="Threshold (6 x std)")
                    plt.axhline(y=threshold_low, color='g', linestyle='-')

                    # Show the noise window (-40 to -20 ms and +20ms to +40ms) as a shaded area
                    noise_start = -40  # in ms
                    noise_end = -20  # in ms
                    plt.axvspan(noise_start, noise_end, color='gray', alpha=0.3)
                    noise_start = 20  # in ms
                    noise_end = 40  # in ms
                    plt.axvspan(noise_start, noise_end, color='gray', alpha=0.3, label="Noise Window (-40 to -20ms & +20ms to +40ms)")

                    # Plot title with MU and grid/channel information
                    plt.title(f"{filename} \n MUAP for MU#{mu_idx}, Grid#{grid} - Best Channel: Grid {grid}, Row {best_channel_row}, Col {best_channel_col}")
                    plt.xlabel("Time (ms)")
                    plt.ylabel("Amplitude")
                    plt.legend(loc="upper right")
                    if display_figures_inline:
                        plt.show()
                    else:
                        plt.close()

            else:
                realigned_spike_times[grid].append(spikes)  # No valid spikes, keep original

    return realigned_spike_times


In [ ]:
def lowpass_filter(input_signal, fsamp, freq_cutoff, filter_order):
    nyquist = 0.5 * fsamp
    normal_cutoff = freq_cutoff / nyquist
    b, a = butter(filter_order, normal_cutoff, btype='low', analog=False)
    filtered_signal = filtfilt(b, a, input_signal)
    return filtered_signal

def normalize_force_and_identify_plateaus(force_temp, input_fsamp, display_plot=False, condition=1, filename="filename not specified"):
    """
    Normalize the force signal and extract plateau samples.
    """
    ### Detrend the force signal (there is drifting in some conditions)
    force_temp_mean = np.mean(force_temp)
    t = np.arange(len(force_temp)) / input_fsamp
    force_temp_polyfit = np.polynomial.Polynomial.fit(t, force_temp, deg=polynomial_degree_for_detrending)  # fit degree N polynomial
    force_temp -= force_temp_polyfit(t) # subtract the polynomial fit from the force signal
    force_temp += force_temp_mean
    ### Normalize the force signal
    if force_is_3D:
        force_temp_3D = force_temp
        force_temp = force_temp[0,:]
        std_ratios_relative_to_first_force_component = [1,
                np.std(force_temp_3D[1,:])/np.std(force_temp_3D[0,:]),
                np.std(force_temp_3D[2,:])/np.std(force_temp_3D[0,:])]
    epsilon_for_meaningful_rampup_or_down = force_differential_epsilon_up_or_down
    force_temp = force_temp / np.std(force_temp)
    force_smoothed_diff = np.gradient(lowpass_filter(force_temp, input_fsamp, freq_cutoff=force_lowpass_cutoff, filter_order=4))
    force_smoothed_diff_positive = np.full_like(force_smoothed_diff, np.nan)
    positive_indices = np.where(force_smoothed_diff>=+epsilon_for_meaningful_rampup_or_down)[0]
    force_smoothed_diff_positive[positive_indices] = force_smoothed_diff[positive_indices]
    count_positive = len(positive_indices)
    force_smoothed_diff_negative = np.full_like(force_smoothed_diff, np.nan)
    negative_indices = np.where(force_smoothed_diff<-epsilon_for_meaningful_rampup_or_down)[0]
    force_smoothed_diff_negative[negative_indices] = force_smoothed_diff[negative_indices]
    count_negative = len(negative_indices)
    plt.figure()
    plt.plot(force_smoothed_diff, color="gray")
    plt.plot(force_smoothed_diff_positive, color="C1", linewidth = 3, alpha = 0.75,
             label=f"{count_positive} samples where derivative is >= {epsilon_for_meaningful_rampup_or_down:.4f}")
    plt.plot(force_smoothed_diff_negative, color="C0", linewidth = 3, alpha = 0.75,
             label=f"{count_negative} samples where derivative is < {epsilon_for_meaningful_rampup_or_down:.4f}")
    plt.legend(loc="upper left")
    if display_figures_inline:
        plt.show()
    else:
        plt.close()
    if count_negative > count_positive:
        force_temp = force_temp * (-1)
    # Min-max normalize force_temp between 0 and 1 relative to percentile low and high
    percentile_low = np.percentile(force_temp, 5)
    percentile_high = np.percentile(force_temp, 95)
    force_temp[:input_fsamp*2] = percentile_low
    force_temp[-input_fsamp*2:] = percentile_low
    force_temp = (force_temp - percentile_low) / (percentile_high - percentile_low)

    plt.figure()
    if force_is_3D:
        plt.title(f"{current_subject} - {current_condition} - 3D force")
        for force_direction in range(force_temp_3D.shape[0]):
            if force_direction==0:
                force_temp_3D[0,:] = force_temp
                plt.plot(force_temp_3D[force_direction,:],label=f"Force direction {force_direction}")
                continue
            force_direction_temp = force_temp_3D[force_direction,:]
            force_direction_temp = force_direction_temp / np.std(force_temp_3D[force_direction,:])
            force_direction_temp = force_direction_temp * std_ratios_relative_to_first_force_component[force_direction]
            if count_negative > count_positive:
                force_direction_temp = force_direction_temp * (-1)
            # Normalize 3D force components relative to main force component (1st channel)
            percentile_low = np.percentile(force_direction_temp, 5)
            force_direction_temp[:input_fsamp*2] = percentile_low
            force_direction_temp[-input_fsamp*2:] = percentile_low
            force_direction_temp = (force_direction_temp - percentile_low) / (percentile_high - percentile_low)
            force_temp_3D[force_direction,:] = force_direction_temp
            plt.plot(force_temp_3D[force_direction,:],label=f"Force direction {force_direction}")
        force_amplitude_3D = np.sqrt(np.sum(force_temp_3D**2,axis=0))
        plt.plot(force_amplitude_3D,color='k',label=f"Magnitude of 3D force vector")
    else:
        plt.title(f"{current_subject} - {current_condition} - force")
        plt.plot(force_temp)
    plt.xlabel("Time (samples)")
    plt.ylabel("Force (max standardized to 1, baseline to 0)")
    plt.legend()
    if os.path.exists(current_folder):
        plt.savefig(f'{current_folder}//{filename[0:-4]}_force_raw.png')
    if display_figures_inline:
        plt.show()
    else:
        plt.close()
    
    # Smooth the force using a 1-second moving average
    smoothing_window_force = int(np.round(input_fsamp * force_smoothing_window_length))  # 1 second window
    force_smoothed = np.convolve(force_temp, np.ones(smoothing_window_force) / smoothing_window_force, mode='same')

    # Compute force_diff (derivative of the smoothed force), using a 3-second moving average
    force_diff = np.diff(force_smoothed)  # First, compute the derivative
    smoothing_window_diff = int(np.round(input_fsamp * 3))  # 3 seconds window for moving average
    force_diff_smoothed = np.convolve(force_diff, np.ones(smoothing_window_diff) / smoothing_window_diff, mode='same')

    # Pad force_diff_smoothed to match the length of force_temp
    force_diff_smoothed = np.pad(force_diff_smoothed, (1, 0), mode='constant', constant_values=0)

    # Normalize the smoothed force_diff for easier threshold calculation
    force_diff_smoothed = force_diff_smoothed / max(force_diff_smoothed)

    # Time axis for plotting/intersections
    time_axis = np.arange(len(force_temp))

    if select_plateaus_manually:
        valid_peaks, valid_troughs = select_plateaus_manually_with_user_interacton(
            force_smoothed, time_axis)

    else:
        # Compute thresholds for peak and trough detection
        std_diff = np.std(force_diff_smoothed)
        thresh_for_peak = 0.5 * std_diff
        thresh_for_trough = -0.5 * std_diff

        # Function to detect intersections
        def find_intersections(x, y, threshold, direction='down'):
            intersections = []
            for i in range(1, len(y)):
                if (y[i - 1] > threshold and y[i] <= threshold) or (y[i - 1] < threshold and y[i] >= threshold):
                    if direction == 'down' and y[i] < y[i - 1]:
                        intersections.append(x[i])
            return intersections

        # Detect intersections (downward slope for peaks and troughs)
        peak_intersections = find_intersections(time_axis, force_diff_smoothed, thresh_for_peak, direction='down')
        trough_intersections = find_intersections(time_axis, force_diff_smoothed, thresh_for_trough, direction='down')
        # Make sure they are sorted
        peak_intersections = sorted(peak_intersections)
        trough_intersections = sorted(trough_intersections)

        # Define minimum plateau duration (in seconds)
        min_plateau_duration_sec = 5
        min_plateau_samples = int(min_plateau_duration_sec * input_fsamp)
        # ---------------------------
        # Combine and sort events
        # ---------------------------
        # Label each index as 'peak' or 'trough'
        events = [(p, 'peak') for p in peak_intersections] + [(t, 'trough') for t in trough_intersections]
        # Sort by time index
        events.sort(key=lambda x: x[0])  # sorts by the first element in tuple
        valid_peaks = []
        valid_troughs = []
        # -----------------------------------------
        # Walk through the sorted list of events
        # -----------------------------------------
        i = 0
        while i < len(events):
            current_time, current_type = events[i]
            # If it's a peak, we look ahead to see if the next event is a trough
            if current_type == 'peak':
                # Check if there *is* a next event
                if i + 1 < len(events):
                    next_time, next_type = events[i + 1]
                    # The next event must be a trough
                    if next_type == 'trough':
                        # Check minimum time difference
                        if (next_time - current_time) >= min_plateau_samples:
                            # We have a valid peak-trough pair
                            valid_peaks.append(current_time)
                            valid_troughs.append(next_time)
                            # Skip the next event (we've already paired it)
                            i += 2
                            continue
                        else:
                            # The trough is too close => remove both from consideration
                            # i.e., skip them
                            i += 2
                            continue
                    else:
                        # The next event is another peak => remove the current peak
                        i += 1
                        continue
                else:
                    # No next event => remove this peak
                    i += 1
                    continue
            else:
                # Current event is a trough that isn't directly paired with a valid peak
                # => skip it
                i += 1
    # End of 'if no manual selection of plateaus'

    # At this point, valid_peaks and valid_troughs only contain
    # properly paired peak–trough events, with adequate time between them.
    # ------------------------------------------------------------------
    # Overwrite original peak_intersections and trough_intersections
    peak_intersections = valid_peaks
    trough_intersections = valid_troughs

    # Normalization of the force data based on peaks and troughs (median during rest = 0, median during plateaus = 1)
    def rest_VS_plateau_force_normalization_and_plateau_samples_identification(force, peaks, troughs, fs=None):
        """
        Normalize the force signal so that:
        1) The median 'rest' force => 0
        2) The median 'plateau' force => 1
        -------
        Returns
        -------
        normalized_force : np.ndarray
            The force data mapped so that the rest median is 0 and the plateau median is 1.
        plateau_samples : list
            Indices of the samples considered part of the plateaus.
        rest_samples : list
            Indices of the samples considered part of the rest intervals.
        plateau_median : float
            The median force value across all plateau intervals.
        rest_force : float
            The median force value across all rest intervals.
        """
        # 1) Identify plateau intervals
        plateau_values = []
        plateau_samples = []
        plateau_samples_at_each_plateau = []
        # Go through each peak -> trough interval
        for i in range(len(peaks)):
            start_idx = peaks[i]
            # Use matching trough or go to end of signal
            if i < len(troughs):
                end_idx = troughs[i]
            else:
                end_idx = len(force) - 1
            plateau_values.extend(force[start_idx:end_idx])
            plateau_samples.extend(range(start_idx, end_idx))
            plateau_samples_at_each_plateau.append(range(start_idx, end_idx))
        # 2) Define rest samples as everything outside the plateau samples
        all_indices = set(range(len(force)))
        plateau_indices = set(plateau_samples)
        rest_indices = list(all_indices - plateau_indices)
        # 3) Compute medians
        #    - If we have no plateau samples, default to 1 to avoid division by zero
        plateau_median = np.median(plateau_values) if len(plateau_values) > 0 else 1.0
        #    - If we have no rest samples, default to 0
        rest_values = force[rest_indices] if len(rest_indices) > 0 else np.array([0])
        # here, "rest_values" actually contains the ramps. So taking the lowest 1% of the values instead (not the minimum value because of potential artifacts)
        rest_force = np.percentile(rest_values, 1) # np.median(rest_values)
        # 4) Apply a linear transform:
        #    rest_force -> 0, plateau_median -> 1
        denom = plateau_median - rest_force
        if np.isclose(denom, 0):
            # Edge case: if rest_force == plateau_median, we can't do normal scaling
            # Fallback: just zero-center around rest_force
            print("Warning: rest_force and plateau_median are the same. "
                "Forcing zero-centering. Plateaus will also be at ~0.")
            normalized_force = force - rest_force
        else:
            normalized_force = (force - rest_force) / denom
        return (normalized_force,
                plateau_samples,
                plateau_samples_at_each_plateau,
                rest_indices,
                plateau_median,
                rest_force)

    # Normalize the force data and get the plateau samples
    (normalized_force, plateau_samples, plateau_samples_at_each_plateau,
        _, _, _) = rest_VS_plateau_force_normalization_and_plateau_samples_identification(
        force_temp, peak_intersections, trough_intersections)
    
    # Compute FFT of the force signal for each plateau and for the entire signal
    # --- controls for shared frequency grid ---
    bins_per_hz = 4           # "at most N bins per Hz"
    fmax = 15.0               # up to 10 Hz (will be clipped to Nyquist)
    fmax = min(fmax, input_fsamp/2.0)
    n_bins = int(np.floor(bins_per_hz * fmax))
    freq_edges = np.linspace(0.0, fmax, n_bins + 1)
    freq_centers = 0.5 * (freq_edges[:-1] + freq_edges[1:])

    def one_sided_psd_binned(x, fs, edges):
        """Return (bin_centers, PSD averaged within bins).
        PSD units: (signal_units^2 / Hz); comparable across different lengths."""
        x = np.asarray(x)
        if x.size < 2:
            return freq_centers, np.full(edges.size - 1, np.nan)

        # de-mean to remove offset
        x = x - np.mean(x)

        N = x.size
        X = np.fft.rfft(x)
        # Periodogram scaling for PSD (linear): |X|^2 / (fs * N), then one-sided
        Pxx = (np.abs(X) ** 2) / (fs * N)
        if N % 2 == 0:  # even N => Nyquist bin present at the end
            if Pxx.size > 2:
                Pxx[1:-1] *= 2.0
        else:
            if Pxx.size > 1:
                Pxx[1:] *= 2.0

        f = np.fft.rfftfreq(N, d=1.0/fs)

        # Restrict to chosen band
        m = (f >= edges[0]) & (f < edges[-1])
        f = f[m]
        Pxx = Pxx[m]

        # Bin by averaging the PSD within each frequency bin
        sums, _ = np.histogram(f, bins=edges, weights=Pxx)
        counts, _ = np.histogram(f, bins=edges)
        with np.errstate(divide='ignore', invalid='ignore'):
            binned = sums / counts
        binned[counts == 0] = np.nan  # no samples fell into that bin

        centers = 0.5 * (edges[:-1] + edges[1:])
        return centers, binned

    # --------------------------------------------
    # Compute PSD (binned) for each plateau & total + coefficient of variation of force during each plateau
    # --------------------------------------------
    psd_each_plateau = []
    force_coeff_of_variation_each_plateau = []
    force_std_each_plateau = []
    force_unfiltered = force_temp * condition

    for plateau_range in plateau_samples_at_each_plateau:
        if len(plateau_range) > 1:
            plateau_signal = force_unfiltered[plateau_range]
            _, psd_binned = one_sided_psd_binned(plateau_signal, input_fsamp, freq_edges)
            psd_each_plateau.append(psd_binned)
            force_std_each_plateau.append(np.std(plateau_signal))
            force_coeff_of_variation_each_plateau.append(
                np.std(plateau_signal) / np.mean(plateau_signal)
            )
        else:
            psd_each_plateau.append(None)
            force_std_each_plateau.append(None)
            force_coeff_of_variation_each_plateau.append(None)

    # "Total" PSD over the concatenated/selected samples
    _, psd_total_binned = one_sided_psd_binned(force_unfiltered[plateau_samples], input_fsamp, freq_edges)

    # Mean PSD across plateaus (ignore missing/empty bins with NaNs)
    valid_psds = [p for p in psd_each_plateau if p is not None]
    if len(valid_psds):
        mean_plateau_psd = np.nanmean(np.vstack(valid_psds), axis=0)
    else:
        mean_plateau_psd = np.full_like(psd_total_binned, np.nan)

    # Scale the normalized force by the condition value
    scaled_force = normalized_force * condition

    # Plot the force if required
    if display_plot:
        fig, axs = plt.subplots(2, 1, figsize=(10, 10))

        # Plot the original and smoothed force, with peaks and troughs
        axs[0].plot(time_axis, force_temp, color="black", label='force_raw')
        axs[0].plot(time_axis, force_smoothed, color="C1", label='force_smoothed')
        if not select_plateaus_manually:
            axs[0].plot(time_axis, force_diff_smoothed, color="C5", label='force_smoothed_diff')
            axs[0].axhline(y=thresh_for_peak, color="C2", label='thresh_for_peak')
            axs[0].axhline(y=thresh_for_trough, color="C2", label='thresh_for_trough')
            axs[0].scatter(peak_intersections, [thresh_for_peak] * len(peak_intersections), color='red', label='Peak Intersections')
            axs[0].scatter(trough_intersections, [thresh_for_trough] * len(trough_intersections), color='blue', label='Trough Intersections')
            axs[0].set_title("Original and Smoothed Force with Diff")

        # Plot the normalized force
        axs[1].plot(time_axis, scaled_force, color="C1", label='force_normalized (scaled)', zorder=1)
        axs[1].scatter(time_axis[plateau_samples], scaled_force[plateau_samples], s=1, color='red',
                    label='Plateau Samples', marker='o', zorder=2)
        axs[1].legend()
        axs[1].set_title("Normalized Force")

        plt.tight_layout()
        plt.title(f'{filename}')
        if os.path.exists(current_folder):
            plt.savefig(f'{current_folder}//{filename[0:-4]}_force.png')
        if display_figures_inline:
            plt.show()
        else:
            plt.close()

        # Second figure, with 2 subplots (top, bottom)
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 6))

        # 1) Top subplot: force during each plateau
        for i, plateau_range in enumerate(plateau_samples_at_each_plateau):
            if len(plateau_range) > 1:
                plateau_signal = force_unfiltered[plateau_range]
                ax1.plot(plateau_range, plateau_signal, color='black', linewidth=0.7)
                mid_point = plateau_range[len(plateau_range)//2]
                max_val = np.max(plateau_signal)
                ax1.text(mid_point, max_val, f'STD={force_std_each_plateau[i]:.2f}% MVC\nCV={force_coeff_of_variation_each_plateau[i]:.3f}',
                        fontsize=8, ha='center', va='bottom')
                ax1.set_title("Force during each plateau")
                ax1.set_xlabel("Time (samples)")
                ax1.set_ylabel("Force (% MVC)")

        # 2) Bottom subplot: PSD in dB for each plateau, total, and mean (up to fmax)
        eps = 1e-20  # to avoid log(0)
        for i, psd_binned in enumerate(psd_each_plateau):
            if psd_binned is not None:
                ax2.plot(freq_centers, 10*np.log10(psd_binned + eps), color='purple', alpha=0.3)

        ax2.plot(freq_centers, 10*np.log10(psd_total_binned + eps), linewidth=2,
                label='Total PSD (dB)', color='black')
        ax2.plot(freq_centers, 10*np.log10(mean_plateau_psd + eps), linestyle='-',
                linewidth=2, label='Mean plateau PSD (dB)', color='purple')

        ax2.set_xlim(0, fmax)
        ax2.set_title("Power Spectral Density (binned, dB)")
        ax2.set_xlabel("Frequency (Hz)")
        ax2.set_ylabel("PSD (dB re (%%MVC)^2/Hz)")
        ax2.legend()

        plt.tight_layout()
        plt.title(f'{filename}')
        if os.path.exists(current_folder):
            plt.savefig(f'{current_folder}//{filename[0:-4]}_force_variations_and_psd.png')
        if display_figures_inline:
            plt.show()
        else:
            plt.close()

    # Keep linear PSD for downstream use
    force_psd_total = psd_total_binned
    force_psd_each_plateau = mean_plateau_psd

    # Return the normalized force, plateau samples, and info about force
    return scaled_force, plateau_samples, plateau_samples_at_each_plateau, force_psd_each_plateau, force_psd_total, force_std_each_plateau, force_coeff_of_variation_each_plateau

In [ ]:
def compute_recruitment_thresholds_and_firing_rates(normalized_force, spike_times,
                                                    plateau_samples_at_each_plateau,
                                                    time_window_to_check_for_ramp=7,  # seconds (used for ramp duration)
                                                    lowpass_filter_force_freq=1,    # Hz cutoff for force filtering
                                                    positive_fraction_threshold=0.5,  # fraction required in ramp window
                                                    derivative_epsilon=0.05,          # threshold for normalized derivative
                                                    threshold_for_MU_to_be_continuous=0.5,  # max gap in s for continuity (MFR)
                                                    input_fsamp=2048,
                                                    display_plot=False, filename='filename not specified'):
    """
    Compute recruitment thresholds and mean firing rates for motor units (MUs) based on spike times,
    normalized force, and pre-determined plateau intervals.
    
    Procedure:
      1. Lowpass filter normalized force (cutoff: lowpass_filter_force_freq Hz) and compute its derivative.
         Normalize the derivative (zero mean, unit std).
      2. For each plateau:
           - Check that at least time_window_to_check_for_ramp seconds of data exist prior to plateau onset.
           - In the ramp window (from plateau_onset - ramp_duration_samples to plateau_onset),
             compute the fraction of samples for which the normalized derivative > derivative_epsilon.
             If that fraction is below positive_fraction_threshold, then the ramp is invalid and the
             recruitment threshold and mean firing rate (MFR) for that plateau are set to NaN.
      3. For a valid ramp:
           - Compute the recruitment threshold using a candidate window for threshold defined as:
                 candidate_start_thresh = max(0, plateau_onset - ramp_duration_samples)
                 candidate_end = plateau_end
             Then, within that window, slide a window of 5 candidate spikes. If a window is found where the time
             difference between the first and the 5th spike is <= input_fsamp samples (≤ 1 s),
             then the recruitment threshold is the filtered force at the time of the first spike in that window and
             its time is recorded; otherwise, assign NaN.
           - Compute the MFR using the candidate window for firing rate defined as:
                 if first plateau: candidate_start_mfr = plateau_onset,
                 else: candidate_start_mfr = previous plateau’s end,
                 candidate_end = plateau_end.
             Then, take all candidate spikes within that window, add the window boundaries,
             compute inter-spike intervals (in s) and if the maximum interval exceeds threshold_for_MU_to_be_continuous,
             MFR is set to NaN, else MFR = 1/mean(intervals).
      4. If display_plot is True, produce a combined figure with:
             - A top (large) subplot showing the filtered force curve (black) with the candidate window (red solid line)
               for recruitment threshold overlaid. Also vertical lines at ramp start (blue) and plateau onset (orange)
               are drawn. A raster plot of spike times (each MU on a separate row) with recruitment threshold markers is
               superimposed on a twin y-axis.
             - A bottom (smaller) subplot showing the normalized derivative (purple) with vertical lines at the same markers
               and a horizontal line at derivative_epsilon.
    
    Returns:
      MU_properties (dict): Dictionary keyed by grid. For each grid, a list is produced (one per MU) with a dictionary:
            {
                'MU': motor unit index,
                'recruitment_thresholds': [threshold value per plateau or NaN],
                'recruitment_times': [spike sample index for threshold or NaN],
                'mean_firing_rates': [mean firing rate (Hz) per plateau or NaN]
            }
    """
    MU_properties = {}
    ramp_duration_samples = int(time_window_to_check_for_ramp * input_fsamp)
    
    # Lowpass filter the normalized force.
    filtered_force = lowpass_filter(normalized_force, input_fsamp, 
                                    freq_cutoff=lowpass_filter_force_freq, filter_order=4)
    
    # Compute derivative and normalize it.
    force_deriv = np.diff(filtered_force)
    force_deriv_normalized = (force_deriv - np.mean(force_deriv)) / np.std(force_deriv)
    
    # Prepare candidate window force array for debugging plot.
    candidate_force_overall = np.full(len(normalized_force), np.nan)
    
    for grid in spike_times.keys():
        MU_properties[grid] = []
        for mu_idx, mu_spikes in enumerate(spike_times[grid]):
            # Avoid empty MUs (because of the cell structure it is stored in in Matlab)
            if len(mu_spikes) < 1:
                continue
            mu_spikes = np.array(mu_spikes).astype(int)
            mu_recruitment_thresholds = []
            mu_recruitment_times = []
            mu_mean_firing_rates = []
            mu_mean_cov = []
            for i, plateau in enumerate(plateau_samples_at_each_plateau):
                plateau_onset = plateau.start if hasattr(plateau, 'start') else plateau[0]
                plateau_end = plateau.stop - 1 if hasattr(plateau, 'stop') else plateau[-1]
                
                if plateau_onset < ramp_duration_samples:
                    mu_recruitment_thresholds.append(np.nan)
                    mu_recruitment_times.append(np.nan)
                    mu_mean_firing_rates.append(np.nan)
                    mu_mean_cov.append(np.nan)
                    continue
                
                ramp_start = plateau_onset - ramp_duration_samples
                ramp_deriv = force_deriv_normalized[ramp_start:plateau_onset]
                pos_fraction = np.sum(ramp_deriv > derivative_epsilon) / len(ramp_deriv) if len(ramp_deriv) else 0
                
                if pos_fraction < positive_fraction_threshold:
                    mu_recruitment_thresholds.append(np.nan)
                    mu_recruitment_times.append(np.nan)
                    mu_mean_firing_rates.append(np.nan)
                    mu_mean_cov.append(np.nan)
                else:
                    # Valid ramp. For recruitment threshold, candidate window includes time before plateau onset.
                    candidate_start_thresh = max(0, plateau_onset - ramp_duration_samples)
                    candidate_end = plateau_end
                    candidate_idx_thresh = np.arange(candidate_start_thresh, candidate_end+1)
                    candidate_idx_thresh = candidate_idx_thresh[candidate_idx_thresh >= candidate_start_thresh]
                    candidate_force_overall[candidate_idx_thresh] = filtered_force[candidate_idx_thresh]
                    
                    # Get candidate spikes for threshold: use all spikes from candidate_start_thresh to candidate_end.
                    candidates_thresh = mu_spikes[(mu_spikes >= candidate_start_thresh) & (mu_spikes <= candidate_end)]
                    if candidates_thresh.size < 5:
                        mu_recruitment_thresholds.append(np.nan)
                        mu_recruitment_times.append(np.nan)
                    else:
                        found = False
                        for j in range(len(candidates_thresh) - 4):
                            window = candidates_thresh[j:j+5]
                            if (window[-1] - window[0]) <= input_fsamp:  # within 1 s
                                recruitment_time_idx = window[0]
                                found = True
                                break
                        if found:
                            mu_recruitment_thresholds.append(filtered_force[recruitment_time_idx])
                            mu_recruitment_times.append(recruitment_time_idx)
                        else:
                            mu_recruitment_thresholds.append(np.nan)
                            mu_recruitment_times.append(np.nan)
                    
                    # Mean firing rate (MFR) calculation uses plateau samples only.
                    candidate_start_mfr = plateau_onset
                    candidate_end_mfr = plateau_end
                    candidate_idx_mfr = np.arange(candidate_start_mfr, candidate_end_mfr+1)
                    candidate_spikes_mfr = mu_spikes[(mu_spikes >= candidate_start_mfr) & (mu_spikes <= candidate_end_mfr)]
                    if candidate_spikes_mfr.size < 2:
                        # need at least 2 spikes to form an interval
                        mfr = np.nan
                        mcov = np.nan
                    else:
                        # check continuity: include start/end as well
                        combined_times = np.sort(np.concatenate(([candidate_start_mfr], candidate_spikes_mfr, [candidate_end_mfr])))
                        if np.max(np.diff(combined_times) / input_fsamp) > threshold_for_MU_to_be_continuous:
                            mfr = np.nan
                            mcov = np.nan
                        else:
                            intervals = np.diff(candidate_spikes_mfr) / input_fsamp  # in seconds
                            mfr = 1 / np.mean(intervals)
                            mcov = np.std(intervals) / np.mean(intervals)
                    mu_mean_firing_rates.append(mfr)
                    mu_mean_cov.append(mcov)
            MU_properties[grid].append({
                'MU': mu_idx,
                'recruitment_thresholds': mu_recruitment_thresholds,
                'recruitment_times': mu_recruitment_times,
                'mean_firing_rates': mu_mean_firing_rates,
                'mean_cov': mu_mean_cov
                })
    
    # Combined debug figure.
    if display_plot:
        import matplotlib.gridspec as gridspec
        fig = plt.figure(figsize=(20, 15))
        gs = fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        ax_force = fig.add_subplot(gs[0])
        ax_deriv = fig.add_subplot(gs[1])
        
        time_axis = np.arange(len(normalized_force)) / input_fsamp
        ax_force.plot(time_axis, filtered_force, color='black', label='Filtered Force')
        ax_force.plot(time_axis, candidate_force_overall, color='red', linestyle='-', linewidth=2, label='Candidate Window')
        for plateau in plateau_samples_at_each_plateau:
            plateau_onset = plateau.start if hasattr(plateau, 'start') else plateau[0]
            t_onset = plateau_onset / input_fsamp
            if plateau_onset >= ramp_duration_samples:
                ramp_start = plateau_onset - ramp_duration_samples
                ax_force.axvline(x=ramp_start/input_fsamp, color='blue', linestyle='--', alpha=0.7)
            ax_force.axvline(x=t_onset, color='orange', linestyle='--', alpha=0.7)
        ax_force.set_xlabel("Time (s)")
        ax_force.set_ylabel("Filtered Force")
        ax_force.set_title(f"{filename} - Filtered Force, Candidate Window & Raster")
        ax_force.legend(loc='upper left')
        
        # Raster overlay (twin y-axis).
        ax_raster = ax_force.twinx()
        mu_global_index = 0
        # raster_spike_times = []
        # raster_mu_indices = []
        # Also, for each MU, plot a marker (e.g., magenta circle) at each recruitment threshold.
        for grid in spike_times.keys():
            for mu_idx, mu_spikes in enumerate(spike_times[grid]):
                # Avoid empty MUs (because of the cell structure it is stored in in Matlab)
                if len(mu_spikes) < 1:
                    continue
                color_of_MU = globals()[f"{data[current_subject][current_condition]['muscle_per_grid'][grid]}_color"]
                mu_spikes = np.array(mu_spikes).astype(int)
                y_val = mu_global_index + 1
                # raster_spike_times.extend(mu_spikes)
                # raster_mu_indices.extend([y_val] * len(mu_spikes))
                # Check recruitment thresholds.
                for entry in MU_properties[grid]:
                    if entry['MU'] == mu_idx:
                        for rec_time in entry['recruitment_times']:
                            if not np.isnan(rec_time):
                                ax_raster.plot(rec_time / input_fsamp, y_val, marker='o', markersize=10, color=color_of_MU, markeredgecolor='black')
                ax_raster.scatter(mu_spikes/input_fsamp,
                                  [y_val] * len(mu_spikes), marker='|', color=color_of_MU)
                mu_global_index += 1
        # ax_raster.scatter(np.array(raster_spike_times)/input_fsamp, raster_mu_indices, marker='|', color=color_of_MU)
        ax_raster.set_ylabel("MU Index (Raster)")
        ax_raster.set_ylim(0, mu_global_index + 1)
        ax_raster.tick_params(axis='y', labelcolor='gray')
        
        # Bottom subplot: Normalized derivative.
        time_axis_deriv = np.arange(1, len(filtered_force)) / input_fsamp
        ax_deriv.plot(time_axis_deriv, force_deriv_normalized, color='purple', label='Normalized Derivative')
        for plateau in plateau_samples_at_each_plateau:
            plateau_onset = plateau.start if hasattr(plateau, 'start') else plateau[0]
            t_onset = plateau_onset / input_fsamp
            if plateau_onset >= ramp_duration_samples:
                ramp_start = plateau_onset - ramp_duration_samples
                ax_deriv.axvline(x=ramp_start/input_fsamp, color='blue', linestyle='--', alpha=0.7)
            ax_deriv.axvline(x=t_onset, color='orange', linestyle='--', alpha=0.7)
        ax_deriv.axhline(y=derivative_epsilon, color='green', linestyle='-', label='Epsilon threshold')
        ax_deriv.set_xlabel("Time (s)")
        ax_deriv.set_ylabel("Normalized Derivative")
        ax_deriv.legend(loc='best')
        
        plt.tight_layout()
        if os.path.exists(current_folder):
            fig.savefig(os.path.join(current_folder, f"{filename}_recruitment_thresholds_debug.png"))
        if display_figures_inline:
            plt.show()
        else:
            plt.close()
    
    return MU_properties


In [ ]:

def process_firing_rates(data, current_subject, current_condition, input_fsamp=2048, Wind_s=0.4, display_plots=False):
    """
    Process motor unit data for the current subject and condition.
    It will generate smoothed firing rates

    Parameters:
    data (dict): Dictionary where processed data will be stored.
    current_subject (str): The subject's identifier.
    current_condition (str): The condition for the current subject.
    input_fsamp (int): Sampling rate.
    Wind_s (float): Hanning window duration (default: 0.4 seconds).
    display_plots (bool): Whether to display the smoothed and preprocessed firing rates plots.

    Returns:
    dict: Updated 'data' dictionary containing processed MU information.
    """

    # Retrieve spike times
    spike_times = data[current_subject][current_condition]['spike_times']

    # Retrieve the force array for the condition
    force = data[current_subject][current_condition]['force']
    force_len = len(force)

    # Create Hanning filter for smoothed firing rates
    Wind_length = round(input_fsamp * Wind_s)
    HanningW = windows.hann(Wind_length)  # Hanning window filter
    HanningW = HanningW / np.sum(HanningW)  # Normalize window to have a unitary area

    # Initialize lists for each processed variable
    smoothed_firing_rates = {grid: [] for grid in data[current_subject][current_condition]['spike_times'].keys()}
 
    # Extract continuous plateau windows from plateau_samples
    plateau_samples = data[current_subject][current_condition]['plateau_samples']

    # failsafe in case plateau_samples is empty = make all samples be plateau samples
    if plateau_samples == []:
        plateau_samples = np.arange(len(data[current_subject][current_condition]['force']))

    # Ensure spike times are cast to integer for binary spike train creation
    for grid in data[current_subject][current_condition]['spike_times'].keys():
        for mu_idx, mu_spikes in enumerate(spike_times[grid]):
            # If MU has spikes, process them
            if len(mu_spikes) > 0:
                mu_spikes = mu_spikes.astype(int)  # Ensure spike times are integers

                binary_spike_train = np.zeros(force_len, dtype=int)
                binary_spike_train[mu_spikes] = 1  # Mark spike times in the binary train

                # Smoothed firing rate
                smoothed_rate = filtfilt(HanningW, 1, binary_spike_train * input_fsamp)

                # Ensure that smoothed_rate matches the length of the force array
                if len(smoothed_rate) > force_len:
                    smoothed_rate = smoothed_rate[:force_len]  # Trim if longer
                elif len(smoothed_rate) < force_len:
                    smoothed_rate = np.pad(smoothed_rate, (0, force_len - len(smoothed_rate)))  # Pad with zeros if shorter

                smoothed_firing_rates[grid].append(smoothed_rate)

    # Store smoothed firing rates and valid MUs
    data[current_subject][current_condition]['smoothed_firing_rates'] = smoothed_firing_rates

    # Initialize for plotting
    if display_plots:
        plt.figure(figsize=(15, 10))  # Make the plots wider
        time_axis = np.arange(force_len) / input_fsamp
        for grid in data[current_subject][current_condition]['spike_times'].keys():
            for mu_idx, smoothed_rate in enumerate(smoothed_firing_rates[grid]):
                color = grid_colors[grid]
                plt.plot(time_axis, smoothed_rate, color=color, alpha=0.5)
        plt.title(f'Smoothed Firing Rates - Subject {current_subject} - Condition {current_condition} (all MUs)')
        plt.xlabel('Time (s)')
        plt.ylabel('Firing Rate (Hz)')
        plt.grid(True)
        if os.path.exists(current_folder):
            plt.savefig(f'{current_folder}//{filei[:-4]}_firing_rates.png')
        if display_figures_inline:
            plt.show()
        else:
            plt.close()

    return data


In [ ]:
def select_plateaus_manually_with_user_interacton(force, time_axis):
    """
    Displays an interactive figure using the Qt backend.
    
    The user clicks on the figure to record ROI boundaries.
    - Press 'z' to undo the last click.
    - Press Enter when finished.
    
    When done, the recorded boundaries are sorted such that:
      * The lowest boundary defines the start of the first ROI.
      * The second lowest defines the end of the first ROI.
      * The third lowest defines the start of the second ROI, etc.
      
    If an odd number of boundaries is recorded, the final ROI will end at the maximum x value.
    
    Non-ROI regions are shaded using axvspan.
    
    Returns:
        roi_starts (np.array of int): The start positions of the ROIs.
        roi_ends   (np.array of int): The end positions of the ROIs.
    """

    # Create figure and plot the force signal.
    fig, ax = plt.subplots()

    # display raster plot of spike times
    raster_spike_times = []
    raster_mu_indices = []
    raster_colors = []
    spike_times = data[current_subject][current_condition]['spike_times']
    mu_total_iter = -1
    for grid in spike_times.keys():
        for mu_idx, mu_spikes in enumerate(spike_times[grid]):
            if len(mu_spikes) == 0:
                continue  # No spikes for this MU, skip
            mu_total_iter += 1  # Increment total motor unit index
            # Convert spike times to sample indices (rounding)
            mu_spikes = np.array(mu_spikes).astype(int)
            # Store spike times and indices for raster plotting
            raster_spike_times.extend(mu_spikes)
            raster_mu_indices.extend([mu_total_iter] * len(mu_spikes))  # Ensure each MU is on a different row
            # Color by grid
            # raster_colors.extend([f"C{grid}"] * len(mu_spikes)) 
            raster_colors.extend([grid_colors[grid][0:3]] * len(mu_spikes)) # without the alpha value
    ax.scatter(np.array(raster_spike_times), raster_mu_indices, 
               zorder = -1, c=raster_colors, marker='|', alpha = 0.25)
    
    ax.plot(time_axis, force*mu_total_iter, color='black', linewidth = 2, label = 'Smoothed force (a.u.)')
    ax.legend()
    ax.set_title(f"{current_subject} - {current_condition}\nClick to add ROI boundaries.\nPress 'z' to undo; press Enter when done.")
    ax.set_xlabel("Time (samples)")
    
    x_min, x_max = time_axis[0], time_axis[-1]
    boundaries = []       # Interactive buffer to store clicked x positions.
    clicked_lines = []    # List to store permanent blue vertical lines for each boundary.
    shading = []          # List to store axvspan objects for non-ROI areas.
    
    def update_shading():
        """Update the shaded regions based on current boundaries."""
        nonlocal shading
        # Remove previous shading.
        for sh in shading:
            sh.remove()
        shading = []
        
        sorted_boundaries = sorted(boundaries)
        if len(sorted_boundaries) == 0:
            fig.canvas.draw_idle()
            return
        # If an odd number of boundaries, assume final ROI extends to x_max.
        if len(sorted_boundaries) % 2 == 1:
            sorted_boundaries.append(x_max)
        # Shade region before the first ROI.
        if sorted_boundaries[0] > x_min:
            shading.append(ax.axvspan(x_min, sorted_boundaries[0], color='gray', alpha=0.3))
        # Shade gaps between ROIs.
        for i in range(0, len(sorted_boundaries)-2, 2):
            roi_end = sorted_boundaries[i+1]
            next_roi_start = sorted_boundaries[i+2]
            if next_roi_start > roi_end:
                shading.append(ax.axvspan(roi_end, next_roi_start, color='gray', alpha=0.3))
        # Shade region after the last ROI.
        if sorted_boundaries[-1] < x_max:
            shading.append(ax.axvspan(sorted_boundaries[-1], x_max, color='gray', alpha=0.3))
        fig.canvas.draw_idle()
    
    def update_boundary_lines():
        """Update permanent blue lines for each recorded boundary."""
        nonlocal clicked_lines
        # Remove existing boundary lines.
        for line in clicked_lines:
            line.remove()
        clicked_lines = []
        # Draw a blue vertical line at each boundary.
        for b in boundaries:
            clicked_lines.append(ax.axvline(x=b, color='blue', linestyle='-', linewidth=1))
        fig.canvas.draw_idle()
    
    finished = False  # Flag to signal that ROI selection is complete.
    
    def on_click(event):
        """Record the x position when the user clicks and update the display."""
        if event.inaxes != ax or event.xdata is None:
            return
        boundaries.append(event.xdata)
        update_boundary_lines()
        update_shading()
    
    def on_key(event):
        """Handle key presses:
           - 'enter' finishes the selection.
           - 'z' undoes the last click.
        """
        nonlocal finished
        if event.key == 'enter':
            finished = True
            fig.canvas.stop_event_loop()
        elif event.key.lower() == 'z':
            if boundaries:
                boundaries.pop()  # Remove the last clicked boundary.
                update_boundary_lines()
                update_shading()
    
    # Connect event callbacks.
    cid_click = fig.canvas.mpl_connect('button_press_event', on_click)
    cid_key   = fig.canvas.mpl_connect('key_press_event', on_key)
    
    print("ROI selection in progress.\nClick to add boundaries; press 'z' to undo; press Enter when done.")
    # Start an event loop that waits until the user presses Enter.
    fig.canvas.start_event_loop(0)
    while not finished:
        plt.pause(0.1)
    
    # Disconnect the event handlers.
    fig.canvas.mpl_disconnect(cid_click)
    fig.canvas.mpl_disconnect(cid_key)
    plt.close(fig)
    
    # Process the recorded boundaries.
    sorted_boundaries = sorted(boundaries)
    if len(sorted_boundaries) == 0:
        # No clicks: the ROI covers the entire signal.
        roi_starts = [x_min]
        roi_ends   = [x_max]
    else:
        if len(sorted_boundaries) % 2 == 1:
            sorted_boundaries.append(x_max)
        roi_starts = []
        roi_ends   = []
        for i in range(0, len(sorted_boundaries), 2):
            roi_starts.append(sorted_boundaries[i])
            roi_ends.append(sorted_boundaries[i+1])
    
    # Convert boundaries to integer arrays.
    roi_starts = np.array(roi_starts, dtype=int)
    roi_ends   = np.array(roi_ends, dtype=int)
    
    print("Final ROIs:")
    print("Starts:", roi_starts)
    print("Ends:  ", roi_ends)
    
    return roi_starts, roi_ends

# Example usage:
# roi_starts, roi_ends = select_plateaus_manually_with_user_interacton(force, time_axis)


In [ ]:
def find_optimal_plateau_subset_for_K_plateaus(valid_MUs_per_plateau, nb_plateaus_to_keep):
    """
    Given:
    - valid_MUs_per_plateau: a (nb_MUs x nb_plateaus) numpy array of 0/1,
        where rows are Motor Units (MUs), columns are plateaus.
    - nb_plateaus_to_keep (K): the number of columns (plateaus) to select (K < nb_plateaus).
    Returns:
    - best_nb_remaining_MUs: the maximum number of MUs (rows) that can be retained
                            by using exactly K columns
    - best_plateaus_subset:  the subset of K column indices that achieves that maximum
                            number of rows
    - best_valid_MUs:        the indices of the MUs (rows) that remain (i.e., have all 1s 
                            in the chosen subset of columns)
    """
    nb_MUs, nb_plateaus = valid_MUs_per_plateau.shape
    
    best_nb_remaining_MUs = -1
    best_plateaus_subset = None
    best_valid_MUs = None  # We'll store the row indices that remain

    # Enumerate all subsets of columns of size nb_plateaus_to_keep
    for cols in itertools.combinations(range(nb_plateaus), nb_plateaus_to_keep):
        # Extract the submatrix for these columns only
        submat = valid_MUs_per_plateau[:, cols]  # shape (nb_MUs, nb_plateaus_to_keep)
        
        # Identify which rows have all 1s across these K columns
        row_mask = np.all(submat == 1, axis=1)  # Boolean mask of shape (nb_MUs,)
        coverage = np.sum(row_mask)             # Number of rows that remain
        
        # Update the best number of rows if this subset is better
        if coverage > best_nb_remaining_MUs:
            best_nb_remaining_MUs = coverage
            best_plateaus_subset = cols
            best_valid_MUs = np.where(row_mask)[0]  # Indices of the rows that remain

    return best_nb_remaining_MUs, best_plateaus_subset, best_valid_MUs

def Select_plateaus_for_dim_and_coh_analyses(data, current_subject, current_condition, display_plots=False, 
                                              minimum_mu_coverage=0.5):
    """
    Select a subset of plateaus for dimensionality and coherence analyses such that the
    number of motor units (MUs) retained is maximized.
    
    A MU is considered “valid” in a plateau if a criterion is met (e.g. based on its spike times).
    This function computes a matrix (nb_MUs x nb_plateaus) of 0/1 values (0 = invalid, 1 = valid),
    and then uses an optimization routine to select a subset of plateaus (of size K) that retains the
    maximum number of MUs. The loop terminates when the optimal subset retains at least
    (nb_MUs × minimum_mu_coverage) MUs.
    
    Parameters:
      data: nested dictionary structure for the subject and condition.
      current_subject: key for current subject.
      current_condition: key for current condition.
      display_plots (bool): whether to display a heatmap for debugging.
      minimum_mu_coverage (float): minimal fraction (between 0 and 1) of MUs that must be valid across the selected
                                   plateaus (default=0.5).
    
    Returns:
      data: the input dictionary updated with:
            'selected_plateaus' (the indices of the selected plateaus) and
            'valid_MUs_given_selected_plateaus' (the indices of the MUs retained).
    """
    
    import itertools
    import numpy as np
    from matplotlib.colors import ListedColormap
    import matplotlib.pyplot as plt
    
    # Number of plateaus (from your data)
    nb_plateaus = len(data[current_subject][current_condition]['plateau_samples_at_each_plateau'])
    nb_MUs = len(data[current_subject][current_condition]['MU_spike_times_list'])
    
    # Build the validity matrix: 1 if MU is valid for a plateau, 0 otherwise.
    valid_MUs_per_plateau = np.zeros((nb_MUs, nb_plateaus))  # shape: (nb_MUs, nb_plateaus)
    
    for plateau_i in range(nb_plateaus):
        for mn_i in range(nb_MUs):
            # Get the lower and upper bounds (sample indices) of the current plateau.
            plateau_current_lower_bound = data[current_subject][current_condition]['plateau_samples_at_each_plateau'][plateau_i][0]
            plateau_current_upper_bound = data[current_subject][current_condition]['plateau_samples_at_each_plateau'][plateau_i][-1]
            
            # Retrieve the spike times for this MU.
            spikes_within_plateau = data[current_subject][current_condition]['MU_spike_times_list'][mn_i]
            # Select only spikes that occur strictly between the lower and upper bound.
            spikes_within_plateau = spikes_within_plateau[(spikes_within_plateau > plateau_current_lower_bound) &
                                                          (spikes_within_plateau < plateau_current_upper_bound)]
            # Append the boundaries to ensure we capture gaps at the start and end.
            spikes_within_plateau = np.concatenate(([plateau_current_lower_bound],
                                                      spikes_within_plateau,
                                                      [plateau_current_upper_bound]))
            # If the maximum inter-spike interval (converted to seconds) is below a threshold, mark as valid.
            if (np.max(np.diff(spikes_within_plateau))/input_fsamp) < ISI_thresh_for_discontinuity:
                valid_MUs_per_plateau[mn_i, plateau_i] = 1

    if not select_plateaus_manually:
        # Select the number of plateaus to keep depending on the condition.
        # Find keys in nb_plateaus_to_select that match current_condition.
        matching_keys = [key for key in nb_plateaus_to_select.keys() if key in current_condition]
        if len(matching_keys) == 1:
            matched_key = matching_keys[0]
            nb_plateaus_to_keep = nb_plateaus_to_select[matched_key]
        elif len(matching_keys) > 1:
            print(f"Warning: Multiple matches found {matching_keys}, selecting the first one.")
            matched_key = matching_keys[0]
            nb_plateaus_to_keep = nb_plateaus_to_select[matched_key]
        else:
            print("No matching key found, using default value.")
            nb_plateaus_to_keep = 1
    else: # if plateaus have been selected manually, keep all of them
        nb_plateaus_to_keep = nb_plateaus

    # Optimize plateau selection.
    best_nb_remaining_MUs = 0
    nb_iter_optimize_MUs_to_keep = -1
    
    while (best_nb_remaining_MUs < nb_MUs * minimum_mu_coverage) and ((nb_iter_optimize_MUs_to_keep + 1) < nb_plateaus_to_keep):
        nb_iter_optimize_MUs_to_keep += 1
        K = nb_plateaus_to_keep - nb_iter_optimize_MUs_to_keep
        best_nb_remaining_MUs, best_plateaus_subset, remaining_valid_MUs = find_optimal_plateau_subset_for_K_plateaus(valid_MUs_per_plateau, nb_plateaus_to_keep=K)
        print(f"Optimal solution for {K} plateaus:")
        print(f"    Best coverage = {best_nb_remaining_MUs} MUs (out of {nb_MUs})")
        print(f"    Best subset of plateaus = {best_plateaus_subset}")
        print(f"    Remaining MUs: {remaining_valid_MUs}")
    
    # If no subset meets the desired coverage, force at least one plateau.
    if best_plateaus_subset is None:
        best_nb_remaining_MUs, best_plateaus_subset, remaining_valid_MUs = find_optimal_plateau_subset_for_K_plateaus(valid_MUs_per_plateau, nb_plateaus_to_keep=1)
    
    data[current_subject][current_condition]['selected_plateaus'] = best_plateaus_subset
    data[current_subject][current_condition]['valid_MUs_given_selected_plateaus'] = remaining_valid_MUs

    # (Optional) Plot the heatmap for debugging.
    if display_plots:
        valid_MUs_per_plateau_to_plot = np.copy(valid_MUs_per_plateau)
        valid_MUs_per_plateau_to_plot[:,best_plateaus_subset] += 2
        all_rows = set(range(nb_MUs))
        invalid_rows = all_rows - set(remaining_valid_MUs)
        for r in invalid_rows:
            valid_MUs_per_plateau_to_plot[r, :] = 4
        from matplotlib.colors import ListedColormap
        discrete_cmap = ListedColormap(["royalblue","brown","skyblue", "gold", "darkgray"])
        plt.figure(figsize=(6,6))
        sns.heatmap(valid_MUs_per_plateau_to_plot, cmap=discrete_cmap, cbar=False, 
                    linewidths=0.25, linecolor='white', vmin=0, vmax=4)
        plt.gca().invert_yaxis()
        plt.xlabel("Plateau #")
        plt.ylabel("MU #")
        plt.title(f'Selected plateaus (n={nb_plateaus_to_keep}) for dimensionality and coherence analyses, and remaining valid MUs \nSubject {current_subject} - Condition {current_condition} (Valid MUs)')
        import matplotlib.patches as mpatches
        mapping = {
            0: ("Invalid (plateau not selected)", discrete_cmap.colors[0]),
            1: ("Valid (plateau not selected)", discrete_cmap.colors[1]),
            2: ("Invalid (plateaus selected)", discrete_cmap.colors[2]),
            3: ("Valid (plateaus selected)", discrete_cmap.colors[3]),
            4: ("Invalid MUs", discrete_cmap.colors[4])
        }
        unique_codes = np.unique(valid_MUs_per_plateau_to_plot.astype(int))
        legend_order = [0,1,2,3,4]
        legend_patches = [mpatches.Patch(color=mapping[code][1], label=mapping[code][0])
                          for code in legend_order if code in unique_codes]
        plt.legend(handles=legend_patches, loc="upper right", bbox_to_anchor=(1.6, 1))
        if os.path.exists(current_folder):
            plt.savefig(f'{current_folder}//{filei[:-4]}_selected_plateaus_and_valid_MUs.png', bbox_inches='tight')
        if display_figures_inline:
            plt.show()
        else:
            plt.close()
    
    return data


In [ ]:
def Prepare_spike_trains_and_firing_rates_for_analyses(data, current_subject, current_condition, display_plots=False):
    # Get list of MU being valid or not based on selected plateaus and whether or not they are continuous on the plateaus
    nb_MUs = len(data[current_subject][current_condition]['MU_spike_times_list'])
    valid_MUs_given_selected_plateaus = data[current_subject][current_condition]['valid_MUs_given_selected_plateaus']
    # Get selected plateaus and their corresponding samples
    selected_plateaus = data[current_subject][current_condition]['selected_plateaus']
    sample_ranges_corresponding_to_selected_plateaus = [
        data[current_subject][current_condition]['plateau_samples_at_each_plateau'][i]
        for i in selected_plateaus]
    total_selected_samples_nb = 0
    for range_plateau_i in sample_ranges_corresponding_to_selected_plateaus:
        total_selected_samples_nb += len(range_plateau_i)
    # Initializing preprocessed smoothed discharge rates on selected plateaus as a np.array [nb_MUs x time (sum of samples of selected plateaus)], with nan values for invalid MUs => For factor analysis
    preprocessed_smoothed_firing_rates_during_selected_plateaus = np.full((nb_MUs, total_selected_samples_nb), np.nan)
    smoothed_firing_rates_full_temp = data[current_subject][current_condition]['smoothed_firing_rates']
    # Initializing binary discharge matrix on selected plateaus as a np.array [nb_MUs x time (sum of samples of selected plateaus)], with nan values for invalid MUs => For factor analysis with surrogate data, and coherence analysis
    binary_discharge_matrix_during_selected_plateaus = np.full((nb_MUs, total_selected_samples_nb), np.nan)
    spike_times_during_selected_plateaus = [[] for _ in range(nb_MUs)]
    spike_times_full_temp = data[current_subject][current_condition]['spike_times']
    # Filling the initialized matrix
    end_of_previous_range = -1
    end_of_each_selected_plateau = []
    selected_plateau_samples_idx_total = []
    for range_plateau_i in sample_ranges_corresponding_to_selected_plateaus:
        current_sample_range = np.arange(len(range_plateau_i))+1+end_of_previous_range
        selected_plateau_samples_idx_total.extend(np.arange(range_plateau_i[0], range_plateau_i[-1]+1))
        for mn_i in range(nb_MUs): # valid_MUs_given_selected_plateaus:
            # Find corresponding grid and the index within the grid
            grid_of_MU = data[current_subject][current_condition]['MU_grid_idx_list'][mn_i]
            nb_of_MUs_in_previous_grids = len(np.where(np.array(data[current_subject][current_condition]['MU_grid_idx_list'])<grid_of_MU)[0])
            idx_of_MN_within_grid = mn_i - nb_of_MUs_in_previous_grids
            # Smoothed discharge rates - preprocess
            if mn_i in valid_MUs_given_selected_plateaus:
                temp_smoothed_firing_rate = np.copy(smoothed_firing_rates_full_temp[grid_of_MU][idx_of_MN_within_grid])
                temp_smoothed_firing_rate = temp_smoothed_firing_rate[range_plateau_i]
                temp_smoothed_firing_rate = detrend(temp_smoothed_firing_rate)
                temp_smoothed_firing_rate = temp_smoothed_firing_rate - np.mean(temp_smoothed_firing_rate)
                temp_smoothed_firing_rate = temp_smoothed_firing_rate / np.std(temp_smoothed_firing_rate)
                preprocessed_smoothed_firing_rates_during_selected_plateaus[mn_i,current_sample_range] = temp_smoothed_firing_rate
            # Binary spike times
            temp_spike_times = np.copy(spike_times_full_temp[grid_of_MU][idx_of_MN_within_grid])
            temp_spike_times = temp_spike_times[(temp_spike_times >= range_plateau_i[0]) & (temp_spike_times <= range_plateau_i[-1])]
            temp_spike_times -= range_plateau_i[0]
            temp_binary_spike_times = np.zeros(len(range_plateau_i))
            temp_binary_spike_times[temp_spike_times.astype(int)] = 1
            binary_discharge_matrix_during_selected_plateaus[mn_i,current_sample_range] = temp_binary_spike_times
            spike_times_during_selected_plateaus[mn_i].extend(temp_spike_times + end_of_previous_range + 1)
        # end of "for each MU"
        end_of_previous_range = current_sample_range[-1]
        end_of_each_selected_plateau.append(end_of_previous_range)
    # end of "for each plateau"

    # ASSIGN VALUES
    data[current_subject][current_condition]['preprocessed_smoothed_firing_rates_during_selected_plateaus'] = preprocessed_smoothed_firing_rates_during_selected_plateaus
    data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'] = binary_discharge_matrix_during_selected_plateaus
    data[current_subject][current_condition]['spike_times_during_selected_plateaus'] = spike_times_during_selected_plateaus
    data[current_subject][current_condition]['selected_plateaus_samples_idx'] = selected_plateau_samples_idx_total

    if display_plots:
        # PLOT - smoothed firing rates
        plt.figure(figsize=(30,6))
        previous_muscle_of_MN = ""
        for mn_i in range(nb_MUs):
            muscle_of_MN = data[current_subject][current_condition]['MU_muscle_list'][mn_i]
            color_of_plot = globals()[f"{muscle_of_MN}_color"]
            if muscle_of_MN != previous_muscle_of_MN:
                plt.plot(np.arange(total_selected_samples_nb)/input_fsamp, preprocessed_smoothed_firing_rates_during_selected_plateaus[mn_i,:], color=color_of_plot, alpha = 1, linewidth = 1.5,
                        label = f"{muscle_of_MN}")
            else:
                plt.plot(np.arange(total_selected_samples_nb)/input_fsamp, preprocessed_smoothed_firing_rates_during_selected_plateaus[mn_i,:], color=color_of_plot, alpha = 0.1, linewidth = 1.5)
            previous_muscle_of_MN = muscle_of_MN
        plt.xlabel("Second")
        plt.ylabel("Firing rate (detrended and normalized)")
        plt.title(f"Preprocessed smoothed firing rates on selected plateaus, only valid MUs\nSubject {current_subject} - Condition {current_condition}")
        plt.axvline(x=0, color='black', linestyle='--')
        for end_i in end_of_each_selected_plateau:
            plt.axvline(x=end_i/input_fsamp, color='black', linestyle='--', linewidth = 3, alpha=0.5)
        plt.xlim((-1,total_selected_samples_nb/input_fsamp+1))
        plt.legend(loc="upper right")
        for end_i in end_of_each_selected_plateau:
            plt.axvline(x=end_i, color='black', linestyle='--')
        if os.path.exists(current_folder):
            plt.savefig(f'{current_folder}//{filei[:-4]}_preprocessed_smoothed_firing_rates_selected.png', bbox_inches='tight')
        if display_figures_inline:
            plt.show()
        else:
            plt.close()

        # PLOT - spike trains
        plt.figure(figsize=(30,6))
        # valid_count = -1
        for mn_i in range(nb_MUs):
            muscle_of_MN = data[current_subject][current_condition]['MU_muscle_list'][mn_i]
            color_of_plot = globals()[f"{muscle_of_MN}_color"]
            if mn_i not in data[current_subject][current_condition]['valid_MUs_given_selected_plateaus']:
                color_of_plot = mix_colors(color_of_plot, "#979797") # invalid MUs are greyed out
                previous_muscle_of_MN = muscle_of_MN
            if muscle_of_MN != previous_muscle_of_MN:
                plt.scatter(np.array(spike_times_during_selected_plateaus[mn_i])/input_fsamp, np.ones(len(spike_times_during_selected_plateaus[mn_i]))*(mn_i+1),
                            color=color_of_plot, alpha = 0.5, marker = '|', label = f"{muscle_of_MN}")
            else:
                plt.scatter(np.array(spike_times_during_selected_plateaus[mn_i])/input_fsamp, np.ones(len(spike_times_during_selected_plateaus[mn_i]))*(mn_i+1),
                            color=color_of_plot, alpha = 0.5, marker = '|')
            previous_muscle_of_MN = muscle_of_MN
        plt.axvline(x=0, color='black', linestyle='--')
        for end_i in end_of_each_selected_plateau:
            plt.axvline(x=end_i/input_fsamp, color='black', linestyle='--', linewidth = 3, alpha=0.5)
        plt.title(f"Spike trains on selected plateaus\nSubject {current_subject} - Condition {current_condition}")
        plt.xlabel("Second")
        plt.ylabel("MN #")
        if os.path.exists(current_folder):
            plt.savefig(f'{current_folder}//{filei[:-4]}_spike_trains_selected.png', bbox_inches='tight')
        if display_figures_inline:
            plt.show()
        else:
            plt.close()

    return data

In [ ]:
# --------------------------------------
# Helper functions for CSV
# --------------------------------------
def append_to_csv(file_path, row_dict, fieldnames):
    """
    Appends a single dictionary 'row_dict' to a CSV file at 'file_path',
    creating the file with header if it doesn't exist.
    """
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode='a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row_dict)

def write_big_csv(file_path, data_list, fieldnames):
    """
    Writes a list of dictionary rows (data_list) into a CSV file at file_path,
    overwriting if it already exists (no old rows accumulate).
    """
    with open(file_path, mode='w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in data_list:
            writer.writerow(row)

In [ ]:
# Initialize an empty dictionary to store the data
data = {}

### LOAD EXPERIMENTAL DATA (.mat)

nb_of_files = -1

# Build default field names for the MU properties CSV
fieldnames = ["subject", "condition","MU_idx","muscle_of_MU","grid",
              "Recruitment_thresholds","Recruitment_threshold_mean","firing_rates","firing_rates_mean","cov","cov_mean"]
MU_properties_all_subjects_csv = []
all_subjects_csv_path = path_of_files

for filei in files_to_analyze_experimental_data:
    nb_of_files += 1

    file_path = os.path.join(path_of_files, filei)
    current_subject, current_condition = filei.replace('.mat', '').split('_', 1)
    print("Currently loading...")
    print("     current_subject = ", current_subject)
    print("     current_condition = ", current_condition)

    # Create a new folder for each participant, and sub-folders for each condition
    path_for_participant = os.path.join(path_of_files, current_subject)
    path_for_participant_and_condition = os.path.join(path_for_participant, f'{current_subject}_{current_condition}')
    if not os.path.exists(path_for_participant):
        os.makedirs(path_for_participant)
    if not os.path.exists(path_for_participant_and_condition):
        os.makedirs(path_for_participant_and_condition)
    current_folder = path_for_participant_and_condition

    # Initialize dict keys
    if current_subject not in data:
        data[current_subject] = {}
    if current_condition not in data[current_subject]:
        data[current_subject][current_condition] = {}
    current_muscle = current_condition.split('_')[0]
    nb_grids = nb_grids_origin[current_muscle]

    # Load .mat file
    data[current_subject][current_condition] = load_file(file_path)

    # Overwrite local CSV => remove if it already exists
    subject_and_condition_csv_path = os.path.join(path_for_participant_and_condition, "MU_properties.csv")
    if os.path.exists(subject_and_condition_csv_path):
        os.remove(subject_and_condition_csv_path)

    # Reorganize EMG data
    data[current_subject][current_condition]['EMG'] = reorganize_emg(
        data[current_subject][current_condition]['EMG'], 
        data[current_subject][current_condition]['channels_mask']
    )

    # Estimate MUAP onset times
    data[current_subject][current_condition]['spike_times'] = realign_spike_times(
        data[current_subject][current_condition]['EMG'], 
        data[current_subject][current_condition]['spike_times'], 
        sampling_rate=input_fsamp, 
        display_muap=False, #display_muap=True, 
        filename=filei
    )

    # Normalize and scale force data
    # Use a regular expression to extract the number
    match_number = re.search(r'\d+', current_condition)
    if match_number:
        extracted_number = float(match_number.group())  # Convert the extracted number to float
        # print("Extracted number:", extracted_number)
        current_condition_force = extracted_number
    else:
        print("No number found in the condition name")
        if ('abduction' in current_condition) or ('flexion' in current_condition) or ('Eyes' in current_condition):
            current_condition_force = 20
        else:
            current_condition_force = 100
    if force_is_3D:
        (normalized_force, plateau_samples, plateau_samples_at_each_plateau,
        force_psd_each_plateau, force_psd_total, force_std_each_plateau, force_coeff_of_variation_each_plateau) = normalize_force_and_identify_plateaus(
            data[current_subject][current_condition]['force_3D'],
            input_fsamp=input_fsamp, 
            display_plot=True, 
            condition=current_condition_force,  # Scaling by current condition
            filename=filei
        )
    else:
        (normalized_force, plateau_samples, plateau_samples_at_each_plateau,
        force_psd_each_plateau, force_psd_total, force_std_each_plateau, force_coeff_of_variation_each_plateau) = normalize_force_and_identify_plateaus(
            data[current_subject][current_condition]['force'],
            input_fsamp=input_fsamp, 
            display_plot=True, 
            condition=current_condition_force,  # Scaling by current condition
            filename=filei
        )      
    # if current_condition == 'FDI_flexion':
    #     sys.error("stop")
    data[current_subject][current_condition]['force'] = normalized_force
    data[current_subject][current_condition]['plateau_samples'] = plateau_samples
    data[current_subject][current_condition]['plateau_samples_at_each_plateau'] = plateau_samples_at_each_plateau
    data[current_subject][current_condition]['force_psd'] = {}
    data[current_subject][current_condition]['force_psd']['total'] = force_psd_total
    data[current_subject][current_condition]['force_psd']['each_plateau'] = force_psd_each_plateau
    data[current_subject][current_condition]['force_coeff_of_var_each_plateau'] = force_coeff_of_variation_each_plateau
    data[current_subject][current_condition]['force_std_each_plateau'] = force_std_each_plateau 

    # Call process_firing_rates function to update the existing 'data' structure
    data = process_firing_rates(
        data, current_subject, current_condition, input_fsamp=input_fsamp, Wind_s=Wind_s, display_plots=True)

    del data[current_subject][current_condition]['EMG'] # removing the EMG data to save memory

    # Assign muscle names to the grids and motor units
    current_muscle = current_condition.split('_')[0]
    # current_contraction = current_condition.split('_')[1]
    data[current_subject][current_condition]['muscle_per_grid'] = []
    if current_muscle == 'Quadriceps':
        for gridi in range(nb_grids_origin[current_muscle]):
            if gridi in VM_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('VM')
            elif gridi in VL_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('VL')
    elif current_muscle == 'TA':
        for gridi in range(nb_grids_origin[current_muscle]):
            data[current_subject][current_condition]['muscle_per_grid'].append('TA')
    elif current_muscle == 'TricepsSurae':
        for gridi in range(nb_grids_origin[current_muscle]):
            if gridi in SOL_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('SOL')
            elif gridi in GM_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('GM')
    elif current_muscle == 'Fingers':
        for gridi in range(nb_grids_origin[current_muscle]):
            if gridi in FDI_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('FDI')
            elif gridi in APB_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('APB')
    elif current_muscle == 'FDI':
        for gridi in range(nb_grids_origin[current_muscle]):
            if gridi in FDI_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('FDI')
    elif current_muscle == 'VL':
        for gridi in range(nb_grids_origin[current_muscle]):
            if gridi in FDI_grids:
                data[current_subject][current_condition]['muscle_per_grid'].append('VL')
    
    # Compute recruitment thresholds
    data[current_subject][current_condition]['MU_properties'] = compute_recruitment_thresholds_and_firing_rates(
        data[current_subject][current_condition]['force'], 
        data[current_subject][current_condition]['spike_times'],
        data[current_subject][current_condition]['plateau_samples_at_each_plateau'],
        time_window_to_check_for_ramp=7,  # in seconds
        lowpass_filter_force_freq = 1,
        positive_fraction_threshold = 0.5,
        derivative_epsilon = 0.05,
        threshold_for_MU_to_be_continuous = ISI_thresh_for_discontinuity,
        input_fsamp=input_fsamp,  # Sampling rate
        display_plot=True,
        filename=filei
    )

    # Flatten the spike times dictionnary into a single list of NumPy arrays
    #  flattened_list = [array for arrays in data[current_subject][current_condition]['spike_times'].values() for array in arrays]
    MU_spike_times_list = []
    MU_grid_idx_list = []
    for key, arrays in data[current_subject][current_condition]['spike_times'].items():
        MU_spike_times_list.extend(arrays)  # Add all arrays to the flattened list
        MU_grid_idx_list.extend([int(key)] * len(arrays))  # Add the key for each array
    MU_all_recruitment_thresholds_list = []
    MU_mean_recruitment_thresholds_list = np.zeros(len(MU_spike_times_list))
    MU_all_firing_rates_list = []
    MU_mean_firing_rates_list = np.zeros(len(MU_spike_times_list))
    MU_all_cov_list = []
    MU_mean_cov_list = np.zeros(len(MU_spike_times_list))
    MU_empty = np.zeros(len(MU_spike_times_list))
    MU_muscle_list = []

    current_grid = -1
    current_mu_within_grid = -1
    for mui in range(len(MU_spike_times_list)):
        # Get corresponding grid and within-grid MU idx
        current_grid_previous = current_grid
        current_grid = MU_grid_idx_list[mui]
        if current_grid_previous != current_grid:
            current_mu_within_grid = -1
        current_mu_within_grid += 1
        # find corresponding muscle MUs
        MU_muscle_list.append(data[current_subject][current_condition]['muscle_per_grid'][current_grid])
        # find empty MUs
        if len(MU_spike_times_list[mui]) < 1:
            MU_empty[mui] = 1
        else: # Get properties
            MU_all_recruitment_thresholds_list.append(np.array(
                data[current_subject][current_condition]['MU_properties'][current_grid][current_mu_within_grid]['recruitment_thresholds']))
            MU_mean_recruitment_thresholds_list[mui] = np.nanmean(
                data[current_subject][current_condition]['MU_properties'][current_grid][current_mu_within_grid]['recruitment_thresholds'])
            MU_all_firing_rates_list.append(np.array(
                data[current_subject][current_condition]['MU_properties'][current_grid][current_mu_within_grid]['mean_firing_rates']))
            MU_mean_firing_rates_list[mui] = np.nanmean(
                data[current_subject][current_condition]['MU_properties'][current_grid][current_mu_within_grid]['mean_firing_rates'])
            MU_all_cov_list.append(np.array(
                data[current_subject][current_condition]['MU_properties'][current_grid][current_mu_within_grid]['mean_cov']))
            MU_mean_cov_list[mui] = np.nanmean(
                data[current_subject][current_condition]['MU_properties'][current_grid][current_mu_within_grid]['mean_cov'])
        
    # Remove empty MUs from all lists
    MU_spike_times_list = [MU_spike_times_list[i] for i in range(len(MU_spike_times_list)) if MU_empty[i] == 0]
    MU_grid_idx_list = [MU_grid_idx_list[i] for i in range(len(MU_grid_idx_list)) if MU_empty[i] == 0]
    MU_muscle_list = [MU_muscle_list[i] for i in range(len(MU_muscle_list)) if MU_empty[i] == 0]
    MU_mean_recruitment_thresholds_list = [MU_mean_recruitment_thresholds_list[i] for i in range(len(MU_mean_recruitment_thresholds_list)) if MU_empty[i] == 0]
    MU_mean_firing_rates_list = [MU_mean_firing_rates_list[i] for i in range(len(MU_mean_firing_rates_list)) if MU_empty[i] == 0]
    MU_mean_cov_list = [MU_mean_cov_list[i] for i in range(len(MU_mean_cov_list)) if MU_empty[i] == 0]

    data[current_subject][current_condition]['Empty_MUs_removed_idx'] = MU_empty
    data[current_subject][current_condition]['MU_spike_times_list'] = MU_spike_times_list
    data[current_subject][current_condition]['MU_grid_idx_list'] = MU_grid_idx_list
    data[current_subject][current_condition]['MU_muscle_list'] = MU_muscle_list
    data[current_subject][current_condition]['MU_mean_recruitment_thresholds_list'] = MU_mean_recruitment_thresholds_list
    data[current_subject][current_condition]['MU_mean_firing_rates_list'] = MU_mean_firing_rates_list
    data[current_subject][current_condition]['MU_mean_cov_list'] = MU_mean_cov_list

    # Update iteratively a csv with MU properties, which is saved at the end (once all files have been loaded)
    for mui in range(len(MU_spike_times_list)):
        row = {
            "subject":current_subject,
            "condition":current_condition,
            "MU_idx":mui,
            "muscle_of_MU":MU_muscle_list[mui],
            "grid":MU_grid_idx_list[mui],
            "Recruitment_thresholds":MU_all_recruitment_thresholds_list[mui],
            "Recruitment_threshold_mean":MU_mean_recruitment_thresholds_list[mui],
            "firing_rates":MU_all_firing_rates_list[mui],
            "firing_rates_mean":MU_mean_firing_rates_list[mui],
            "cov": MU_all_cov_list[mui],
            "cov_mean": MU_mean_cov_list[mui]
        }
        # Write to the local CSV (for this subject)
        append_to_csv(subject_and_condition_csv_path, row, fieldnames)
        # Also add to the global list for the big CSV (all subjects)
        MU_properties_all_subjects_csv.append(row)
    # end of "for each MU" / "each csv row"

    data = Select_plateaus_for_dim_and_coh_analyses(data, current_subject, current_condition, display_plots=True, minimum_mu_coverage = min_proportion_of_MUs_to_keep)
    data = Prepare_spike_trains_and_firing_rates_for_analyses(data, current_subject, current_condition, display_plots=True)

    if export_processed_spike_trains_as_hdf5_file_for_new_analysis:
        spike_trains_output_as_hdf5_file = os.path.join(path_of_files, f"{current_subject}_{current_condition}.h5")
        with h5py.File(spike_trains_output_as_hdf5_file, "w") as f:
            # Save onlythe motoneuron and pool indices + the spike trains
            # 1) motoneuron and pool indices
            mn_pool_grp = f.create_group("motoneurons_and_pools_indices")
            # (a) pool_list_by_MN -- a string array
            pool_list = data[current_subject][current_condition]['MU_muscle_list']
            # make a numpy array of dtype "variable‐length UTF‐8 string"
            str_dt = string_dtype(encoding="utf-8")
            mn_pool_grp.create_dataset(
                "pool_list_by_MN",
                data=np.array(pool_list, dtype=str_dt),
                dtype=str_dt)
            # (b) idx_of_MN_by_pool -- subgroup of integer arrays
            by_pool_grp = mn_pool_grp.create_group("idx_of_MN_by_pool")
            idx_of_MN_by_pool = {}
            for mn_idx, pool_name in enumerate(pool_list):
                if pool_name not in idx_of_MN_by_pool:
                    idx_of_MN_by_pool[pool_name] = []
                idx_of_MN_by_pool[pool_name].append(mn_idx)
            for poolname, idx_array in idx_of_MN_by_pool.items():
                idx_array = np.array(idx_array)
                # poolname is something like "VM", "VL", ...
                by_pool_grp.create_dataset(poolname, data=idx_array.astype(int))

            # 2) spike trains
            spikes = f.create_group("spike_trains")
            mn_spikes = spikes.create_group("MN")
            for i, tr in enumerate(data[current_subject][current_condition]['MU_spike_times_list']):
                mn_spikes.create_dataset(f"MN_{i}", data=tr)
            print(f"Processed spike trains saved to {spike_trains_output_as_hdf5_file}")
    # end of "if export_processed_spike_trains_as_hdf5_file_for_new_analysis"

# End of "for each file"
gc.collect()

# --------------------------------------------------------------------
# Write results of MU properties for all subjects
# --------------------------------------------------------------------
all_subjects_csv_path = os.path.join(path_of_files, "MU_properties_ALL.csv")
write_big_csv(all_subjects_csv_path, MU_properties_all_subjects_csv, fieldnames)


In [ ]:
# CSV AND FIGURES ABOUT FORCE VARIABILITY & PSD

# -----------------------------
# Params & helpers
# -----------------------------
# Set to the same fmax used when computing PSDs (default in your code was 10.0)
FMAX_USED = 10.0
EPS = 1e-20

# Save folder (optional); if you have a variable path_of_files already, we’ll use it.
PATH_OF_FILES = f"{path_of_files}" # globals().get("path_of_files", ".")
# os.makedirs(PATH_OF_FILES, exist_ok=True)

# Your color map and special-case mapping
MUSCLE_COLORS = {
    "Quadriceps": "#ff3c00",
    "TA": "#00c921",
    "TricepsSurae": "#0066ff",
    "FDI": "#00aeda",
}
DEFAULT_COLOR = "gray"

CORRESPONDANCE_MUSCLE_SPECIAL_CASES = {
    "Fingers": "FDI"
}

def map_muscle(m):
    return CORRESPONDANCE_MUSCLE_SPECIAL_CASES.get(m, m)

def parse_condition(cond):
    # expects "Muscle_Intensity", e.g., "Quadriceps_10percent"
    if "_" in cond:
        m, i = cond.split("_", 1)
    else:
        m, i = cond, ""
    return map_muscle(m), i

def safe_mean_list(x_list):
    if x_list is None:
        return np.nan
    xs = [x for x in x_list if x is not None]
    return float(np.nan) if len(xs) == 0 else float(np.nanmean(xs))

def first_psd_and_len(data):
    # Find one PSD vector to infer NBINS
    for subj, dsub in data.items():
        for cond, dcond in dsub.items():
            psd = dcond.get("force_psd", {}).get("each_plateau", None)  # mean PSD across plateaus
            if psd is not None:
                psd = np.asarray(psd, dtype=float)
                return psd, psd.size
    return None, None

# Determine frequency bin centers from PSD length
psd_sample, NBINS = first_psd_and_len(data)
if NBINS is None:
    raise RuntimeError("No PSD found in data to infer frequency bins.")
dfreq = FMAX_USED / NBINS
freq_centers = (np.arange(NBINS) + 0.5) * dfreq  # matches earlier bin-centers style

# Intensity ordering
INTENSITY_ORDER = ["10percent", "40percent"]

# -----------------------------
# 1) CSV: variability per plateau (std & cov)
# -----------------------------
stats_rows = []
subjects = sorted(list(data.keys()))

for subj in subjects:
    for cond, dcond in data[subj].items():
        muscle, intensity = parse_condition(cond)

        std_list = dcond.get("force_std_each_plateau", None)                 # list per plateau
        cov_list = dcond.get("force_coeff_of_var_each_plateau", None)        # list per plateau

        if std_list is None and cov_list is None:
            continue

        n_plateaus = 0
        if isinstance(std_list, (list, tuple)):
            n_plateaus = max(n_plateaus, len(std_list))
        if isinstance(cov_list, (list, tuple)):
            n_plateaus = max(n_plateaus, len(cov_list))

        for p_idx in range(n_plateaus):
            std_val = None
            cov_val = None
            if isinstance(std_list, (list, tuple)) and p_idx < len(std_list):
                std_val = std_list[p_idx]
            if isinstance(cov_list, (list, tuple)) and p_idx < len(cov_list):
                cov_val = cov_list[p_idx]

            stats_rows.append({
                "subject": subj,
                "muscle": muscle,
                "intensity": intensity,
                "plateau_index": p_idx + 1,
                "std": std_val,
                "cov": cov_val,
            })

df_stats = pd.DataFrame(stats_rows)
df_stats = df_stats.sort_values(["subject", "muscle", "intensity", "plateau_index"]).reset_index(drop=True)

# Save CSV
stats_csv_path = os.path.join(PATH_OF_FILES, "force_variability_by_plateau.csv")
df_stats.to_csv(stats_csv_path, index=False)
print(f"Saved: {os.path.abspath(stats_csv_path)}")

# -----------------------------
# 2) CSV: mean PSD (across plateaus) by condition
#     one row per subject × muscle × intensity, columns = frequencies
# -----------------------------
psd_rows = []
for subj in subjects:
    for cond, dcond in data[subj].items():
        muscle, intensity = parse_condition(cond)
        psd_mean = dcond.get("force_psd", {}).get("each_plateau", None)  # mean across plateaus
        if psd_mean is None:
            continue
        psd_mean = np.asarray(psd_mean, dtype=float)
        if psd_mean.size != NBINS:
            # if binning differs, you could resample; for now, skip to keep rectangular
            continue

        row = {
            "subject": subj,
            "muscle": muscle,
            "intensity": intensity,
        }
        for k, f in enumerate(freq_centers):
            row[f"f_{f:.2f}Hz"] = psd_mean[k]
        psd_rows.append(row)

df_psd = pd.DataFrame(psd_rows)
df_psd = df_psd.sort_values(["subject", "muscle", "intensity"]).reset_index(drop=True)

psd_csv_path = os.path.join(PATH_OF_FILES, "force_psd_mean_by_condition.csv")
df_psd.to_csv(psd_csv_path, index=False)
print(f"Saved: {os.path.abspath(psd_csv_path)}")

# -----------------------------
# 3) FIGURE: std & cov (4 subplots)
# -----------------------------
# Intensities present, ordered
intensities = sorted(
    {parse_condition(c)[1] for subj in subjects for c in data[subj].keys()},
    key=lambda x: (INTENSITY_ORDER.index(x) if x in INTENSITY_ORDER else 999, x)
)
muscles = sorted({parse_condition(c)[0] for subj in subjects for c in data[subj].keys()})

# Collect per-subject means (over plateaus) for each metric
per_subj_std = defaultdict(lambda: defaultdict(dict))  # [intensity][muscle][subject] = mean std
per_subj_cov = defaultdict(lambda: defaultdict(dict))  # [intensity][muscle][subject] = mean cov

for subj in subjects:
    for cond, dcond in data[subj].items():
        muscle, intensity = parse_condition(cond)
        std_mean = safe_mean_list(dcond.get("force_std_each_plateau", None))
        cov_mean = safe_mean_list(dcond.get("force_coeff_of_var_each_plateau", None))
        if not np.isnan(std_mean):
            per_subj_std[intensity][muscle][subj] = std_mean
        if not np.isnan(cov_mean):
            per_subj_cov[intensity][muscle][subj] = cov_mean

def gather_vals(metric_dict):
    vals = []
    for intensity in intensities[:2]:  # expect two
        for muscle in muscles:
            vals.extend(list(metric_dict[intensity].get(muscle, {}).values()))
    vals = np.array(vals, dtype=float)
    vals = vals[np.isfinite(vals)]
    if vals.size == 0:
        return (0.0, 1.0)
    lo, hi = float(np.nanmin(vals)), float(np.nanmax(vals))
    if lo == hi:
        lo, hi = (lo - 0.1, hi + 0.1) if hi == 0 else (0.9*lo, 1.1*hi)
    return lo, hi

# std_ylim = gather_vals(per_subj_std)
# cov_ylim = gather_vals(per_subj_cov)
std_ylim = [0, gather_vals(per_subj_std)[1]*1.1]
cov_ylim = [0, gather_vals(per_subj_cov)[1]*1.1]

fig, axes = plt.subplots(2, 2, figsize=(16, 10), sharex="col")
for col, intensity in enumerate(intensities[:2]):
    # Top: STD
    ax = axes[0, col]
    xs = np.arange(len(muscles))
    for i_m, m in enumerate(muscles):
        color = MUSCLE_COLORS.get(m, DEFAULT_COLOR)
        vals = list(per_subj_std[intensity].get(m, {}).values())
        # jittered subject points
        for j, v in enumerate(vals):
            ax.scatter(i_m + (j - len(vals)/2) * 0.02, v, s=30, color=color, alpha=0.7)
        # diamond = mean per muscle
        if len(vals):
            ax.scatter(i_m, np.mean(vals), s=140, marker="D", color=color, edgecolor="k", zorder=3)
    ax.set_title(f"STD (mean over plateaus) — {intensity}")
    ax.set_ylabel("STD (%MVC)")
    ax.set_ylim(std_ylim)
    ax.set_xticks(xs)
    ax.set_xticklabels(muscles, rotation=0)

    # Bottom: COV
    ax = axes[1, col]
    for i_m, m in enumerate(muscles):
        color = MUSCLE_COLORS.get(m, DEFAULT_COLOR)
        vals = list(per_subj_cov[intensity].get(m, {}).values())
        for j, v in enumerate(vals):
            ax.scatter(i_m + (j - len(vals)/2) * 0.02, v, s=30, color=color, alpha=0.7)
        if len(vals):
            ax.scatter(i_m, np.mean(vals), s=140, marker="D", color=color, edgecolor="k", zorder=3)
    ax.set_title(f"COV (mean over plateaus) — {intensity}")
    ax.set_ylabel("COV (ratio)")
    ax.set_ylim(cov_ylim)
    ax.set_xticks(xs)
    ax.set_xticklabels(muscles, rotation=0)

fig.suptitle("Force variability by muscle and intensity (points = subjects; diamonds = muscle means)", y=1.02)
fig.tight_layout()
plt.savefig(f"{path_of_files}force_variability_by_muscle_and_intensity.png", bbox_inches='tight')
plt.show()


# -----------------------------
# 4) FIGURE: PSD in dB — grid (rows=muscles, cols=intensities)
# -----------------------------
def collect_psd_db_limits(intensity):
    vals = []
    for subj in subjects:
        for cond, dcond in data[subj].items():
            m, inten = parse_condition(cond)
            if inten != intensity:
                continue
            psd_mean = dcond.get("force_psd", {}).get("each_plateau", None)
            if psd_mean is None: 
                continue
            arr = 10 * np.log10(np.asarray(psd_mean, dtype=float) + EPS)
            vals.append(arr)
    if not len(vals):
        return (-100, 0)
    allv = np.concatenate(vals)
    allv = allv[np.isfinite(allv)]
    if allv.size == 0:
        return (-100, 0)
    lo, hi = float(np.nanmin(allv)), float(np.nanmax(allv))
    if lo == hi:
        lo, hi = lo - 1.0, hi + 1.0
    return (lo, hi)

n_rows = len(muscles)
n_cols = min(2, len(intensities))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 3*n_rows), sharex=True)

if n_rows == 1 and n_cols == 1:
    axes = np.array([[axes]])
elif n_rows == 1:
    axes = axes.reshape(1, n_cols)
elif n_cols == 1:
    axes = axes.reshape(n_rows, 1)

# per-column y-lims in dB
col_ylims_db = {col: collect_psd_db_limits(intensities[col]) for col in range(n_cols)}

for i_m, muscle in enumerate(muscles):
    for col, intensity in enumerate(intensities[:n_cols]):
        ax = axes[i_m, col]
        color = MUSCLE_COLORS.get(muscle, DEFAULT_COLOR)
        for subj in subjects:
            for cond, dcond in data[subj].items():
                m, inten = parse_condition(cond)
                if m != muscle or inten != intensity:
                    continue
                psd_mean = dcond.get("force_psd", {}).get("each_plateau", None)
                if psd_mean is None:
                    continue
                psd_mean = np.asarray(psd_mean, dtype=float)
                if psd_mean.size != NBINS:
                    continue
                ax.plot(freq_centers, 10*np.log10(psd_mean + EPS), color=color, alpha=0.5)
        ax.set_title(f"{muscle} — {intensity}")
        ax.set_ylim(col_ylims_db[col])
        if i_m == n_rows - 1:
            ax.set_xlabel("Frequency (Hz)")
        if col == 0:
            ax.set_ylabel("PSD (dB re (%%MVC)^2/Hz)")

fig.suptitle("Mean PSD across plateaus by muscle & intensity (per-subject curves, dB scale)", y=1.02)
fig.tight_layout()
plt.savefig(f"{path_of_files}force_PSD_by_muscle_and_intensity_decibels.png", bbox_inches='tight')
plt.show()


# -----------------------------
# 5) FIGURE: PSD linear — same layout, linear y; per-intensity limits
# -----------------------------
def collect_psd_lin_limits(intensity):
    vals = []
    for subj in subjects:
        for cond, dcond in data[subj].items():
            m, inten = parse_condition(cond)
            if inten != intensity:
                continue
            psd_mean = dcond.get("force_psd", {}).get("each_plateau", None)
            if psd_mean is None:
                continue
            arr = np.asarray(psd_mean, dtype=float)
            vals.append(arr)
    if not len(vals):
        return (0.0, 1.0)
    allv = np.concatenate(vals)
    allv = allv[np.isfinite(allv)]
    if allv.size == 0:
        return (0.0, 1.0)
    lo, hi = float(np.nanmin(allv)), float(np.nanmax(allv))
    if lo == hi:
        lo, hi = (lo - 0.1, hi + 0.1) if hi == 0 else (0.9*lo, 1.1*hi)
    return (lo, hi)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 3*n_rows), sharex=True)
if n_rows == 1 and n_cols == 1:
    axes = np.array([[axes]])
elif n_rows == 1:
    axes = axes.reshape(1, n_cols)
elif n_cols == 1:
    axes = axes.reshape(n_rows, 1)

col_ylims_lin = {col: collect_psd_lin_limits(intensities[col]) for col in range(n_cols)}

for i_m, muscle in enumerate(muscles):
    for col, intensity in enumerate(intensities[:n_cols]):
        ax = axes[i_m, col]
        color = MUSCLE_COLORS.get(muscle, DEFAULT_COLOR)
        for subj in subjects:
            for cond, dcond in data[subj].items():
                m, inten = parse_condition(cond)
                if m != muscle or inten != intensity:
                    continue
                psd_mean = dcond.get("force_psd", {}).get("each_plateau", None)
                if psd_mean is None:
                    continue
                psd_mean = np.asarray(psd_mean, dtype=float)
                if psd_mean.size != NBINS:
                    continue
                ax.plot(freq_centers, psd_mean, color=color, alpha=0.5)
        ax.set_title(f"{muscle} — {intensity}")
        ax.set_ylim(col_ylims_lin[col])
        if i_m == n_rows - 1:
            ax.set_xlabel("Frequency (Hz)")
        if col == 0:
            ax.set_ylabel("PSD (units^2/Hz)")

fig.suptitle("Mean PSD across plateaus by muscle & intensity (per-subject curves, linear scale)", y=1.02)
fig.tight_layout()
plt.savefig(f"{path_of_files}force_PSD_by_muscle_and_intensity_linear.png", bbox_inches='tight')
plt.show()




In [ ]:
import os
import pandas as pd

# Function to get list of matched CSV files
def get_match_csv_files(folder_path, substring):
    matching_files = [
        os.path.join(folder_path, file)  # Full path
        for file in os.listdir(folder_path) 
        if file.endswith(".csv") and substring in file
    ]
    return matching_files

substring = "match"
match_csv_files = get_match_csv_files(path_of_files, substring)

match_files_data = {}

# Loop through the matched files
for file in match_csv_files:
    # Extract just the filename (without the full path)
    filename_only = os.path.basename(file)  # Extract filename from path
    filename_without_ext = os.path.splitext(filename_only)[0]  # Remove .csv extension
    parts = filename_without_ext.split("_")  # Split by "_"

    if len(parts) >= 3:  # Ensure there are at least three parts (subject, muscle, "match")
        subject = parts[0]  # First substring before the first "_"
        muscle = parts[1]   # Second substring between first and second "_"
    else:
        subject, muscle = None, None  # Handle unexpected filename format
    
    # Load CSV file
    df = pd.read_csv(file)

    # Store column names in a dictionary (filename -> column names)
    column_names = df.columns.tolist()

    conditions_matched_temp = []
    matched_MU_idx = {}
    for columni in range(len(column_names)):
        if columni > 0:  # Skip first iteration (idx of MUs)
            conditions_matched_temp.append(column_names[columni])
            matched_MU_idx[column_names[columni]] = df.iloc[:, columni].values

    # Store the extracted information
    match_files_data[subject] = {}
    match_files_data[subject][muscle] = {}
    for condition in conditions_matched_temp:
        match_files_data[subject][muscle][condition] = {
            "filename": filename_only,  # Use only filename, not full path
            "MUs_idx_list_arbitrary": df.iloc[:, 0].values,
            "matched_MUs_idx": matched_MU_idx[condition] - 1,  # -1 because of 0-indexing
            "full_table": df
        }

# # # # # LOOP THROUGH DATA, AND ADD MATCHED INFO, IF ANY # # # # #
for current_subject in data.keys():
    for current_condition in data[current_subject].keys():
        data[current_subject][current_condition]['matched_MUs_idx'] = []  # Initialize as empty list
        splitted_condition = current_condition.split('_')
        current_muscle = splitted_condition[0]
        if len(splitted_condition) > 1:
            current_task = splitted_condition[1]
            # print(current_subject, current_condition, current_muscle, current_task)
            if current_subject in match_files_data:
                if current_muscle in match_files_data[current_subject]:
                    if current_task in match_files_data[current_subject][current_muscle]:
                        data[current_subject][current_condition]['matched_MUs_idx'] = match_files_data[current_subject][current_muscle][current_task]['matched_MUs_idx']
        data[current_subject][current_condition]['matched_MUs_idx_global'] = []  # Initialize as empty list
        data[current_subject][current_condition]['matched_MUs_idx_local_per_grid'] = {}  # Initialize as empty list
        if len(data[current_subject][current_condition]['matched_MUs_idx'])>0:
            for mui_matched in data[current_subject][current_condition]['matched_MUs_idx']:
                corresponding_grid = (mui_matched // 100) - 1 # -1 because of 0-indexing
                corresponding_local_idx = mui_matched % 100 # -1 because of 0-indexing already performed earlier
                if corresponding_grid not in data[current_subject][current_condition]['matched_MUs_idx_local_per_grid'].keys():
                    data[current_subject][current_condition]['matched_MUs_idx_local_per_grid'][corresponding_grid] = []
                data[current_subject][current_condition]['matched_MUs_idx_local_per_grid'][corresponding_grid].append(corresponding_local_idx)
                nb_of_MUs_in_all_previous_grids = 0
                for gridi in range(corresponding_grid):
                    nb_of_MUs_in_all_previous_grids += len(data[current_subject][current_condition]['spike_times'][gridi])
                data[current_subject][current_condition]['matched_MUs_idx_global'].append(nb_of_MUs_in_all_previous_grids + corresponding_local_idx)
        # convert back to numpy arrays
        data[current_subject][current_condition]['matched_MUs_idx_global'] = np.flipud(np.array(data[current_subject][current_condition]['matched_MUs_idx_global']))
        for gridi in data[current_subject][current_condition]['matched_MUs_idx_local_per_grid'].keys():
            data[current_subject][current_condition]['matched_MUs_idx_local_per_grid'][gridi] = np.array(data[current_subject][current_condition]['matched_MUs_idx_local_per_grid'][gridi])

# Update matched_MUs_idx_global based on the kept MUs
for current_subject in data.keys():
    for current_condition in data[current_subject].keys():
        if 'matched_MUs_idx_global' in data[current_subject][current_condition]:
            # Retrieve the mask for valid (kept) MUs
            empty_MUs_mask = data[current_subject][current_condition].get('Empty_MUs_removed_idx', None)

            if empty_MUs_mask is not None:
                # Get the indices of MUs that were kept
                valid_indices = np.where(empty_MUs_mask == 0)[0]  # Extract indices of kept MUs
                
                # Update matched_MUs_idx_global by mapping old indices to new ones
                updated_matched_indices = []
                for old_idx in data[current_subject][current_condition]['matched_MUs_idx_global']:
                    if old_idx in valid_indices:
                        new_idx = np.where(valid_indices == old_idx)[0][0]  # Find new index
                        updated_matched_indices.append(new_idx)

                # Convert back to numpy array and store the updated indices
                data[current_subject][current_condition]['matched_MUs_idx_global'] = np.array(updated_matched_indices)



In [ ]:
# Generate surrogate signal function (used to estimate the dimensionality obtained FA)
# Create surrogate random signal
# Only for the first pool

def Create_shuffled_surrogate_spike_trains(spike_times_list, signal_length, perform_preprocessing = True, nb_shuffle_iter=1, sanity_check_plot = False):
    taper_duration = len(HanningW) / input_fsamp  # Duration of the taper (artefact removals) in seconds
    smoothed_shuffled_firing_rates = []
    print(f'Creating {nb_shuffle_iter} surrogate shuffled signal(s)')
    for shuffli in range(nb_shuffle_iter):
        print('     Shuffling iteration ' + str(shuffli) + '...')
        # initialize the output array
        smoothed_shuffled_firing_rates.append(np.zeros((len(spike_times_list),signal_length)) )
        for mni in range(len(spike_times_list)):
            shuffled_ISI_temp = np.random.permutation(np.diff(spike_times_list[mni])) # list of random ISIs (one list per motoneuron)
            shuffled_spike_times_temp = np.cumsum(shuffled_ISI_temp)
            shuffled_binary_spike_train_temp = np.zeros(signal_length)
            shuffled_binary_spike_train_temp[shuffled_spike_times_temp.astype(int)] = 1
            # Smooth using filtfilt
            shuffled_binary_firing_rate_temp = filtfilt(HanningW, 1, shuffled_binary_spike_train_temp * input_fsamp)
            # Compute the median of the signal
            signal_median = np.median(shuffled_binary_firing_rate_temp)
            # Define taper length in samples
            taper_samples = int(taper_duration * input_fsamp)
            # Create a Hanning window taper
            taper_window = windows.hann(taper_samples * 2)  # Symmetric taper
            # Apply taper to the beginning
            shuffled_binary_firing_rate_temp[:taper_samples] *= taper_window[:taper_samples]
            shuffled_binary_firing_rate_temp[:taper_samples] += (1 - taper_window[:taper_samples]) * signal_median  # Blend with median
            # Apply taper to the end
            shuffled_binary_firing_rate_temp[-taper_samples:] *= taper_window[-taper_samples:]
            shuffled_binary_firing_rate_temp[-taper_samples:] += (1 - taper_window[-taper_samples:]) * signal_median  # Blend with median
            if perform_preprocessing:
                shuffled_binary_firing_rate_temp = detrend(shuffled_binary_firing_rate_temp)
                shuffled_binary_firing_rate_temp = shuffled_binary_firing_rate_temp - np.mean(shuffled_binary_firing_rate_temp)
                if np.std(shuffled_binary_firing_rate_temp) > 0.001: # avoid division by zero
                    shuffled_binary_firing_rate_temp = shuffled_binary_firing_rate_temp / np.std(shuffled_binary_firing_rate_temp)
            # Store the final smoothed and tapered signal
            smoothed_shuffled_firing_rates[-1][mni, :] = shuffled_binary_firing_rate_temp
            
    sanity_check_plot = False
    if sanity_check_plot:
        # PLOT - smoothed firing rates from first spike times shuffling iteration
        plt.figure(figsize=(12,5))
        previous_muscle_of_MN = ""
        for mn_i in range(len(spike_times_list)):
            muscle_of_MN = data[current_subject][current_condition]['MU_muscle_list'][mn_i]
            color_of_plot = globals()[f"{muscle_of_MN}_color"]
            if muscle_of_MN != previous_muscle_of_MN:
                plt.plot(np.arange(signal_length)/input_fsamp, smoothed_shuffled_firing_rates[0][mn_i,:], color=color_of_plot, alpha = 0.5, linewidth = 1.5,
                        label = f"{muscle_of_MN}")
            else:
                plt.plot(np.arange(signal_length)/input_fsamp, smoothed_shuffled_firing_rates[0][mn_i,:], color=color_of_plot, alpha = 0.1, linewidth = 1.5)
            previous_muscle_of_MN = muscle_of_MN
        plt.xlabel("Second")
        plt.ylabel("Firing rates")
        plt.title(f"Smoothed firing rates of surrogate, shuffled spike trains (1st shuffling iteration)\nSubject {current_subject} - Condition {current_condition}")
        plt.legend(loc="upper right")
        if display_figures_inline:
            plt.show()
        else:
            plt.close()

    return smoothed_shuffled_firing_rates


def Get_Rsquared_from_factor_analysis(firing_rates_array, max_nb_of_factors=6):
    # Find columns that contain NaN or infinite values
    bad_columns = np.where(np.logical_or(np.isnan(firing_rates_array), np.isinf(firing_rates_array)).any(axis=0))[0]

    if bad_columns.size > 0:
        print(f"Warning: Removing columns with NaN or infinite values at indices: {bad_columns.tolist()}")
        firing_rates_array = np.delete(firing_rates_array, bad_columns, axis=1)

    # Check for columns with near-zero variance
    stds = np.std(firing_rates_array, axis=0)
    zero_std_columns = np.where(stds < 1e-10)[0]
    if zero_std_columns.size > 0:
        print(f"Warning: Removing columns with near-zero variance at indices: {zero_std_columns.tolist()}")
        firing_rates_array = np.delete(firing_rates_array, zero_std_columns, axis=1)

    # Initialize storage for results
    loadings = []
    scores = []
    reconstructed_Rsquared = []

    for factori in range(max_nb_of_factors - 1):
        fa = FactorAnalyzer(n_factors=factori + 1, rotation='promax')
        fa.fit(firing_rates_array)
        loadings.append(fa.loadings_)  # weights (coefficients) for each MU
        scores.append(fa.transform(firing_rates_array))  # The latent factors

        # Get R² value of FA's data reconstruction
        reconstructed_data_temp = np.dot(loadings[factori], scores[factori].T)
        Rsquared_per_MN_temp = []
        
        for mni in range(firing_rates_array.shape[1]):
            r = np.corrcoef(firing_rates_array[:, mni], reconstructed_data_temp[mni])[0, 1]
            Rsquared_per_MN_temp.append(r ** 2)  # Compute R² as r²
        
        reconstructed_Rsquared.append(np.mean(Rsquared_per_MN_temp))

    # Add 0 at the beginning of the array
    reconstructed_Rsquared = np.insert(reconstructed_Rsquared, 0, 0.0)

    return reconstructed_Rsquared, loadings, scores



In [ ]:
import matplotlib.ticker as ticker

def plot_factor_loadings(loadings, matched_idx, ax):
    """
    Plots factor loadings as:
    - A bar plot if n_factors == 1
    - A 2D biplot if n_factors == 2
    - A 3D biplot if n_factors == 3
    - No plot if n_factors < 1 or > 3

    Parameters:
    - loadings: (MxN) array of factor loadings.
    """

    n_factors = loadings.shape[1]  # Number of selected factors

    # Check if the number of factors is valid
    if n_factors < 1 or n_factors > 3:
        print(f"Factor Analysis selected {n_factors} factors. A biplot cannot be displayed.")
        return

    # plt.figure(figsize=(8, 8))

    if n_factors == 1:
        # **1 Factor → Bar Plot**
        ax.bar(range(loadings.shape[0]), loadings[:, 0], color="red", alpha=0.7)
        for i in range(loadings.shape[0]):
            if i in matched_idx:
                ax.bar(i, loadings[i, 0], color="black", alpha=0, edgecolor="black")
        ax.set_xlabel("Motor units (global indices)")
        ax.set_ylabel("Factor Loadings (weights)")
        ax.set_xticks(range(loadings.shape[0]), [f"MU #{i}" for i in range(loadings.shape[0])], rotation=45)
        # ax.grid()

    elif n_factors == 2:
        # **2 Factors → 2D Biplot**
        factor_1, factor_2 = loadings[:, 0], loadings[:, 1]

        # Compute Colors Based on Factor Loadings (RGB from absolute values)
        abs_loadings = np.abs(loadings)
        normalized_loadings = abs_loadings / np.max(abs_loadings, axis=0)
        colors = np.clip(normalized_loadings, 0, 1)
        zero_column = np.zeros((colors.shape[0], 1))  # Shape (rows, 1)
        colors = np.concatenate((colors, zero_column), axis=1)

        ax.axhline(0, color='black', linestyle='--', linewidth=1.5)
        ax.axvline(0, color='black', linestyle='--', linewidth=1.5)

        for i in range(loadings.shape[0]):
            color = colors[i]  # Assign color based on factor loadings
            if i in matched_idx:
                ax.scatter(factor_1[i], factor_2[i], color='orange', s=350, alpha = 0.5)
                ax.scatter(factor_1[i], factor_2[i], color='black', s=160)
            ax.scatter(factor_1[i], factor_2[i], color=color, s=80)
            ax.plot([0, factor_1[i]], [0, factor_2[i]], color=color, alpha=0.7, linewidth=2)  # No arrowhead
            # plt.text(factor_1[i], factor_2[i], f"Var{i+1}", fontsize=12, ha='right')

        ax.grid()
        ax.axis("equal")
        ax.set_xlabel("Latent factor 1", color = "red")
        ax.set_ylabel("Latent factor 2", color = "green")

    elif n_factors == 3:
        # **3 Factors → 3D Biplot**
        factor_1, factor_2, factor_3 = loadings[:, 0], loadings[:, 1], loadings[:, 2]

        # Compute Colors Based on Factor Loadings (RGB from absolute values)
        abs_loadings = np.abs(loadings)
        normalized_loadings = abs_loadings / np.max(abs_loadings, axis=0)
        colors = np.clip(normalized_loadings, 0, 1)

        # # fig = plt.figure(figsize=(10, 8))
        # ax.remove()  # Remove the existing 2D subplot
        # ax = fig.add_subplot(axes, projection='3d')  # Convert the existing subplot to 3D

        ax.set_xlim([-1, 1])
        ax.set_ylim([-1, 1])
        ax.set_zlim([-1, 1])

        for i in range(loadings.shape[0]):
            color = colors[i]  # Assign color based on factor loadings
            ax.plot([0, factor_1[i]], [0, factor_2[i]], [0, factor_3[i]], color=color, alpha=0.7, linewidth=1.5)
            if i in matched_idx:
                ax.scatter(factor_1[i], factor_2[i], color='orange', s=350, alpha = 0.5)
                ax.scatter(factor_1[i], factor_2[i], color='black', s=160)
            ax.scatter(factor_1[i], factor_2[i], factor_3[i], color=color, s=80, edgecolors='black')

            # Projected Shadows
            shadow_alpha = 0.5
            # shadow of factors 1 and 2
            ax.plot([0, factor_1[i]], [0, factor_2[i]], [-1], color=([0,0,.5] + np.array([1, 1, 1])) / 2, alpha=shadow_alpha, linewidth=1.5)
            ax.scatter(factor_1[i], factor_2[i], -1, color=([0,0,.5] + np.array([1, 1, 1])) / 2, s=80, alpha=shadow_alpha, edgecolors='none')
            # shadow of factors 2 and 3
            ax.plot([0, factor_1[i]], [1], [0, factor_3[i]], color=([.5,0,0] + np.array([1, 1, 1])) / 2, alpha=shadow_alpha, linewidth=1.5)
            ax.scatter(factor_1[i], 1, factor_3[i], color=([.5,0,0] + np.array([1, 1, 1])) / 2, s=80, alpha=shadow_alpha, edgecolors='none')
            # shadow of factors 1 and 3
            ax.plot([-1], [0, factor_2[i]], [0, factor_3[i]], color=([0,.5,0] + np.array([1, 1, 1])) / 2, alpha=shadow_alpha, linewidth=1.5)
            ax.scatter(-1, factor_2[i], factor_3[i], color=([0,.5,0] + np.array([1, 1, 1])) / 2, s=80, alpha=shadow_alpha, edgecolors='none')

            # ax.text(factor_1[i], factor_2[i], factor_3[i], f"Var{i+1}", fontsize=10)

        ax.set_xlabel("Latent factor 1", color = "red")
        ax.set_ylabel("Latent factor 2", color = "green")
        ax.set_zlabel("Latent factor 3", color = "blue")
        # ax = fig.add_subplot(111, projection='3d')
        # set_axes_equal(ax)

        # Reduce background grid size
        ax.xaxis.set_major_locator(ticker.MaxNLocator(4))
        ax.yaxis.set_major_locator(ticker.MaxNLocator(4))
        ax.zaxis.set_major_locator(ticker.MaxNLocator(4))
        ax.set_box_aspect([1,1,1])

    # if display_figures_inline:
    #     plt.show()
    # else:
    #     plt.close()


In [ ]:
# Checkpoint to clean up some space in the memory
gc.collect()

In [ ]:
##### PERFORM DIMENSIONALITY ANALYSIS

# --------------------------------------
# User-defined parameter for the analysis
# --------------------------------------
# max_nb_of_factors = 6  # e.g., 6, but can be anything you need
# surrogate_iter_nb = 1

# ------------------------------
# PERFORM DIMENSIONALITY ANALYSIS
# ------------------------------
if perform_analysis_dimensionality:
    print("Starting dimensionality analysis...")

    # --------------------------------------
    # Build dynamic fieldnames, depending on max_nb_of_factors
    # --------------------------------------
    fieldnames = ["subject", "condition", "muscles"]
    for factor_i in range(1, max_nb_of_factors+1):
        fieldnames.append(f"R2_f{factor_i}")
    fieldnames += [
        "selected_nb_of_dimensions",
        "nb_mus",
        "avg_slope_surrogate_R2"
    ]

    # This will collect all rows (from all subjects & conditions) so that
    # we can write one "big CSV" at the end.
    all_dimensionality_results = []

    for current_subject in data.keys():
        for current_condition in data[current_subject].keys():
            data[current_subject][current_condition]['FA_results'] = {}

            # Create folder to save output (per subject, per condition)
            folder_subject_condition = os.path.join(
                path_of_files,
                current_subject,
                f'{current_subject}_{current_condition}'
            )
            if not os.path.exists(folder_subject_condition):
                os.makedirs(folder_subject_condition)

            print(f"     Dimensionality analysis for {current_subject} - {current_condition}")

            # Overwrite local CSV => remove if it already exists
            csv_path = os.path.join(folder_subject_condition, "Dimensionality_analysis_results.csv")
            if os.path.exists(csv_path):
                os.remove(csv_path)

            valid_MUs = data[current_subject][current_condition]['valid_MUs_given_selected_plateaus']
            muscles_unique = list(set(data[current_subject][current_condition]['MU_muscle_list']))
            muscle_pairs = list(itertools.combinations(muscles_unique, 2))

            # Build possible muscle groups (single muscles + pairs)
            muscle_groups_for_FA = {}
            if fa_on_first_pool_only_for_sim and (('sim' in current_subject) or ('sim' in current_condition)): # factor analysis on first pool only
                muscle_groups_for_FA[0] = muscles_unique[0]
            else:
                iter_muscle_group_FA = 0
                for muscle_i in muscles_unique:
                    muscle_groups_for_FA[iter_muscle_group_FA] = muscle_i
                    iter_muscle_group_FA += 1
                for muscle_pair_i in muscle_pairs:
                    muscle_groups_for_FA[iter_muscle_group_FA] = [muscle_pair_i[0], muscle_pair_i[1]]
                    iter_muscle_group_FA += 1
                
            # Prepare the figure
            plt.figure(figsize=(8,5))

            muscle_iter = -1
            for muscle_current_FA in muscle_groups_for_FA.values():
                if type(muscle_current_FA) == list:
                    muscle_current_FA = tuple(muscle_current_FA) # convert to tuple so that it can be used as key for a dict

                data[current_subject][current_condition]['FA_results'][muscle_current_FA] = {}
                muscle_iter += 1

                MU_muscle_list = data[current_subject][current_condition]['MU_muscle_list']
                # Valid MUs only
                MU_idx_for_FA_only_valid_MUs = [MU_muscle_list[i] for i in valid_MUs]

                # Identify the relevant MU indices
                if isinstance(muscle_current_FA, str):
                    # Single muscle
                    muscle_corresponding_MUs = [
                        i for i, val in enumerate(MU_muscle_list)
                        if val == muscle_current_FA
                    ]
                    muscle_corresponding_MUs_only_valid_MUs = [
                        i for i, val in enumerate(MU_idx_for_FA_only_valid_MUs)
                        if val == muscle_current_FA
                    ]
                else:
                    # List of muscles (pair)
                    muscle_corresponding_MUs = np.where(
                        np.isin(MU_muscle_list, muscle_current_FA)
                    )[0]
                    muscle_corresponding_MUs_only_valid_MUs = np.where(
                        np.isin(MU_idx_for_FA_only_valid_MUs, muscle_current_FA)
                    )[0]

                MU_idx_for_FA = np.intersect1d(muscle_corresponding_MUs, valid_MUs)
                if len(MU_idx_for_FA) < max_nb_of_factors+1:
                    temp_max_nb_of_factors = len(MU_idx_for_FA) - 1
                else:
                    temp_max_nb_of_factors = max_nb_of_factors

                # --------------------------------------------------
                # Surrogate data creation
                # --------------------------------------------------
                filtered_spike_times_for_surrogate_signal = [
                    data[current_subject][current_condition]['spike_times_during_selected_plateaus'][i]
                    for i in muscle_corresponding_MUs_only_valid_MUs
                ]
                surrogate_dataset = Create_shuffled_surrogate_spike_trains(
                    filtered_spike_times_for_surrogate_signal,
                    len(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'][0]),
                    perform_preprocessing=True,
                    nb_shuffle_iter=surrogate_iter_nb,
                    sanity_check_plot=False
                )

                # --------------------------------------------------
                # True data R²
                # --------------------------------------------------
                true_data_R2 = np.full(max_nb_of_factors,np.nan)
                temp_rsqured, fa_coef_for_each_MUs, latent_factors = Get_Rsquared_from_factor_analysis(
                    data[current_subject][current_condition]['preprocessed_smoothed_firing_rates_during_selected_plateaus'][MU_idx_for_FA,:].T,
                    max_nb_of_factors=temp_max_nb_of_factors
                )
                # fa_coef_for_each_MUs =
                # list of N elements, with N being the number of factors to try
                # each element is a MxF matrix, with M being the number of motor units and F being the number of factors
                true_data_R2[:len(temp_rsqured)] = temp_rsqured
                slope_true_R2 = np.gradient(true_data_R2)

                # --------------------------------------------------
                # Surrogate data R²
                # --------------------------------------------------
                surrogate_data_R2_list = []
                for shuffle_i in range(surrogate_iter_nb):
                    surrogate_data_R2_list.append(np.full(max_nb_of_factors,np.nan))
                    temp_rsqured, _, _ = Get_Rsquared_from_factor_analysis(
                            surrogate_dataset[shuffle_i].T,
                            max_nb_of_factors=temp_max_nb_of_factors)
                    surrogate_data_R2_list[-1][:len(temp_rsqured)] = temp_rsqured
                surrogate_data_R2 = np.nanmean(np.array(surrogate_data_R2_list).T, axis=1)
                slope_surrogate_R2 = np.nanmin(np.gradient(surrogate_data_R2))

                # --------------------------------------------------
                # Dimensionality threshold
                # --------------------------------------------------
                # We find first factor index 'N' where slope_true_R2 < surrogate_data_R2 => dimension = N - 1
                # Note: ensure that the index is found; otherwise, handle edge cases.
                temp_bool_dim = np.where(slope_true_R2 < slope_surrogate_R2)[0]
                add_to_legend_temp = f""
                if len(temp_bool_dim) >= 1:
                    selected_nb_of_dimensions = temp_bool_dim[0] - 1
                else:
                    # Case 1 = not enough MUs
                    if np.all(np.isnan(slope_true_R2)):
                        selected_nb_of_dimensions = 0
                    elif np.sum(np.sum(np.isnan(slope_true_R2)))>=1:
                        nan_slope_idx = np.where(np.isnan(slope_true_R2))[0]
                        # find the first nan slope value. Dimensionality = index of this value - 1
                        # Unless there is only one slope value. Then, it is the first index
                        if nan_slope_idx[0]==1:
                            selected_nb_of_dimensions = 1
                        else:
                            selected_nb_of_dimensions = nan_slope_idx[0]-1
                    # Case 2 = True R² slope never goes below surrogate R² slope
                    else:
                        selected_nb_of_dimensions = temp_max_nb_of_factors-1
                        add_to_legend_temp = f"\nWarning: True slope is never < surrogate slope"

                # Assign values of FA results for later use
                if selected_nb_of_dimensions == 0: # ensuring the script still works in this edge case (typicallhy when not enough MUs to perform FA)
                    fa_coef_for_each_MUs = {}
                    fa_coef_for_each_MUs[-1] = np.ones((len(MU_idx_for_FA),1))
                data[current_subject][current_condition]['FA_results'][muscle_current_FA]['fa_weights_for_each_MUs'] = fa_coef_for_each_MUs[selected_nb_of_dimensions-1]
                data[current_subject][current_condition]['FA_results'][muscle_current_FA]['dimensionality'] = selected_nb_of_dimensions
                data[current_subject][current_condition]['FA_results'][muscle_current_FA]['MU_idx_for_FA'] = MU_idx_for_FA

                # Get similarity matrix between MUs according to their fa_weights
                # Compute all pairwise dot products
                latent_factor_space_vectors_dot_products = np.dot(fa_coef_for_each_MUs[selected_nb_of_dimensions-1],
                    fa_coef_for_each_MUs[selected_nb_of_dimensions-1].T)  # Compute full pairwise dot-product matrix
                # latent_factor_space_vectors_dot_products = np.where(np.tril(latent_factor_space_vectors_dot_products) != 0, np.tril(latent_factor_space_vectors_dot_products), np.nan)
                data[current_subject][current_condition]['FA_results'][muscle_current_FA]['latent_factor_space_vectors_dot_products'] = latent_factor_space_vectors_dot_products

                # --------------------------------------------------
                # Plot
                # --------------------------------------------------
                #       # Plot of R²
                if isinstance(muscle_current_FA, str):
                    figure_color = globals()[f"{muscle_current_FA}_color"]
                    muscle_label = muscle_current_FA
                else:
                    figure_color = mix_colors(
                        globals()[f"{muscle_current_FA[0]}_color"],
                        globals()[f"{muscle_current_FA[1]}_color"]
                    )
                    muscle_label = str(muscle_current_FA)  # or " & ".join(muscle_current_FA)
                if ("," in muscle_label) and display_Rsquared_for_muscle_pair==False:
                    print("Not displaying R² plot for synergist muscles combination")
                else:
                    # Just an empty line to make space in the legend
                    plt.plot(0,0,color="white",label=" \n ")
                    plt.plot(
                        true_data_R2,
                        label=f"{muscle_label} - True ({len(MU_idx_for_FA)} MUs)",
                        color=figure_color,
                        linewidth=3,
                        alpha=0.7
                    )
                    plt.plot(
                        surrogate_data_R2,
                        label=f"{muscle_label} - Surrogate",
                        linestyle='--',
                        color=figure_color,
                        alpha=1
                    )
                    plt.scatter(
                        selected_nb_of_dimensions,
                        true_data_R2[selected_nb_of_dimensions],
                        s=150, edgecolors=figure_color, facecolors="white",
                        linewidth=3, zorder=10,
                        label=f"{muscle_label} - {selected_nb_of_dimensions}\ndimensions selected{add_to_legend_temp}",
                    )


                # ------------------------------------------------------
                # Prepare the row for the local CSV and big CSV
                # ------------------------------------------------------
                row = {
                    "subject":       current_subject,
                    "condition":     current_condition,
                    "muscles":      muscle_label,
                    "selected_nb_of_dimensions": selected_nb_of_dimensions,
                    "nb_mus":        len(MU_idx_for_FA),
                    "avg_slope_surrogate_R2": slope_surrogate_R2
                }
                # Add the R² values for each factor
                for factor_i in range(1, max_nb_of_factors+1):
                    row[f"R2_f{factor_i}"] = true_data_R2[factor_i - 1]

                # Write to the local CSV (for this subject + condition)
                append_to_csv(csv_path, row, fieldnames)

                # Also add to our global list for the big CSV
                all_dimensionality_results.append(row)

            # End muscle_groups_for_FA loop

            # Save figure
            plt.legend(title="Visually, dimensionality seems\nunderestimated when the number\nof MUs is low",
                       loc="upper right", bbox_to_anchor=(1.5, 1))
            plt.xlabel("Nb of factors")
            plt.ylabel("R²")
            plt.ylim([0,1])
            plt.title(
                f"Dimensionality analysis (over "
                f"{data[current_subject][current_condition]['preprocessed_smoothed_firing_rates_during_selected_plateaus'][MU_idx_for_FA, :].shape[1]/input_fsamp:.1f} s)\n"
                f"Subject {current_subject} - Condition {current_condition}"
            )
            if os.path.exists(folder_subject_condition):
                plt.savefig(
                    os.path.join(folder_subject_condition, "Dimensionality_analysis_results.png"),
                    bbox_inches='tight'
                )
            else:
                print("Output folder does not exist; figure not saved")

            # End of "for each muscle or muscle pair"
            if display_figures_inline:
                plt.show()
            else:
                plt.close()

            # # # # # # Plot of the "latent factor space" and also save results for only matched MUs
            # Create figure and subplots
            fig, axes = plt.subplots(nrows=len(muscle_groups_for_FA), ncols=2, figsize=(16, 8*len(muscle_groups_for_FA))) # Ensure axes is always iterable (even if there's only one subplot)

            if len(muscle_groups_for_FA) == 1:
                axes = [axes]
            for muscle_iter, (muscle_FA_i, ax_pair) in enumerate(zip(data[current_subject][current_condition]['FA_results'].keys(), axes)):
                ax1, ax2 = ax_pair  # First column = FA plot, Second column = heatmap
                # -------------------
                # First Column: FA Biplot
                # -------------------
                latent_factor_loadings = data[current_subject][current_condition]['FA_results'][muscle_FA_i]['fa_weights_for_each_MUs']

                # Find the indices in MU_idx_for_FA that correspond to matched_MUs
                MU_idx_for_FA = np.sort(data[current_subject][current_condition]['FA_results'][muscle_FA_i]['MU_idx_for_FA'])
                if 'matched_MUs_idx_global' in data[current_subject][current_condition].keys():
                    matched_MUs_temp = np.sort(data[current_subject][current_condition]['matched_MUs_idx_global'])
                else:
                    matched_MUs_temp = []
                matched_MUs_used_for_FA_temp = np.intersect1d(MU_idx_for_FA, matched_MUs_temp)
                matched_MUs_used_for_FA_relative_to_matched_idx = np.searchsorted(matched_MUs_temp, matched_MUs_used_for_FA_temp)
                matched_MUs_used_for_FA_relative_to_valid_FA_idx = np.searchsorted(MU_idx_for_FA, matched_MUs_used_for_FA_temp)

                n_factors = latent_factor_loadings.shape[1]  # Number of latent factors
                # If there are exactly 3 factors, convert ax1 into a 3D subplot
                if n_factors == 3:
                    ax1.remove()  # Remove the existing 2D subplot
                    ax1 = fig.add_subplot(len(muscle_groups_for_FA), 2, 2*muscle_iter+1, projection='3d')
                plot_factor_loadings(latent_factor_loadings, matched_MUs_temp, ax=ax1)  # Pass the updated ax1
                ax1.set_title(f"Projection into latent factor space for {muscle_FA_i}\nMatched MUs highlighted")
                # -------------------
                # Second Column: Heatmap of Dot Product Matrix
                # -------------------
                latent_factor_space_vectors_dot_products = data[current_subject][current_condition]['FA_results'][muscle_FA_i]['latent_factor_space_vectors_dot_products']
                heatmap = sns.heatmap(latent_factor_space_vectors_dot_products,
                    cmap="RdYlBu_r", mask=np.isnan(latent_factor_space_vectors_dot_products), 
                    annot=False, linecolor="none", cbar=True,
                    ax=ax2, vmin=-1, vmax=1)
                ax2.set_title(f"Pairwise dot products of MUs' positions in latent factor space ({muscle_FA_i})")
                ax2.set_aspect("equal")  # <-- Make the heatmap square
                # Add a label to the color bar
                cbar = heatmap.collections[0].colorbar
                cbar.ax.set_ylabel("Dot Product Value", fontsize=12)

            latent_factor_loadings_only_matched = np.full((len(matched_MUs_temp),n_factors), np.nan)
            latent_factor_loadings_only_matched[matched_MUs_used_for_FA_relative_to_matched_idx,:] = latent_factor_loadings[matched_MUs_used_for_FA_relative_to_valid_FA_idx,:]

            latent_factor_space_vectors_dot_products_only_matched = np.full((len(matched_MUs_temp),len(matched_MUs_temp)), np.nan)
            latent_factor_space_vectors_dot_products_only_matched[
                np.ix_(matched_MUs_used_for_FA_relative_to_matched_idx, matched_MUs_used_for_FA_relative_to_matched_idx)] = latent_factor_space_vectors_dot_products[
                    np.ix_(matched_MUs_used_for_FA_relative_to_valid_FA_idx, matched_MUs_used_for_FA_relative_to_valid_FA_idx)]

            data[current_subject][current_condition]['FA_results'][muscle_FA_i]['fa_weights_only_matched_MUs'] = latent_factor_loadings_only_matched
            data[current_subject][current_condition]['FA_results'][muscle_FA_i]['latent_factor_space_vectors_dot_products_only_matched_MUs'] = latent_factor_space_vectors_dot_products_only_matched

            plt.suptitle(f"Latent factor space (Subject {current_subject} - Condition {current_condition})") 
            plt.tight_layout() 
            if os.path.exists(folder_subject_condition):
                plt.savefig(
                        os.path.join(folder_subject_condition, "Dimensionality_latent_factor_space.png"),
                        bbox_inches='tight')
            else:
                print("Output folder does not exist; figure not saved")
            if display_figures_inline:
                plt.show()
            else:
                plt.close()
        # End of "for each condition"
    # End of "for each subject"

    # --------------------------------------------------------------------
    # Now write ONE big CSV with all rows, saved at 'path_of_files' level
    # --------------------------------------------------------------------
    big_csv_path = os.path.join(path_of_files, "Dimensionality_analysis_ALL.csv")
    write_big_csv(big_csv_path, all_dimensionality_results, fieldnames)

    print(f"\nA concatenated CSV with all dimensionality results was saved at:\n{big_csv_path}")

else:
    print("Dimensionality analysis (factor analysis) not performed because 'perform_analysis_dimensionality=False'.")



In [ ]:
# Visual check of the factors in latent space according to the selected MUs
for current_subject in match_files_data.keys():
    for current_muscle in match_files_data[current_subject].keys():
        conditions_compared = [current_muscle + "_" + key for key in match_files_data[current_subject][current_muscle].keys()]
        condition_combinations_list = list(itertools.combinations(conditions_compared, 2))

        plt.figure(figsize=(8, 6))  # Create a new figure for each subject & muscle
        legend_added = set()  # Track conditions already added to the legend

        # Store scatter plot data for each condition
        scatter_data = {condition: {"x": [], "y": [], "color": f"C{idx}"} for idx, condition in enumerate(conditions_compared)}

        for mni in range(len(data[current_subject][conditions_compared[0]]['matched_MUs_idx'])):
            conditions_latent_space_MU_pos = {}

            for idx, conditioni in enumerate(conditions_compared):
                muscle_FA_i = 'FDI' # temporary fix
                # Access the correct FA results
                fa_data = data[current_subject][conditioni]['FA_results'][muscle_FA_i]['fa_weights_only_matched_MUs']
                
                # ensure that there are 2 dimensions at least
                if fa_data.shape[1]<2:
                    continue

                # Ensure we don't index out of bounds
                if mni >= fa_data.shape[0]: 
                    continue

                # Store latent space positions
                conditions_latent_space_MU_pos[conditioni] = np.array([
                    fa_data[mni, 0], 
                    fa_data[mni, 1]
                ])

                # Collect scatter points per condition (avoiding repeated plt.scatter calls)
                if not np.isnan(conditions_latent_space_MU_pos[conditioni]).any():
                    scatter_data[conditioni]["x"].append(fa_data[mni, 0])
                    scatter_data[conditioni]["y"].append(fa_data[mni, 1])

            # Connect corresponding MUs across conditions
            for cond1, cond2 in condition_combinations_list:
                if cond1 in conditions_latent_space_MU_pos and cond2 in conditions_latent_space_MU_pos:
                    if not np.isnan(conditions_latent_space_MU_pos[cond1]).any() and not np.isnan(conditions_latent_space_MU_pos[cond2]).any():
                        plt.plot(
                            [conditions_latent_space_MU_pos[cond1][0], conditions_latent_space_MU_pos[cond2][0]],
                            [conditions_latent_space_MU_pos[cond1][1], conditions_latent_space_MU_pos[cond2][1]],
                            color='purple', alpha=0.5, zorder = 0
                        )

        # Now, plot each condition **once** with a unique label
        for condition, values in scatter_data.items():
            if values["x"]:  # Only plot if there are valid points
                plt.scatter(values["x"], values["y"], color=values["color"], label=condition)

        # Add legend only if there are labeled points
        if any(len(values["x"]) > 0 for values in scatter_data.values()):
            plt.legend(title="Conditions", loc="upper right")

        plt.xlabel("Latent Factor 1")
        plt.ylabel("Latent Factor 2")
        plt.title(f"{current_subject}, {current_muscle}\nComparison of Matched MUs in Latent Space")
        if display_figures_inline:
            plt.show()
        else:
            plt.close()


In [ ]:
# Compare recruitment thresholds across matched conditions
condition_dodge_visual = 0.1

for current_subject in match_files_data.keys():
    for current_muscle in match_files_data[current_subject].keys():
        conditions_compared = [current_muscle + "_" + key for key in match_files_data[current_subject][current_muscle].keys()]
        condition_combinations_list = list(itertools.combinations(conditions_compared, 2))

        plt.figure(figsize=(15, 5)) 

        for cond1, cond2 in condition_combinations_list:
            # Retrieve matched indices for condition 1
            matched_idx_current_cond1 = data[current_subject][cond1]['matched_MUs_idx_local_per_grid']
            matched_idx_current_cond2 = data[current_subject][cond2]['matched_MUs_idx_local_per_grid']

            recruitment_thresh_cond1 = {}
            mean_recruitment_thresh_cond1 = []
            # Compute mean recruitment thresholds for ordering in condition 1
            for gridi in matched_idx_current_cond1.keys():
                recruitment_thresh_cond1[gridi] = []
                for mni in matched_idx_current_cond1[gridi]:  # Loop through matched MUs
                    current_RTs = data[current_subject][cond1]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds']
                    mean_RTs = np.nanmean(current_RTs)  # Compute mean recruitment threshold
                    mean_recruitment_thresh_cond1.append((gridi, mni, mean_RTs))  # Store for sorting
            # Sort MUs based on their mean recruitment threshold in condition 1
            mean_recruitment_thresh_cond1.sort(key=lambda x: x[2])  # Sort by mean RTs
            # Build a sorted order list based on condition 1
            sorted_indices_cond1 = {gridi: [] for gridi in matched_idx_current_cond1.keys()}
            for sorted_index, (gridi, mni, _) in enumerate(mean_recruitment_thresh_cond1):
                sorted_indices_cond1[gridi].append(mni)  # Save sorted order per grid
            
            # Create a mapping for `cond2` to ensure we fetch correct indices
            sorted_indices_cond2 = {gridi: [] for gridi in matched_idx_current_cond2.keys()}
            for gridi in sorted_indices_cond1.keys():
                sorted_indices_cond2[gridi] = [
                    matched_idx_current_cond2[gridi][
                        np.where(matched_idx_current_cond1[gridi] == mni)[0][0]
                    ] if mni in matched_idx_current_cond1[gridi] else None  # Map only existing indices
                    for mni in sorted_indices_cond1[gridi] ]

            # Plot recruitment thresholds for condition 1
            matched_idx_iter_accumulator = -1
            mean_RTs_cond1 = []
            for gridi in sorted_indices_cond1.keys():
                for mni in sorted_indices_cond1[gridi]:  # Use sorted indices
                    matched_idx_iter_accumulator += 1
                    current_RTs = data[current_subject][cond1]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds']
                    mean_RTs_cond1.append(np.nanmean(current_RTs))
                    plt.scatter(x=np.ones_like(current_RTs) * matched_idx_iter_accumulator - condition_dodge_visual, y=current_RTs, s=50, color="C0", alpha=0.5)
                    plt.scatter(x=matched_idx_iter_accumulator - condition_dodge_visual, y=mean_RTs_cond1[-1], s=100, color="C0", alpha=1, edgecolors='black')
            plt.plot(np.arange(len(sorted_indices_cond1[gridi]))-condition_dodge_visual, mean_RTs_cond1, color="C0", alpha=0.5, linewidth=2, label = cond1)

            recruitment_thresh_cond2 = {}
            mean_recruitment_thresh_cond2 = []
            # Compute mean recruitment thresholds for ordering in condition 2
            for gridi in matched_idx_current_cond2.keys():
                recruitment_thresh_cond2[gridi] = []
                for mni in matched_idx_current_cond2[gridi]:  # Loop through matched MUs
                    current_RTs = data[current_subject][cond2]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds']
                    mean_RTs = np.nanmean(current_RTs)  # Compute mean recruitment threshold
                    mean_recruitment_thresh_cond2.append((gridi, mni, mean_RTs))  # Store for sorting
            # Sort MUs based on their mean recruitment threshold in condition 2
            mean_recruitment_thresh_cond2.sort(key=lambda x: x[2])  # Sort by mean RTs

            ###### Deordering in condition 2 relative to condition 1 ######################################
            # Build the appropriate mapping between the matched indices of different conditions ###########
            # Extract condition types (assuming the format "muscle_condition")
            cond1_type = cond1.split('_')[1]  # e.g. "abduction"
            cond2_type = cond2.split('_')[1]  # e.g. "flexion"
            # Retrieve the full table for the current subject/muscle. 
            # (Assuming it's the same for both conditions, or you can pick one of them.)
            full_table = match_files_data[current_subject][current_muscle][cond1_type]['full_table']
            # Ensure full_table is a pandas DataFrame.
            # Now build a mapping from condition1 indices to condition2 indices:
            if cond1_type == 'abduction' and cond2_type == 'flexion':
                mapping_dict = dict(zip(full_table[cond1_type], full_table[cond2_type]))
            else:
                raise ValueError("Unexpected condition types.")
            #### Actual deoredering calculations ###############################################
            rank_cond1 = np.arange(len(mean_recruitment_thresh_cond1))
            rank_cond2 = np.zeros(len(mean_recruitment_thresh_cond1))  # same length since we compare matched MUs
            iter_mn_accumulator = -1
            for _, mni, _ in mean_recruitment_thresh_cond1:
                iter_mn_accumulator += 1
                # Convert the index from condition 1 to the corresponding index in condition 2
                mapped_mni = mapping_dict.get(mni+101, None) # +101 is a quick fix because there is only one grid (+100), so the mapping is consistent. The +1 is for the 1-indexing of Matlab
                if mapped_mni is None:
                    index_match = np.nan
                else:
                    # Look for the mapped index in the sorted condition 2 list.
                    index_match = next((i for i, tup in enumerate(mean_recruitment_thresh_cond2) if tup[1] == mapped_mni-101), np.nan) # -100 is a quick fix because there is only one grid, so the mapping is consistent. The -1 is for the 1-indexing of Matlab
                rank_cond2[iter_mn_accumulator] = index_match
            ranking_difference = rank_cond2 - rank_cond1

            # Plot recruitment thresholds for condition 2 using the corrected order
            matched_idx_iter_accumulator = -1
            mean_RTs_cond2 = []
            for gridi in sorted_indices_cond2.keys():
                for mni in sorted_indices_cond2[gridi]:  # Use the remapped order for `cond2`
                    matched_idx_iter_accumulator += 1
                    if mni is not None:  # Only plot valid mappings
                        current_RTs = data[current_subject][cond2]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds']
                        mean_RTs_cond2.append(np.nanmean(current_RTs))
                        plt.scatter(x=np.ones_like(current_RTs) * matched_idx_iter_accumulator + condition_dodge_visual, y=current_RTs, s=50, color="C1", alpha=0.5)
                        plt.scatter(x=matched_idx_iter_accumulator + condition_dodge_visual, y=np.nanmean(current_RTs), s=100, color="C1", alpha=1, edgecolors='black')
                        # Plot change in recruitment ordering
                        plt.arrow(x = matched_idx_iter_accumulator, y = mean_RTs_cond1[matched_idx_iter_accumulator],
                                  dx = 0, dy = ranking_difference[matched_idx_iter_accumulator],
                                  width = 0.03, length_includes_head = True, overhang = 0,
                                  head_width=0.15, head_length=0.5, color = 'purple', alpha = 1)
                        if ranking_difference[matched_idx_iter_accumulator] < 0:
                            annot_text = f'{ranking_difference[matched_idx_iter_accumulator]:.0f}'
                        else:
                            annot_text = f'+{ranking_difference[matched_idx_iter_accumulator]:.0f}'
                        plt.text(matched_idx_iter_accumulator, mean_RTs_cond1[matched_idx_iter_accumulator] + 1.5,
                            annot_text,
                            fontsize=11, color='purple', ha='center', weight='bold',
                            bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))
            plt.plot(np.arange(len(sorted_indices_cond1[gridi]))+condition_dodge_visual, mean_RTs_cond2, color="C1", alpha=0.5, linewidth=2, label = cond2)
            
        ########################## Compute spearman correlation coefficient (rho): ##########################
        rho_between_cond, _ = stats.spearmanr(mean_RTs_cond1, mean_RTs_cond2)

        # Compute intra-class Spearman correlation for cond1
        # Get the number of trials from the first motor unit (assuming all have same length)
        n_trials = 0
        for gridi in sorted_indices_cond1.keys():
            if sorted_indices_cond1[gridi]:
                for mni in range(len(data[current_subject][cond1]['recruitment_thresholds'][gridi])):
                    mni_n_trials_temp = len(data[current_subject][cond1]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds'])
                    if mni_n_trials_temp > n_trials:
                        n_trials = mni_n_trials_temp
        # Build a list of arrays, one per trial, where each array collects the trial's recruitment thresholds
        trial_thresholds_cond1 = [ [] for _ in range(n_trials) ]
        for gridi in sorted_indices_cond1.keys():
            for mni in sorted_indices_cond1[gridi]:
                current_RTs = data[current_subject][cond1]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds']
                # If current_RTs is an array (e.g., one value per trial) then add each trial value:
                for t in range(n_trials):
                        if t >= len(current_RTs):
                            trial_thresholds_cond1[t].append(np.nan)
                        else:
                            trial_thresholds_cond1[t].append(current_RTs[t])
        # Convert lists to numpy arrays for convenience:
        trial_thresholds_cond1 = [np.array(trial_data) for trial_data in trial_thresholds_cond1]
        # Compute all pairwise Spearman correlations across trials:
        rho_values_cond1 = []
        for t1 in range(n_trials - 1):
            for t2 in range(t1 + 1, n_trials):
                # Create a mask selecting indices where both t1 and t2 arrays are non-NaN
                mask = ~np.isnan(trial_thresholds_cond1[t1]) & ~np.isnan(trial_thresholds_cond1[t2])
                # Ensure there are at least two points to compute a correlation
                if np.sum(mask) < 2:
                    continue
                rho, _ = stats.spearmanr(trial_thresholds_cond1[t1][mask], trial_thresholds_cond1[t2][mask])
                rho_values_cond1.append(rho)
        # For a summary measure, take the mean of these correlations
        intra_corr_cond1 = np.nanmean(rho_values_cond1)
        print(f"Intra-class (within cond1) Spearman correlation: {intra_corr_cond1:.2f}")

        # Repeat a similar procedure for cond2:
        n_trials2 = 0
        for gridi in sorted_indices_cond2.keys():
            if sorted_indices_cond2[gridi]:
                for mni in range(len(data[current_subject][cond2]['recruitment_thresholds'][gridi])):
                    mni_n_trials_temp = len(data[current_subject][cond2]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds'])
                    if mni_n_trials_temp > n_trials2:
                        n_trials2 = mni_n_trials_temp
        trial_thresholds_cond2 = [ [] for _ in range(n_trials2) ]
        for gridi in sorted_indices_cond2.keys():
            for mni in sorted_indices_cond2[gridi]:
                if mni is not None:  # if the mapping is valid
                    current_RTs = data[current_subject][cond2]['recruitment_thresholds'][gridi][mni]['recruitment_thresholds']
                    for t in range(n_trials2):
                        if t >= len(current_RTs):
                            trial_thresholds_cond2[t].append(np.nan)
                        else:
                            trial_thresholds_cond2[t].append(current_RTs[t])
        trial_thresholds_cond2 = [np.array(trial_data) for trial_data in trial_thresholds_cond2]
        rho_values_cond2 = []
        for t1 in range(n_trials2 - 1):
            for t2 in range(t1 + 1, n_trials2):
                # Create a mask selecting indices where both t1 and t2 arrays are non-NaN
                mask = ~np.isnan(trial_thresholds_cond2[t1]) & ~np.isnan(trial_thresholds_cond2[t2])
                # Ensure there are at least two points to compute a correlation
                if np.sum(mask) < 2:
                    continue
                rho, _ = stats.spearmanr(trial_thresholds_cond2[t1][mask], trial_thresholds_cond2[t2][mask])
                rho_values_cond2.append(rho)
        intra_corr_cond2 = np.nanmean(rho_values_cond2)
        print(f"Intra-class (within cond2) Spearman correlation: {intra_corr_cond2:.2f}")
        
        plt.xlabel("Matched motor units (sorted by condition 1 mean recruitment thresholds)")
        plt.ylabel("Recruitment Thresholds (% MVC)")
        plt.title(f"Comparison of Recruitment Thresholds - {current_subject}, {current_muscle}, {conditions_compared[0]} VS {conditions_compared[1]}\nSpearman's (rank-based) correlation coefficient = {rho_between_cond:.2f}\nIntra-class correlation: within {conditions_compared[0]} = {intra_corr_cond1:.2f} +/- {np.nanstd(rho_values_cond1):.2f}; within {conditions_compared[1]} = {intra_corr_cond2:.2f} +/- {np.nanstd(rho_values_cond2):.2f}")
        plt.legend()
        if display_figures_inline:
            plt.show()
        else:
            plt.close()


In [ ]:
def COH_between_csts(cst1, cst2, fsamp=2048, coh_window_length=1, coh_windows_overlap = 0.5, max_freq=100):
    n_per_seg = coh_window_length * fsamp # convert window length to samples instead of s
    n_overlap = np.round(n_per_seg * coh_windows_overlap).astype(int)
    n_fft = np.round(upsampling_frequency_resolution * n_per_seg).astype(int)
    # Compute intra-group coherence for group 1
    f, COH_intragroup_X = csd(detrend(cst1), detrend(cst1), window=windows.hann(n_per_seg), noverlap=n_overlap, nfft=n_fft, fs=fsamp)
    # Compute intra-group coherence for group 2
    f, COH_intragroup_Y = csd(detrend(cst2), detrend(cst2), window=windows.hann(n_per_seg), noverlap=n_overlap, nfft=n_fft, fs=fsamp)
    # Compute inter-group coherence
    f, COH_intergroup = csd(detrend(cst1), detrend(cst2), window=windows.hann(n_per_seg), noverlap=n_overlap, nfft=n_fft, fs=fsamp)

    # Limiting the frequencies up to the max frequency we are interested in (for efficiency)
    max_index = np.where(f[f<=max_freq])[0][-1]
    COH_intragroup_X = COH_intragroup_X[:max_index]
    COH_intragroup_Y = COH_intragroup_Y[:max_index]
    COH_intergroup = COH_intergroup[:max_index]
    frequency_bins = f[:max_index]

    coherence_over_frequencies = (np.abs(COH_intergroup) ** 2) / (COH_intragroup_X * COH_intragroup_Y) # Welch's method of coherence calculation

    return frequency_bins, coherence_over_frequencies

In [ ]:
def shuffle_isi_spike_train(spike_train):
    """
    Generate a surrogate spike train by shuffling inter-spike intervals.
    Parameters
    ----------
    spike_train : 1D array_like of {0,1}
        Binary vector where 1 indicates a spike and 0 no spike.
    Returns
    -------
    surrogate : 1D numpy.ndarray of {0,1}
        A new binary spike train of the same length, with the exact same
        number of spikes and the same set of zero-run lengths, but in random order.
    """
    spike_train = np.asarray(spike_train, dtype=int)
    T = spike_train.size
    spike_idxs = np.flatnonzero(spike_train)

    # If no spikes (or only one), just return a copy
    if spike_idxs.size == 0:
        return spike_train.copy()

    # 1) Extract zero-run lengths (including leading & trailing)
    zero_runs = []
    # leading zeros before the first spike
    zero_runs.append(spike_idxs[0])
    # zeros between spikes
    for i in range(spike_idxs.size - 1):
        run_len = spike_idxs[i+1] - spike_idxs[i] - 1
        zero_runs.append(run_len)
    # trailing zeros after the last spike
    zero_runs.append(T - spike_idxs[-1] - 1)

    # 2) Shuffle the zero-run lengths
    shuffled = np.random.permutation(zero_runs)

    # 3) Re-build the train: place each spike after its preceding zero-run
    surrogate = np.zeros(T, dtype=int)
    # position of the first spike
    pos = shuffled[0]
    for i in range(spike_idxs.size):
        if pos < T:
            surrogate[pos] = 1
        # move past this spike + next zero-run
        pos = pos + 1 + shuffled[i+1]

    return surrogate


In [ ]:
# PERFORM COHERENCE ANALYSIS

# --------------------------------------
# User-defined parameter for the analysis
# --------------------------------------
# coh_window_length = 1  # in s
# coh_windows_overlap = 0.75 # % relative to coh_window_length
# # frequency resolution will be = 1/coh_window_length. If coh_window_length = 10s for instance, resolution will be 0.1hz
# upsampling_frequency_resolution = 5 # for the 'nfft' parameter of scipy.csd(). Interpolates the coherence values over the frequencies, to smooth the figure a bit.
# # For example, if the resolution defined by coh_window_length is 1hz, and if upsampling_frequency_resolution = 2, the resolution will be upsampled to 0.5hz
# max_freq = 60 # maximum frequency that is kept
# COH_calc_max_iteration_nb_per_group_size = 1 # More iteration for smaller group sizes, because the value obtained is very dependent upon the exact neurons selected, especially when only a few MNs are used to create the CST
# running mock coherence calculation to get the frequency bins, given the parameters
freq_bins, _ = COH_between_csts(np.zeros(input_fsamp*coh_window_length), np.zeros(input_fsamp*coh_window_length),
                                fsamp=input_fsamp, coh_window_length=coh_window_length, coh_windows_overlap = coh_windows_overlap,
                                max_freq = max_freq)

# --------------------------------------
# Build dynamic fieldnames, depending on nb of frequency bins
# --------------------------------------
fieldnames = ["subject", "condition", "muscles","frequency","coherence","nb_of_MNs_in_group_for_COH_calculation"]
# This will collect all rows (from all subjects & conditions) so that
# we can write one "big CSV" at the end.
all_COH_results = []

if perform_analysis_coherence:
    print("Starting coherence analysis")
    for current_subject in data.keys():
        for current_condition in data[current_subject].keys():
            # Create folder to save output (per subject, per condition)
            folder_subject_condition = os.path.join(path_of_files,current_subject,
                f'{current_subject}_{current_condition}' )
            if not os.path.exists(folder_subject_condition):
                os.makedirs(folder_subject_condition)
            print(f"     Coherence analysis for {current_subject} - {current_condition}")
            # Overwrite local CSV => remove if it already exists
            csv_path = os.path.join(folder_subject_condition, "Coherence_analysis_results.csv")
            if os.path.exists(csv_path):
                os.remove(csv_path)
                
            # valid_MUs = data[current_subject][current_condition]['valid_MUs_given_selected_plateaus']
            # Set all MUs as valid (no need to have a ISI cutoff for COH)
            valid_MUs = np.arange(len(data[current_subject][current_condition]['MU_muscle_list']))
            muscles_unique = list(set(data[current_subject][current_condition]['MU_muscle_list']))
            muscle_pairs = list(itertools.combinations(muscles_unique, 2))

            # Re-organize the binary spike trains from list to matrix (because this is the expected format) = already the case for experimental data, but needs to be done for simulated data
            if type(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'])==list:
                data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'] = np.array(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'])

            # Build possible muscle groups (single muscles + pairs)
            muscle_groups_for_COH = {}
            iter_muscle_group_COH = 0
            for muscle_i in muscles_unique:
                muscle_groups_for_COH[iter_muscle_group_COH] = muscle_i
                iter_muscle_group_COH += 1
            for muscle_pair_i in muscle_pairs:
                muscle_groups_for_COH[iter_muscle_group_COH] = [muscle_pair_i[0], muscle_pair_i[1]]
                iter_muscle_group_COH += 1

            # Prepare the figure
            nb_subplots = len(muscle_groups_for_COH)
            plt.figure(figsize=(nb_subplots*7,6))

            muscle_iter = -1
            for muscle_current_COH in muscle_groups_for_COH.values():
                muscle_iter += 1
                MU_muscle_list = data[current_subject][current_condition]['MU_muscle_list']
                # Identify the relevant MU indices
                muscle_corresponding_MUs = [] # store as list, as we may consider 1 (within-muscle) or 2 (between-muscles) pools of MUs
                if isinstance(muscle_current_COH, str):
                    # Single muscle
                    muscle_corresponding_MUs.append([i for i, val in enumerate(MU_muscle_list) if val == muscle_current_COH])
                else:
                    # List of muscles (pair)
                    for muscle_i in muscle_current_COH:
                        muscle_corresponding_MUs.append([i for i, val in enumerate(MU_muscle_list) if val == muscle_i])
                MU_idx_for_COH = []
                binary_spike_trains_for_COH_analysis_temp = []
                nb_motoneurons_temp = []
                for muscle_corresponding_MUs_iter in muscle_corresponding_MUs:
                    MU_idx_for_COH.append(np.intersect1d(muscle_corresponding_MUs_iter, valid_MUs))
                    binary_spike_trains_for_COH_analysis_temp.append(np.squeeze(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'][[MU_idx_for_COH[-1]]]))
                    # If only one MN, make sure binary_spike_trains_for_COH_analysis_temp is still 2D
                    if len(binary_spike_trains_for_COH_analysis_temp[-1].shape)==1:
                        binary_spike_trains_for_COH_analysis_temp[-1] = binary_spike_trains_for_COH_analysis_temp[-1][np.newaxis, :]
                    nb_motoneurons_temp.append(binary_spike_trains_for_COH_analysis_temp[-1].shape[0])
                
                # From groups of 1 up to half of the motor neurons
                shuffled_idx_list_COH = []
                for muscle_i in range(len(nb_motoneurons_temp)):
                    shuffled_idx_list_COH.append(list.copy(list(range(nb_motoneurons_temp[muscle_i])))) # initialize list of indices to be shuffled iteratively
                if len(nb_motoneurons_temp) == 1: # Case with only one muscle (within-muscle coherence)
                    COH_calc_group_size_nb = int(np.floor(nb_motoneurons_temp[0]/2))
                else: # Case with two muscles (between-muscles coherence)
                    COH_calc_group_size_nb = np.min(nb_motoneurons_temp)
                COH_pooled_per_group_size = []
                # For the surrogate-based CI calculations
                surrogate_COH_for_CI = [] # If this the is the largest group size, compute the confidence interval with the surrogate spike trains method
                printed_shuffle_already = False
                n_shuffles = 100
                alphas = [0.05, 0.01]

                for group_sizi in range(COH_calc_group_size_nb): # for each size of group
                    coherence_current_group_size = []
                    COH_calc_iteration_nb_per_group_size_temp = int(np.max([np.round(COH_calc_max_iteration_nb_per_group_size/(group_sizi+1)),1]))
                    print(f'        {muscle_current_COH} - Iterating for groups of {group_sizi+1} motoneurons (out of a max group size of {COH_calc_group_size_nb}) - {COH_calc_iteration_nb_per_group_size_temp} iterations')

                    for group_iteri in range(COH_calc_iteration_nb_per_group_size_temp):
                        random.shuffle(shuffled_idx_list_COH[0])
                        idx_cst1 = np.copy(shuffled_idx_list_COH[0][0:group_sizi+1])
                        cst1 = np.nansum(binary_spike_trains_for_COH_analysis_temp[0][idx_cst1,:],axis=0)
                        if len(nb_motoneurons_temp) == 1: # Case with only one muscle (within-muscle coherence)
                            idx_cst2 = np.copy(shuffled_idx_list_COH[0][len(shuffled_idx_list_COH[0])-(group_sizi+1):len(shuffled_idx_list_COH[0])])
                            cst2 = np.nansum(binary_spike_trains_for_COH_analysis_temp[0][idx_cst2,:],axis=0)
                        else: # Case with two muscles (between-muscles coherence)
                            idx_cst2 = np.copy(shuffled_idx_list_COH[1][0:group_sizi+1])
                            cst2 = np.nansum(binary_spike_trains_for_COH_analysis_temp[1][idx_cst2,:],axis=0)
                        # If this the is the largest group size, compute the confidence interval with the surrogate spike trains method
                        if group_sizi == COH_calc_group_size_nb-1:
                            if not printed_shuffle_already:
                                printed_shuffle_already = True
                                print("     Most number of MUs: calculating CI by creating shuffled, surrogate spike trains...")
                            for shuffli in range(n_shuffles):
                                # apply to each MU-train
                                surrogate_trains_1 = np.array([
                                    shuffle_isi_spike_train(train) 
                                    for train in list(binary_spike_trains_for_COH_analysis_temp[0][idx_cst1,:])])
                                surrogate_trains_2 = np.array([
                                    shuffle_isi_spike_train(train) 
                                    for train in list(binary_spike_trains_for_COH_analysis_temp[0][idx_cst2,:])])
                                # sum across motor units
                                cst1_surr = np.nansum(surrogate_trains_1, axis=0)
                                cst2_surr = np.nansum(surrogate_trains_2, axis=0)
                                _, coh_cst_surr_output = COH_between_csts(cst1_surr, cst2_surr,
                                    fsamp=input_fsamp, coh_window_length=coh_window_length, coh_windows_overlap = coh_windows_overlap,
                                    max_freq = max_freq)
                                surrogate_COH_for_CI.append(coh_cst_surr_output.real) # keep only real part
                        # # # # #
                        _, coh_cst_output = COH_between_csts(cst1, cst2,
                                fsamp=input_fsamp, coh_window_length=coh_window_length, coh_windows_overlap = coh_windows_overlap,
                                max_freq = max_freq)
                        coherence_current_group_size.append(coh_cst_output)
                    # end of "for each iteration (within a group size)"
                    COH_pooled_per_group_size.append(np.nanmean(coherence_current_group_size, axis=0) )

                    # If this the is the largest group size, compute the confidence interval ##############
                    if group_sizi == COH_calc_group_size_nb-1:
                        # Standard, simple method (Muecli et al 2022)
                        L = len(cst1)
                        n_per_seg = coh_window_length * input_fsamp # get duration in samples instead of s
                        n_overlap = np.round(n_per_seg * coh_windows_overlap).astype(int) # get duration in samples instead of s
                        step = n_per_seg - n_overlap
                        N = int(np.floor((L - n_overlap) / step))
                        CL_simple = []
                        CL_surrogate = []
                        for alpha in alphas:
                            CL_simple.append(1 - (1 - alpha)**(1/(N - 1)))
                            # Surrogate (shuffled ISIs) method (https://pmc.ncbi.nlm.nih.gov/articles/PMC9223438/?utm_source=chatgpt.com Biomed Signal Proces Control 2022)
                            CL_surrogate.append(
                                np.percentile(surrogate_COH_for_CI,
                                (1-alpha)*100, axis=0)
                            )

                    # Add analysis results to csv - once for each MN group size
                    freq_iter = -1
                    muscle_label = str(muscle_current_COH) # Make sure the label is a string
                    for freq_i in freq_bins:
                        freq_iter += 1
                        # Create a new dictionary for this frequency bin
                        new_row = {
                            "subject": current_subject,
                            "condition": current_condition,
                            "muscles": muscle_label,
                            "frequency": freq_i,  # set the current frequency
                            "nb_of_MNs_in_group_for_COH_calculation": group_sizi+1 # +1 because of zero-indexing
                        }
                        new_row["coherence"] = COH_pooled_per_group_size[group_sizi][freq_iter].real
                        # if group_sizi >= 0: # start at 0 because of zero-indexing
                        #     new_row["coherence"] = COH_pooled_per_group_size[group_sizi][freq_iter].real
                        # else:
                        #     new_row["coherence"] = np.nan

                        # Write to the local CSV (for this subject + condition)
                        append_to_csv(csv_path, new_row, fieldnames)
                        # Also add to our global list for the big CSV
                        all_COH_results.append(new_row)
                    # end of "for each group size)"
                # Plot
                plt.subplot(1,nb_subplots,muscle_iter+1)
                # Assign correct color
                if isinstance(muscle_current_COH, str):
                    if ("Sim_parameters" in data[current_subject][current_condition].keys()) and (color_by_pool_or_by_condition == 'condition'):
                        figure_color = Find_color_for_sim_by_condition(current_subject, current_condition)
                    else:
                        figure_color = globals()[f"{muscle_current_COH}_color"]
                    muscle_label = muscle_current_COH
                    plt.title(f"{muscle_current_COH} - mean coherence\nMax MN group size = {COH_calc_group_size_nb}")
                else:
                    figure_color = mix_colors(globals()[f"{muscle_current_COH[0]}_color"],globals()[f"{muscle_current_COH[1]}_color"])
                    muscle_label = str(muscle_current_COH)
                    plt.title(f"{muscle_current_COH[0]} VS {muscle_current_COH[1]} - mean coherence\nMax MN group size = {COH_calc_group_size_nb}")
                for group_sizi in range(COH_calc_group_size_nb):
                    max_alpha = 0.35
                    min_alpha = 0.15
                    figure_alpha_temp = min_alpha + (group_sizi/COH_calc_group_size_nb)*(max_alpha-min_alpha)
                    # plt.plot(freq_bins, COH_pooled_per_group_size[group_sizi], color=figure_color, linewidth=2, alpha = figure_alpha_temp,
                    #          label = f"groups of {group_sizi+1} MNs")
                    # Highlight a specific MN nb (to help comparisons)
                    plt.plot(freq_bins, COH_pooled_per_group_size[group_sizi], color=figure_color, linewidth=1.5, alpha = figure_alpha_temp)
                    if group_sizi == 5:
                        plt.plot(freq_bins, COH_pooled_per_group_size[group_sizi],
                                 color='k', linewidth=1.5, alpha = 1, zorder = 5, linestyle='--',
                                 label="5 MNs")
                iter_alpha_thresh = -1
                for alpha in alphas:
                    iter_alpha_thresh += 1
                    opacity = (iter_alpha_thresh+1) / len(alphas)
                    # plt.axhline(y=CL_simple[iter_alpha_thresh],
                    #       linewidth=1.5, linestyle='--', color='red', alpha = opacity, zorder = 10,
                    #        label=f'{(1-alpha)*100}% CI (simple method)')
                    plt.plot(freq_bins, CL_surrogate[iter_alpha_thresh],
                            linewidth=1.5, linestyle='-', color='red', alpha = opacity, zorder=10,
                            label=f'{(1-alpha)*100}% CI (surrogate spike trains method)')
                plt.xlabel("Frequency (Hz)")
                plt.ylabel("Coherence")
                plt.title(f"Up to {COH_calc_group_size_nb} MNs")
                plt.ylim(0,1)
                plt.xlim(0,max_freq)
                plt.legend(loc="upper right")

            # End of "for each muscle or muscle pair"

            # Display and save figure
            plt.suptitle(f'Participant {current_subject}, {current_condition}')
            plt.tight_layout()
            if os.path.exists(folder_subject_condition):
                plt.savefig(os.path.join(folder_subject_condition, "Coherence_analysis_results.png"))
            else:
                print("Output folder does not exist; figure not saved")
            if display_figures_inline:
                plt.show()
            else:
                plt.close()
        # End of "for each condition"
    # End of "for each subject"

    # Now write ONE big CSV with all rows, saved at 'path_of_files' level
    big_csv_path = os.path.join(path_of_files, "Coherence_analysis_ALL.csv")
    write_big_csv(big_csv_path, all_COH_results, fieldnames)

    print(f"\nA concatenated CSV with all coherence results was saved at:\n{big_csv_path}")
else:
    print("Coherence analysis not performed, because 'perform_analysis_coherence=False'. Carrying on with the script...")

In [ ]:
def bandstop_sos(data, fs, lowcut, highcut, order=4):
    """
    Zero‑phase Butterworth band‑stop (notch) filter.
    
    Parameters
    ----------
    data : 1D array
        The signal to filter (e.g. your CST).
    fs : float
        Sampling rate in Hz.
    lowcut : float
        Lower edge of the stop band in Hz.
    highcut : float
        Upper edge of the stop band in Hz.
    order : int
        The filter order.
    
    Returns
    -------
    filtered : 1D array
        The band‑stopped signal.
    """
    nyq = fs / 2.0
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='bandstop', output='sos')
    return sosfiltfilt(sos, data)

In [ ]:
# # --------------------------------------------------- #
# # perform_analysis_frequency_components_dimensionality
# # --------------------------------------------------- #

if perform_analysis_frequency_components_dimensionality:

    # ────────────────────────────────────────────────────────────
    # 1) Set your parameters up front
    # ────────────────────────────────────────────────────────────
    fs        = input_fsamp        # e.g. 2 048 Hz
    lowpass_smoothing_of_wavelet_analysis_output = 1.0 # 60 # in hz
    min_freq  = 1                # lowest band (Hz)—warn if <1 Hz
    max_freq  = 60.0               # highest band (Hz)
    n_freqs   = 60                 # how many wavelet scales/bands
    wavelet   = 'morl'             # complex Morlet
    trim_cyc  = 3                  # how many cycles @ min_freq to cut off
    # cwt_clim  = None               # e.g. (vmin, vmax) or None to auto
    cut_cst_to     = None # 60.0              # seconds of CST to keep for fast testing; set to None to disable
    notch_filter_mean_firing_rate = False

    # ────────────────────────────────────────────────────────────
    # 2) Build your frequency vector & corresponding scales
    # ────────────────────────────────────────────────────────────
    if min_freq < 1.0:
        import warnings
        warnings.warn("Using min_freq < 1 Hz can be unstable", UserWarning)

    freqs = np.linspace(min_freq, max_freq, n_freqs)
    # center frequency of your wavelet
    cf = pywt.central_frequency(wavelet)  
    # scale-to-frequency: scale = cf * fs / freq
    scales = cf * fs / freqs

    # ────────────────────────────────────────────────────────────
    # 3) Compute the CWT
    # ────────────────────────────────────────────────────────────
    for current_subject in data.keys(): # ['MeJu']: # 
        for current_condition in data[current_subject].keys():
            # Create folder to save output (per subject, per condition)
            # Create folder to save output (per subject, per condition)
            folder_subject_condition_timefreq_analysis = os.path.join(path_of_files,current_subject,
                f'{current_subject}_{current_condition}','Time_Frequency_analysis' )
            if not os.path.exists(folder_subject_condition_timefreq_analysis):
                os.makedirs(folder_subject_condition_timefreq_analysis)
            print(f"     Time-frequency analysis for {current_subject} - {current_condition}")
            # Set all MUs as valid (no need to have a ISI cutoff for COH)
            valid_MUs = np.arange(len(data[current_subject][current_condition]['MU_muscle_list']))
            muscles_unique = list(set(data[current_subject][current_condition]['MU_muscle_list']))
            # Re-organize the binary spike trains from list to matrix (because this is the expected format) = already the case for experimental data, but needs to be done for simulated data
            if type(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'])==list:
                data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'] = np.array(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'])
            for current_muscle in muscles_unique:
                # if current_muscle != 'VL':
                #     continue # for testing
                
                color_of_plot = globals()[f"{current_muscle}_color"]
                # get numeric indices of MUs for this muscle
                MU_list = np.where(
                    np.array(data[current_subject][current_condition]['MU_muscle_list']) == current_muscle
                )[0]
                total_MUs = len(MU_list)
                if total_MUs == 0:
                    continue
                # build CST
                cst = np.nansum(
                    data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'][MU_list],
                    axis=0
                )
                cst = cst - np.nanmean(cst)

                if notch_filter_mean_firing_rate:
                    ### Compute mean DR of pool considered
                    # grab the list once
                    mpg = data[current_subject][current_condition]['muscle_per_grid']
                    # find all N such that mpg[N] == current_muscle
                    Ns = [i for i, m in enumerate(mpg) if m == current_muscle]
                    # now for each such N, extract the mean_firing_rates over all K in that sub‐dict
                    means_per_N = [
                        np.nanmean([
                            entry['mean_firing_rates']
                            for entry in data[current_subject][current_condition]['MU_properties'][N]
                        ])
                        for N in Ns
                    ]
                    DR_cov_per_N = [
                        1/np.nanmean([
                            entry['mean_cov']
                            for entry in data[current_subject][current_condition]['MU_properties'][N]
                        ])
                        for N in Ns
                    ]
                    overall_DR_mean = np.nanmean(means_per_N)
                    overall_DR_std = np.nanstd(means_per_N)
                    overall_DR_cov = np.nanmean(DR_cov_per_N)
                    # define your notch band
                    low_f  = overall_DR_mean - overall_DR_cov # overall_DR_std
                    high_f = overall_DR_mean + overall_DR_cov # overall_DR_std
                    # guard against invalid bands
                    low_f  = max(low_f, 0.1)           # can’t go below DC
                    high_f = min(high_f, input_fsamp/2 - 0.1)  # must stay below Nyquist
                    print(f"Notching CST around {low_f:.2f}–{high_f:.2f} Hz (mean ± std)")
                    # apply the notch filter to your mean‑centered CST
                    cst = bandstop_sos(cst, input_fsamp, low_f, high_f, order=4)

                # Make the CST way smaller, 
                plt.figure(figsize=(20,5))
                plt.plot(np.arange(len(cst))/input_fsamp, cst, color=color_of_plot, linewidth=0.2, alpha=0.5)
                plt.ylabel(f"CST amplitude (binary spikes)\nMean-centered")
                plt.xlabel("Time (s)")
                plt.title(f"{current_subject} - {current_condition} - {current_muscle} - CST")
                # === APPLY CUT FOR FAST TESTING ===
                if cut_cst_to is not None:
                    max_samp = int(cut_cst_to * fs)
                    cst      = cst[:max_samp]
                    plt.axvline(x=cut_cst_to, color='red', linestyle='--', label=f"Cut CST to {cut_cst_to} s")

                coeffs, frequencies = pywt.cwt(cst, scales, wavelet, sampling_period=1/fs)
                # coeffs shape = (n_freqs, n_timepoints)

                power = np.abs(coeffs)**2
                amps = np.abs(coeffs)

                # SMOOTH
                def lowpass_sos(amps, fs, cutoff, order=4):
                    """
                    Low-pass filter amplitude envelopes via SOS for smooth trends.
                    
                    Parameters:
                        amps (ndarray): shape (n_freqs, n_timepoints)
                        fs (float): sampling rate in Hz
                        cutoff (float): cutoff frequency in Hz
                        order (int): filter order
                    
                    Returns:
                        ndarray: low-pass filtered amps, same shape as input
                    """
                    nyq = fs / 2.0
                    norm_cut = cutoff / nyq
                    sos = butter(order, norm_cut, btype='low', output='sos')
                    return np.array([sosfiltfilt(sos, band) for band in amps])
            
                def highpass_sos(data, fs, cutoff=0.5, order=4):
                    """
                    Zero-phase high-pass filter to remove <cutoff Hz drift.
                    """
                    sos = butter(order, cutoff, btype='highpass', fs=fs, output='sos')
                    return sosfiltfilt(sos, data)
                
                amps = lowpass_sos(amps, fs, cutoff=lowpass_smoothing_of_wavelet_analysis_output) #max_freq, order=4)  # Smooth the amplitude envelopes
                # 1) High-pass each band’s envelope at 0.5 Hz
                amps = np.array([ highpass_sos(band, fs, cutoff=0.5) for band in amps ])

                # ────────────────────────────────────────────────────────────
                # 4) Trim the edge artifacts
                # ────────────────────────────────────────────────────────────
                trim_samp = int((trim_cyc/min_freq) * fs)
                power = power[:, trim_samp:-trim_samp]
                amps = amps[:, trim_samp:-trim_samp]
                t = np.arange(len(cst)) / fs
                t_trim = t[trim_samp:-trim_samp]

                # 2) Subtract the mean of each band (now near zero baseline)
                amps = amps - amps.mean(axis=1, keepdims=True)
                # 3) Shift upward to make everything ≥0 again (required by NMF)
                amps = np.array([ band - np.nanmin(band) for band in amps ])

                # ────────────────────────────────────────────────────────────
                # 5) Bring in your torque (no trimming)
                # ────────────────────────────────────────────────────────────
                plateau_idx = data[current_subject][current_condition]['selected_plateaus_samples_idx']
                torque = data[current_subject][current_condition]['force'][plateau_idx]
                # smooth the torque
                torque = lowpass_sos(torque[np.newaxis, :], fs, cutoff=lowpass_smoothing_of_wavelet_analysis_output).squeeze()  # Smooth the torque signal
                if cut_cst_to is not None:
                    # cut torque to match the CST length
                    torque = torque[:max_samp]
                # align torque by cropping the same amount on both ends if you want it to match
                torque_trim = torque[trim_samp:-trim_samp]

                # PLOT
                N = len(frequencies) # len(freqs)
                cmap = plt.get_cmap('plasma', N)
                # define boundaries so each freq gets its own color-bin
                # bounds = np.linspace(freqs.min(), freqs.max(), N+1)
                bounds = np.linspace(frequencies.min(), frequencies.max(), N+1)
                norm   = mpl.colors.BoundaryNorm(boundaries=bounds, ncolors=N)
                fig, ax = plt.subplots(figsize=(60, 6))
                ax2 = ax.twinx()  # for torque on the right
                # plot each band
                for i, freq in enumerate(frequencies): #enumerate(freqs):
                    color = cmap(i)
                    ax.plot(t_trim, amps[i], color=color, linewidth=1, alpha=0.3)
                # torque on secondary axis
                ax2.plot(t_trim, torque_trim, color=color_of_plot, linewidth=2, label='Torque', alpha = 0.7)
                # labels & title
                ax.set_xlabel("Time (s)")
                ax.set_ylabel("CWT amplitude (Morlet)")
                ax2.set_ylabel("Torque")
                fig.suptitle(f"{current_subject} – {current_condition} – {current_muscle} – CWT amplitude")
                # create an axis on the right for the colorbar
                divider = make_axes_locatable(ax)
                cax = divider.append_axes("right", size="5%", pad=0.1)
                sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
                sm.set_array([])
                cbar = fig.colorbar(
                    sm,
                    cax=cax,
                    boundaries=bounds,
                    ticks=frequencies # freqs
                )
                cbar.set_label("Frequency (Hz)")
                # legend for torque
                ax2.legend(loc='upper right')
                plt.tight_layout()
                plt.savefig(f'{folder_subject_condition_timefreq_analysis}/{current_muscle}_CWT_amplitude.png')
                plt.show()

                # TIME-FREQUENCY SPECTRUM PLOT
                # 1) Create figure and GridSpec
                fig = plt.figure(figsize=(60, 6))
                gs = GridSpec(2, 1, height_ratios=[3, 1], hspace=0.0)

                ax_wt = fig.add_subplot(gs[0, 0])
                ax_tq = fig.add_subplot(gs[1, 0], sharex=ax_wt)

                # 2) Plot your CWT image
                cwt_clim = (0, np.nanmax(amps))
                im = ax_wt.imshow(
                    amps,
                    aspect='auto',
                    origin='lower',
                    extent=[t_trim[0], t_trim[-1], freqs[0], freqs[-1]],
                    clim=cwt_clim,
                    cmap=cmr.rainforest,
                    interpolation=None
                )
                ax_wt.set_ylabel('Frequency (Hz)')
                ax_wt.set_title(f'{current_subject} – {current_condition} – {current_muscle}\nCWT power (Morlet)')

                # 3) Plot torque
                ax_tq.plot(t_trim, torque_trim, color=color_of_plot, linewidth=5, alpha=0.7)
                ax_tq.set_ylabel('Torque')
                ax_tq.set_xlabel('Time (s)')

                # 4) Manually add a colorbar axes at the right edge of ax_wt, without resizing either subplot
                pos = ax_wt.get_position()  # Bbox of ax_wt in figure coordinates
                cax = fig.add_axes([
                    pos.x1 + 0.005,   # slightly to the right of ax_wt
                    pos.y0,          # same bottom as ax_wt
                    0.02,            # width of the colorbar
                    pos.height       # same height as ax_wt
                ])
                fig.colorbar(im, cax=cax, label='Amplitude')
                plt.savefig(f'{folder_subject_condition_timefreq_analysis}/{current_muscle}_Time_Frequency_spectrum.png')
                plt.show()

                # === Assumptions ===
                # amp_trim: np.ndarray of shape (n_freqs, n_timepoints)
                # freqs: np.ndarray of length n_freqs
                # t_trim: np.ndarray of length n_timepoints
                # current_subject, current_condition, current_muscle defined

                # --- 1) Prepare data for NMF ---
                X = amps.T  # shape (n_timepoints, n_freqs)
                total_var = np.linalg.norm(X, 'fro')**2

                # --- 2) Compute explained variance for 1..10 components ---
                max_n = 3
                explained = np.zeros(max_n)
                for k in range(1, max_n + 1):
                    model = NMF(
                        n_components=k,
                        init='nndsvda',
                        random_state=0,
                        max_iter=1000,    # ← bump from 200 to 1 000 (or more)
                        tol=1e-4          # ← tighten or loosen the stopping tolerance
                    )
                    W = model.fit_transform(X)
                    H = model.components_
                    X_hat = W.dot(H)
                    explained[k-1] = 1 - (np.linalg.norm(X - X_hat, 'fro')**2 / total_var)

                # 3) Plot the **correct** variance‐explained curve
                plt.figure(figsize=(6,4))
                plt.plot(np.arange(0, max_n+1), np.insert(explained,0,0)*100, 'o-')
                plt.xlabel('Number of NMF Components')
                plt.ylabel('Variance Explained (%)')
                plt.title('NMF: Variance Explained vs. # Components')
                plt.grid(True)
                plt.tight_layout()
                plt.savefig(f'{folder_subject_condition_timefreq_analysis}/{current_muscle}_NMF_VAF.png')
                plt.show()

                # --- 4) 3D state-space of first 3 NMF components ---
                k3 = min(3, max_n)
                model3 = NMF(
                        n_components=k3,
                        init='nndsvda',
                        random_state=0,
                        max_iter=1000,    # ← bump from 200 to 1 000 (or more)
                        tol=1e-4          # ← tighten or loosen the stopping tolerance
                    )
                W3 = model3.fit_transform(X)  # shape (n_timepoints, k3)

                fig = plt.figure(figsize=(10, 10))
                ax = fig.add_subplot(111, projection='3d')
                # ax.plot(W3[:, 0], W3[:, 1], W3[:, 2], lw=1)
                ax.scatter3D(W3[:, 0], W3[:, 1], W3[:, 2], c=color_of_plot, s=10, alpha=0.002)
                ax.set_xlabel('Comp1')
                ax.set_ylabel('Comp2')
                ax.set_zlabel('Comp3')
                ax.set_title(f'{current_subject} – {current_condition} – {current_muscle}\nNMF First 3 Components Trajectory')
                plt.tight_layout()
                plt.savefig(f'{folder_subject_condition_timefreq_analysis}/{current_muscle}_NMF_3D.png')
                plt.show()

                # --- 5) Heatmap of NMF loadings for first 10 components ---
                modelN = NMF(
                        n_components=max_n,
                        init='nndsvda',
                        random_state=0,
                        max_iter=1000,    # ← bump from 200 to 1 000 (or more)
                        tol=1e-4          # ← tighten or loosen the stopping tolerance
                    )
                Wn = modelN.fit_transform(X)
                Hn = modelN.components_  # shape (max_n, n_freqs)

                # individual variance per component
                indiv_var = explained
                # labels = [f'Comp{i+1} ({indiv_var[i]*100:.1f}%)' for i in range(max_n)]
                labels = [f'Comp{i+1}' for i in range(max_n)]
                df_load_nmf = pd.DataFrame(Hn.T, index=freqs, columns=labels)

                plt.figure(figsize=(8, 6))
                sns.heatmap(df_load_nmf, cmap='viridis', yticklabels=5)
                plt.xlabel('NMF Components')
                plt.ylabel('Frequency (Hz)')
                plt.title(f'{current_subject} – {current_condition} – {current_muscle}\nNMF Loadings (3 factors) by Frequency Band')
                plt.tight_layout()
                plt.savefig(f'{folder_subject_condition_timefreq_analysis}/{current_muscle}_NMF_3comp_loadings_heatmap.png')
                plt.show()


In [ ]:
def plot_average_psd_vs_group_size(
    data,
    subject,
    condition,
    muscles_unique,
    input_fsamp,
    reps_base=100
):
    """
    For each muscle in `muscles_unique`, draw random groups of size k from its MU list
    (reps_base//k draws each), compute the summed-CST PSD (0–200Hz, normalized),
    average across draws, and plot all k‐curves together.
    """
    for current_muscle in muscles_unique:
        color_of_plot = globals()[f"{current_muscle}_color"]
        # get numeric indices of MUs for this muscle
        MU_list = np.where(
            np.array(data[subject][condition]['MU_muscle_list']) == current_muscle
        )[0]
        total_MUs = len(MU_list)
        if total_MUs == 0:
            continue

        ### Compute mean DR of pool considered
        # grab the list once
        mpg = data[current_subject][current_condition]['muscle_per_grid']
        # find all N such that mpg[N] == current_muscle
        Ns = [i for i, m in enumerate(mpg) if m == current_muscle]
        # now for each such N, extract the mean_firing_rates over all K in that sub‐dict
        means_per_N = [
            np.nanmean([
                entry['mean_firing_rates']
                for entry in data[current_subject][current_condition]['MU_properties'][N]
            ])
            for N in Ns
        ]
        overall_mean = np.nanmean(means_per_N)
        # print("Positions N matching current_muscle:", Ns)
        # print("Mean firing rates per N:", means_per_N)
        # print("Overall mean:", overall_mean)

        # prepare to collect mean‐PSDs
        all_avg_PSDs = []

        freqs = None

        # for each group‐size k = 1..total_MUs
        for k in range(1, total_MUs+1):
            n_reps = max(1, reps_base // k)
            psd_accum = []

            for _ in range(n_reps):
                # sample k distinct MUs
                chosen = np.random.choice(MU_list, size=k, replace=False)
                # build CST
                cst = np.nansum(
                    data[subject][condition]['binary_discharge_matrix_during_selected_plateaus'][chosen],
                    axis=0
                )
                cst = cst - np.nanmean(cst)

                # compute PSD
                nperseg = min(len(cst), input_fsamp)
                f, Pxx = welch(cst, fs=input_fsamp,
                               window='hann', nperseg=nperseg,
                               noverlap=np.round(nperseg/2).astype(int),
                               detrend='constant')
                # Pxx = Pxx / np.trapz(Pxx, f)

                # keep only up to 200 Hz
                mask200 = f <= 200
                if freqs is None:
                    freqs = f[mask200]
                psd_accum.append(Pxx[mask200])
                psd_accum[-1] = psd_accum[-1] / np.sum(psd_accum[-1])

            # average across this group‐size
            avg_psd = np.mean(psd_accum, axis=0)
            all_avg_PSDs.append((k, avg_psd))

        # Get the slopes at each frequencies for every increasing nb of MUs
        psd_matrix = np.vstack([avg for (k, avg) in all_avg_PSDs])
        delta_PSDs_for_increasing_nb_of_MUs =  np.diff(psd_matrix, axis=0)
        print(delta_PSDs_for_increasing_nb_of_MUs.shape)

        # now plot them
        # 1) compute slope-of-increase statistics
        mean_slope = delta_PSDs_for_increasing_nb_of_MUs.mean(axis=0)
        std_slope  = delta_PSDs_for_increasing_nb_of_MUs.std(axis=0)
        min_slope  = delta_PSDs_for_increasing_nb_of_MUs.min(axis=0)
        max_slope  = delta_PSDs_for_increasing_nb_of_MUs.max(axis=0)

        # 2) define zoom mask (0–5 Hz)
        zoom_mask = freqs <= 5
        freqs_zoom    = freqs[zoom_mask]
        psd_zoom_mat  = psd_matrix[:, zoom_mask]
        psd_zoom_max = np.max(np.max(psd_zoom_mat))
        slope_zoom    = mean_slope[zoom_mask]
        std_slope_z   = std_slope[zoom_mask]
        min_slope_z   = min_slope[zoom_mask]
        max_slope_z   = max_slope[zoom_mask]

        # 3) build figure with GridSpec
        fig = plt.figure(figsize=(16,10))
        gs  = gridspec.GridSpec(2, 2,
                                width_ratios =[4,1],
                                height_ratios=[1,1],
                                wspace=0.3, hspace=0.3)

        ax_psd       = fig.add_subplot(gs[0,0])
        ax_slope     = fig.add_subplot(gs[1,0]) #, sharex=ax_psd)
        ax_psd_zoom  = fig.add_subplot(gs[0,1]) #, sharey=ax_psd)
        ax_slope_zoom= fig.add_subplot(gs[1,1]) #, sharey=ax_slope)

        # 4) top-left: all PSD curves
        for k, avg in all_avg_PSDs:
            alpha = max(0.2, (k/psd_matrix.shape[0])*0.5)
            ax_psd.plot(freqs, avg,
                        color=color_of_plot,
                        alpha=alpha,
                        linewidth=1)
        # highlight a few key group‐sizes
        for k, ls in [(1,':'), (10,'--'), (30,'-')]:
            if len(psd_matrix)<=k-1:
                continue
            avg = psd_matrix[k-1]
            ax_psd.plot(freqs, avg,
                        color='k',
                        linestyle=ls,
                        linewidth=2,
                        label=f"{k} MU{'s' if k>1 else ''}")
        ax_psd.axvline(x=overall_mean, linestyle='--', color='r', label = f'Mean discharge rate ({overall_mean:.2f} hz)')
        ax_psd.set_xlim(0,200)
        ax_psd.set_ylabel("Normalized PSD")
        ax_psd.set_title("Normalized PSD as a function of #MUs")
        ax_psd.legend(loc='upper right', title="Group size")
        ax_psd.grid(True, ls='--', alpha=0.3)

        # 5) top-right: zoomed PSD
        for k, avg in all_avg_PSDs:
            alpha = max(0.2, (k/psd_matrix.shape[0])*0.5)
            ax_psd_zoom.plot(freqs_zoom, avg[zoom_mask],
                            color=color_of_plot,
                            alpha=alpha,
                            linewidth=1)
        # same highlights
        for k, ls in [(1,':'), (10,'--'), (psd_matrix.shape[0],'-')]:
            if len(psd_matrix)<=k-1:
                continue
            avg = psd_matrix[k-1]
            ax_psd_zoom.plot(freqs_zoom, avg[zoom_mask],
                            color='k', linestyle=ls, linewidth=2)
        ax_psd_zoom.set_xlim(0,5)
        # ax_psd_zoom.set_ylim(0,psd_zoom_max)
        ax_psd_zoom.set_xlabel("Frequency (Hz)")
        ax_psd_zoom.set_title("PSD (0–5 Hz)")
        ax_psd_zoom.grid(True, ls='--', alpha=0.3)

        # 6) bottom-left: mean slope ± std, min/max
        ax_slope.plot(freqs, mean_slope,
                    color=color_of_plot, lw=2, label="mean ΔPSD")
        # ax_slope.fill_between(freqs,
        #                     mean_slope - std_slope,
        #                     mean_slope + std_slope,
        #                     color=color_of_plot, alpha=0.3,
        #                     label="±1 SD")
        # ax_slope.plot(freqs, min_slope, ':', color=color_of_plot, alpha=0.5, label="min/max")
        # ax_slope.plot(freqs, max_slope, ':', color=color_of_plot, alpha=0.5)
        ax_slope.set_xlim(0,200)
        ax_slope.set_ylabel("ΔPSD per additional MU")
        ax_slope.set_xlabel("Frequency (Hz)")
        ax_slope.set_title("Mean rate of PSD increase (for each new MN in CST)")
        ax_slope.legend(loc='upper right', fontsize='small')
        ax_slope.axhline(y=0, color='k', linestyle='--')
        ax_slope.grid(True, ls='--', alpha=0.3)

        # 7) bottom-right: zoomed slope
        ax_slope_zoom.plot(freqs_zoom, slope_zoom,
                        color=color_of_plot, lw=2)
        # ax_slope_zoom.fill_between(freqs_zoom,
        #                         slope_zoom - std_slope_z,
        #                         slope_zoom + std_slope_z,
        #                         color=color_of_plot, alpha=0.3)
        # ax_slope_zoom.plot(freqs_zoom, min_slope_z, ':', color=color_of_plot, alpha=0.5)
        # ax_slope_zoom.plot(freqs_zoom, max_slope_z, ':', color=color_of_plot, alpha=0.5)
        ax_slope_zoom.set_xlim(0,5)
        ax_slope_zoom.set_xlabel("Frequency (Hz)")
        ax_slope.axhline(y=0, color='k', linestyle='--')
        ax_slope_zoom.set_title("ΔPSD zoom (0–5 Hz)")
        ax_slope_zoom.grid(True, ls='--', alpha=0.3)

        plt.suptitle(f"{subject}, {condition} – {current_muscle} ({total_MUs} MUs)")
        plt.tight_layout()
        if os.path.exists(folder_subject_condition):
            plt.savefig(os.path.join(folder_subject_condition, f"PSD_results_{current_muscle}.png"))
        else:
            print("Output folder does not exist; figure not saved")
        plt.show()


In [ ]:
# --------------------------------------
# COMPUTE POWER SPECTRUM DENSITY OF CST - FOR EACH MUSCLE
# --------------------------------------

# --------------------------------------
# Build dynamic fieldnames, depending on nb of frequency bins
# --------------------------------------
fieldnames = ["subject", "condition", "muscles","frequency","PSD","max_MU_nb"]
# This will collect all rows (from all subjects & conditions) so that
# we can write one "big CSV" at the end.
all_PSD_results = []

if perform_analysis_CST_PSD:
    print("Starting Power Spectrum Density of CST analysis")
    for current_subject in data.keys(): # ['MeJu']:
        for current_condition in data[current_subject].keys():
            # Create folder to save output (per subject, per condition)
            folder_subject_condition = os.path.join(path_of_files,current_subject,
                f'{current_subject}_{current_condition}' )
            if not os.path.exists(folder_subject_condition):
                os.makedirs(folder_subject_condition)
            print(f"     PSD analysis for {current_subject} - {current_condition}")
            # Overwrite local CSV => remove if it already exists
            csv_path = os.path.join(folder_subject_condition, "PSD_analysis_results.csv")
            if os.path.exists(csv_path):
                os.remove(csv_path)

            # valid_MUs = data[current_subject][current_condition]['valid_MUs_given_selected_plateaus']
            # Set all MUs as valid (no need to have a ISI cutoff for COH)
            valid_MUs = np.arange(len(data[current_subject][current_condition]['MU_muscle_list']))
            muscles_unique = list(set(data[current_subject][current_condition]['MU_muscle_list']))

            # Re-organize the binary spike trains from list to matrix (because this is the expected format) = already the case for experimental data, but needs to be done for simulated data
            if type(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'])==list:
                data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'] = np.array(data[current_subject][current_condition]['binary_discharge_matrix_during_selected_plateaus'])
            # — example usage —
            plot_average_psd_vs_group_size(
                data,
                current_subject,
                current_condition,
                muscles_unique,
                input_fsamp=2048,
                reps_base=100
            )

In [ ]:
# COMPUTE CROSS-CORRELATION BETWEEN CST AND TORQUE (new version)

if perform_analysis_CST_torque_correlation:

    #%% Helper functions
    def normalized_cross_correlation(x, y, max_lag):
        lags = np.arange(-max_lag, max_lag + 1, dtype=float)
        r = np.zeros(len(lags))
        for idx, lag in enumerate(lags):
            lag_int = int(lag)
            if lag_int < 0:
                overlap_x = x[-lag_int:]
                overlap_y = y[:len(x) + lag_int]
            elif lag_int > 0:
                overlap_x = x[:len(x) - lag_int]
                overlap_y = y[lag_int:]
            else:
                overlap_x = x
                overlap_y = y
            if len(overlap_x) > 1:
                r[idx] = np.corrcoef(overlap_x, overlap_y)[0, 1]
            else:
                r[idx] = 0
        return lags, r

    def create_binary_matrix_for_plateau_from_list(spike_times_list, mu_indices, window_start, window_end):
        """
        Create a binary matrix for the given window using the spike-times list.
        The spike_times_list is a list of arrays (one per motor unit), in the same order as MU_muscle_list.
        """
        L = int(window_end - window_start)
        binary_mat = np.zeros((len(mu_indices), L))
        for i, mu in enumerate(mu_indices):
            spikes = spike_times_list[mu]
            # If spikes is a list of arrays, concatenate them
            if isinstance(spikes, list):
                spikes = np.concatenate([np.asarray(s).ravel() for s in spikes])
            else:
                spikes = np.asarray(spikes).ravel()
            in_window = spikes[(spikes >= window_start) & (spikes < window_end)] - window_start
            indices = in_window.astype(int)
            indices = indices[(indices >= 0) & (indices < L)]
            binary_mat[i, indices] = 1
        return binary_mat

    #%% Main processing
    csv_rows = []
    correlation_info_dict = {}

    for current_subject in data.keys():
        print(f"Processing subject {current_subject}...")
        correlation_info_dict[current_subject] = {}
        subject_rows = []
        
        subject_folder = os.path.join(path_of_files, current_subject)
        os.makedirs(subject_folder, exist_ok=True)
        
        conditions = list(data[current_subject].keys())
        fig = plt.figure(figsize=(30, 8 * len(conditions)))
        gs = gridspec.GridSpec(len(conditions), 2, width_ratios=[4, 1], wspace=0.15)
        
        for i, current_condition in enumerate(conditions):
            print(f"  Processing condition {current_condition}...")
            correlation_info_dict[current_subject][current_condition] = {}
            
            folder_subject_condition = os.path.join(subject_folder, f"{current_subject}_{current_condition}")
            os.makedirs(folder_subject_condition, exist_ok=True)
            
            # Identify plateau segments
            plateau_samples_temp = data[current_subject][current_condition]['plateau_samples']
            plateau_separations = np.where(np.diff(plateau_samples_temp) > 1)[0]
            plateau_segments = []
            start_idx = 0
            for sep in plateau_separations:
                plateau_segments.append((plateau_samples_temp[start_idx], plateau_samples_temp[sep]))
                start_idx = sep + 1
            plateau_segments.append((plateau_samples_temp[start_idx], plateau_samples_temp[-1]))
            
            # Prepare subplots
            ax_signals = fig.add_subplot(gs[i, 0])
            ax_signals.set_title(f"{current_condition}: Torque and CST")
            ax_corr = fig.add_subplot(gs[i, 1])
            ax_corr.set_title(f"{current_condition}: Cross-correlation")
            ax_corr.set_xlabel("Time lag (s)")
            ax_corr.set_ylabel("Correlation coefficient")
            ax_corr.axvline(0, color='grey', linestyle='--')
            ax_corr.axhline(0, color='grey', linestyle='--')
            ax_corr.set_xlim(-0.5, 0.5)
            ax_corr.set_ylim(-0.5, 1)
            
            # 1) Apply force low-pass filter to the entire force signal once
            full_force = data[current_subject][current_condition]['force']
            force_filtered = lowpass_filter(full_force, fsamp=input_fsamp, 
                                            freq_cutoff=force_lowpass_cutoff, filter_order=4)
            
            # 2) Plot plateaus in a concatenated manner (no blank spaces)
            force_display_list = []
            time_display_list = []
            running_offset = 0.0
            
            for (p_start, p_end) in plateau_segments:
                ext_start = max(0, p_start - edges_to_remove_samples)
                ext_end = p_end + edges_to_remove_samples
                force_seg = force_filtered[ext_start:ext_end].copy()
                force_seg = detrend(force_seg)
                force_seg = force_seg / (np.std(force_seg) if np.std(force_seg) != 0 else 1)
                start_in_array = p_start - ext_start
                end_in_array = p_end - ext_start
                if end_in_array > len(force_seg):
                    end_in_array = len(force_seg)
                final_force = force_seg[start_in_array:end_in_array]
                L = len(final_force)
                t_array = np.arange(L) / input_fsamp + running_offset
                force_display_list.append(final_force)
                time_display_list.append(t_array)
                running_offset += L / input_fsamp
            
            force_concat = np.concatenate(force_display_list)
            t_concat = np.concatenate(time_display_list)
            ax_signals.plot(t_concat, force_concat, color='k', label='Torque', alpha=0.5, linewidth=1)
            
            # Add vertical lines for each plateau boundary (using concatenated time)
            running_time = 0.0
            for (p_start, p_end) in plateau_segments:
                plateau_length = p_end - p_start
                running_time += plateau_length / input_fsamp
                ax_signals.axvline(running_time, color='black', linestyle='--', linewidth=1)
            
            # 3) Process muscle combinations
            mu_muscle_list = data[current_subject][current_condition]['MU_muscle_list']
            unique_muscles = np.unique(mu_muscle_list)
            # Only add combined "all" if more than one unique muscle exists.
            if len(unique_muscles) > 1:
                muscle_combinations = list(unique_muscles) + ['all']
            else:
                muscle_combinations = list(unique_muscles)
            
            for m_idx, muscle in enumerate(muscle_combinations):
                if muscle != 'all':
                    indices = [idx for idx, m in enumerate(mu_muscle_list) if m == muscle]
                    muscle_label = muscle
                else:
                    indices = list(range(len(mu_muscle_list)))
                    # Combine the two unique muscle names for the 'all' case
                    muscle_label = f"{unique_muscles[0]}-{unique_muscles[1]}" if len(unique_muscles) > 1 else unique_muscles[0]
                
                correlation_info_dict[current_subject][current_condition][muscle_label] = {}
                
                # Determine figure color adaptively
                if muscle != 'all':
                    figure_color = globals()[f"{muscle}_color"]
                else:
                    if len(unique_muscles) >= 2:
                        color1 = globals().get(f"{unique_muscles[0]}_color", (0,0,1))
                        color2 = globals().get(f"{unique_muscles[1]}_color", (1,0,0))
                        figure_color = mix_colors(color1, color2)
                    else:
                        figure_color = globals().get(f"{unique_muscles[0]}_color", (0,0,1))
                
                # Use MU_spike_times_list
                spike_times_list = data[current_subject][current_condition]['MU_spike_times_list']
                mu_indices = ([int(mu) for mu, m in enumerate(mu_muscle_list) if m == muscle]
                              if muscle != 'all'
                              else list(range(len(mu_muscle_list))))
                n_MUs = len(mu_indices)
                
                print(f"    Processing muscle {muscle_label} (n_MUs={n_MUs}) for subject {current_subject}, condition {current_condition}...")
                
                plateau_results = []
                cst_display_list = []
                cst_time_list = []
                running_offset_cst = 0.0
                
                for p_idx, (p_start, p_end) in enumerate(plateau_segments):
                    ext_start = max(0, p_start - edges_to_remove_samples)
                    ext_end = p_end + edges_to_remove_samples
                    force_seg = force_filtered[ext_start:ext_end].copy()
                    force_seg = detrend(force_seg)
                    force_seg = force_seg / (np.std(force_seg) if np.std(force_seg) != 0 else 1)
                    
                    binary_mat = create_binary_matrix_for_plateau_from_list(spike_times_list, mu_indices, ext_start, ext_end)
                    CST_seg = np.nansum(binary_mat, axis=0)
                    
                    plateau_freq_results = {}
                    for lowpass_cutoff_i in low_pass_freqs_to_try:
                        print(f"      Subject={current_subject}, cond={current_condition}, muscle={muscle_label}, cutoff={lowpass_cutoff_i} Hz, plateau={p_idx+1}")
                        start_in_array = p_start - ext_start
                        end_in_array = p_end - ext_start
                        smoothed_CST = lowpass_filter(CST_seg, fsamp=input_fsamp, 
                                freq_cutoff=lowpass_cutoff_i, filter_order=4)
                        if end_in_array > len(smoothed_CST):
                            end_in_array = len(smoothed_CST)
                        final_cst = smoothed_CST[start_in_array:end_in_array]
                        final_cst = detrend(final_cst)
                        final_cst = final_cst / (np.std(final_cst) if np.std(final_cst) != 0 else 1)
                        final_force = force_seg[start_in_array:end_in_array]
                        
                        lags_samples, corr_coeff = normalized_cross_correlation(final_force, final_cst, cross_correl_max_lag)
                        lags = lags_samples / float(input_fsamp)
                        max_idx = np.argmax(corr_coeff)
                        best_corr = corr_coeff[max_idx]
                        best_lag = lags[max_idx]
                        
                        plateau_freq_results[lowpass_cutoff_i] = {
                            'max_correlation': best_corr,
                            'time_delay': best_lag,
                            'lag_vector': lags,
                            'correlation_vector': corr_coeff,
                            'smoothed_CST': final_cst,
                            'force': final_force,
                            'n_MUs': n_MUs
                        }
                        
                        subject_rows.append({
                            'subject': current_subject,
                            'condition': current_condition,
                            'muscle': muscle_label,
                            'plateau': p_idx+1,
                            'n_MUs': n_MUs,
                            'low_pass_frequency': lowpass_cutoff_i,
                            'max_correlation': best_corr,
                            'time_delay': best_lag
                        })
                    plateau_results.append(plateau_freq_results)
                    
                    # For plotting the CST at the arbitrary frequency
                    final_cst_arbitrary = plateau_freq_results[arbitrary_lowpass_freq]['smoothed_CST']
                    L_cst = len(final_cst_arbitrary)
                    t_cst = np.arange(L_cst) / input_fsamp + running_offset_cst
                    cst_display_list.append(final_cst_arbitrary)
                    cst_time_list.append(t_cst)
                    running_offset_cst += L_cst / input_fsamp
                
                # Average across plateaus for each frequency
                avg_results = {}
                for lowpass_cutoff_i in low_pass_freqs_to_try:
                    corr_list = np.array([pr[lowpass_cutoff_i]['correlation_vector'] for pr in plateau_results])
                    avg_corr = np.mean(corr_list, axis=0)
                    cst_list = [pr[lowpass_cutoff_i]['smoothed_CST'] for pr in plateau_results]
                    min_len = min(len(x) for x in cst_list)
                    trimmed_cst = [x[:min_len] for x in cst_list]
                    cst_array = np.array(trimmed_cst)
                    avg_smoothed = np.mean(cst_array, axis=0)
                    
                    avg_lags = plateau_results[0][lowpass_cutoff_i]['lag_vector']
                    max_idx = np.argmax(avg_corr)
                    best_corr_avg = avg_corr[max_idx]
                    best_lag_avg = avg_lags[max_idx]
                    
                    avg_results[lowpass_cutoff_i] = {
                        'max_correlation': best_corr_avg,
                        'time_delay': best_lag_avg,
                        'lag_vector': avg_lags,
                        'correlation_vector': avg_corr,
                        'smoothed_CST': avg_smoothed,
                        'n_MUs': n_MUs
                    }
                    subject_rows.append({
                        'subject': current_subject,
                        'condition': current_condition,
                        'muscle': muscle_label,
                        'plateau': 0,
                        'n_MUs': n_MUs,
                        'low_pass_frequency': lowpass_cutoff_i,
                        'max_correlation': best_corr_avg,
                        'time_delay': best_lag_avg
                    })
                
                correlation_info_dict[current_subject][current_condition][muscle_label] = {
                    'per_plateau': plateau_results,
                    'average': avg_results
                }
                
                # Plot if muscle is selected
                if CST_torque_corr_display_only_selected_muscles == 'all' or muscle_label in CST_torque_corr_display_only_selected_muscles:
                    cst_concat = np.concatenate(cst_display_list)
                    t_cst_concat = np.concatenate(cst_time_list)
                    
                    ax_signals.plot(t_cst_concat, cst_concat, color=figure_color, linestyle='-', linewidth=2,
                                    alpha=0.8, label=f'{muscle_label} CST (nMUs={n_MUs}, {arbitrary_lowpass_freq} Hz)')
                    
                    for p_idx, pr in enumerate(plateau_results):
                        ax_corr.plot(pr[arbitrary_lowpass_freq]['lag_vector'],
                                     pr[arbitrary_lowpass_freq]['correlation_vector'],
                                     color=figure_color, linestyle='-', linewidth=1, alpha=0.3)
                    avg_corr = avg_results[arbitrary_lowpass_freq]['correlation_vector']
                    avg_lags = avg_results[arbitrary_lowpass_freq]['lag_vector']
                    ax_corr.plot(avg_lags, avg_corr, color=figure_color, linestyle='-', linewidth=3, alpha=1,
                                 label=f'{muscle_label} avg CC (nMUs={n_MUs}, {arbitrary_lowpass_freq} Hz)')
                    max_idx = np.argmax(avg_corr)
                    best_corr_avg = avg_corr[max_idx]
                    best_lag_avg = avg_lags[max_idx]
                    ax_corr.plot(best_lag_avg, best_corr_avg, 'ro')
                    ax_corr.axhline(best_corr_avg, color='r', linestyle=':')
                    ax_corr.annotate('', xy=(0, best_corr_avg), xytext=(best_lag_avg, best_corr_avg),
                                     arrowprops=dict(arrowstyle='<->', color='black', lw=1, alpha=0.85))
                    
                    total_muscles = len(muscle_combinations)
                    y_pos = 1 - (m_idx + 1) / (total_muscles + 1)
                    annot_text = f"{muscle_label}\nr = {best_corr_avg:.2f}\ndelay = {best_lag_avg:.3f}s"
                    ax_corr.text(0.98, y_pos, annot_text, transform=ax_corr.transAxes,
                                ha='right', va='top', color=figure_color,
                                bbox=dict(facecolor='white', edgecolor='none', pad=2))
            
            ax_signals.set_xlabel("Time (s)")
            ax_signals.set_ylabel("a.u.")
            ax_signals.legend(fontsize='small')
            ax_corr.legend(fontsize='small')
        
        fig.suptitle(f"{current_subject} - Torque and CST correlation", fontsize=16)
        fig.tight_layout(rect=[0, 0, 1, 0.96])
        fig_path = os.path.join(subject_folder, f"{current_subject}_Torque_CST_correlation.png")
        fig.savefig(fig_path, dpi=300)
        if display_figures_inline:
            plt.show()
        else:
            plt.close()
        
        # Save per-subject CSV
        df_subject = pd.DataFrame(subject_rows)
        csv_path = os.path.join(subject_folder, f"{current_subject}_Correl_CST_torque.csv")
        df_subject.to_csv(csv_path, index=False)
        print(f"Saved CSV for subject {current_subject} to {csv_path}")

    # Build global CSV
    rows = []
    for subject in correlation_info_dict:
        for condition in correlation_info_dict[subject]:
            for muscle in correlation_info_dict[subject][condition]:
                per_plateau = correlation_info_dict[subject][condition][muscle]['per_plateau']
                for p_idx, plateau_results in enumerate(per_plateau):
                    for freq in plateau_results:
                        info = plateau_results[freq]
                        rows.append({
                            'subject': subject,
                            'condition': condition,
                            'muscle': muscle,
                            'plateau': p_idx+1,
                            'n_MUs': info.get('n_MUs', 0),
                            'low_pass_frequency': freq,
                            'max_correlation': info['max_correlation'],
                            'time_delay': info['time_delay']
                        })
                avg = correlation_info_dict[subject][condition][muscle]['average']
                for freq in avg:
                    info = avg[freq]
                    rows.append({
                        'subject': subject,
                        'condition': condition,
                        'muscle': muscle,
                        'plateau': 0,
                        'n_MUs': info.get('n_MUs', 0),
                        'low_pass_frequency': freq,
                        'max_correlation': info['max_correlation'],
                        'time_delay': info['time_delay']
                    })

    df_global = pd.DataFrame(rows)
    global_csv_path = os.path.join(path_of_files, "Global_Correl_CST_torque.csv")
    df_global.to_csv(global_csv_path, index=False)
    print(f"Global CSV saved to {global_csv_path}")
